In [4]:
import os

def get_filename_without_extension(file_path):
    # Get the base name of the file
    base_name = os.path.basename(file_path)
    # Split the base name into name and extension and return only the name part
    return os.path.splitext(base_name)[0]

def process_lean_files(directory):
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.lean'):
                file_path = os.path.join(root, filename)
                process_lean_file(file_path)

def process_lean_file(file_path):
    print("Processing", file_path)
    with open(file_path, 'r') as file:
        lines = file.readlines()

    file_name = get_filename_without_extension(file_path)
    example_count = 1  # Initialize example counter
    modified_lines = []
    for line in lines:
        if line.startswith("example "):
            modified_lines.append(f"theorem {file_name}_ex{example_count} " + line[len("example "):])
            example_count += 1
        else:
            modified_lines.append(line)

    with open(file_path, 'w') as file:
        file.writelines(modified_lines)

# Example usage:
process_lean_files('/home/mcwave/code/math-in-lean/MIL')


Processing /home/mcwave/code/math-in-lean/MIL/Common.lean
Processing /home/mcwave/code/math-in-lean/MIL/C01_Introduction/S02_Overview.lean
Processing /home/mcwave/code/math-in-lean/MIL/C01_Introduction/S01_Getting_Started.lean
Processing /home/mcwave/code/math-in-lean/MIL/C01_Introduction/solutions/Solutions_S02_Overview.lean
Processing /home/mcwave/code/math-in-lean/MIL/C01_Introduction/solutions/Solutions_S01_Getting_Started.lean
Processing /home/mcwave/code/math-in-lean/MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean
Processing /home/mcwave/code/math-in-lean/MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean
Processing /home/mcwave/code/math-in-lean/MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean
Processing /home/mcwave/code/math-in-lean/MIL/C05_Elementary_Number_Theory/solutions/Solutions_S01_Irrational_Roots.lean
Processing /home/mcwave/code/math-in-lean/MIL/C05_Elementary_Number_Theory/solutions/Solutions_S03_Infinitely_Many_Primes.lean


In [58]:
def classify_lean_element(line):
    VARIABLE_TYPES = ['ℝ']
    
    # Clean up the line
    line = line.strip()
    if not line:
        return None, None, None

    # Split the line around the ':'
    parts = line.split(':', 1)
    if len(parts) != 2:
        return None, None, None  # Skip malformed lines

    # Clean up names and types
    name_part, type_info = parts[0].strip(), parts[1].strip()

    if type_info.startswith('Type'):
        return 'type', name_part.split(), type_info
    
    if type_info.startswith('Set'):
        return 'set', name_part.split(), type_info
    
    if type_info.startswith(('∀', '∃', '¬')):
        return 'hypothesis', name_part.split(), type_info
    
    if type_info in VARIABLE_TYPES:
        return 'variable', name_part.split(), type_info
    
    # Check for function definitions or lambda functions
    if '→' in type_info or 'fun' in type_info or ':=' in type_info:
        # Check if it's a hypothesis disguised as a function (look for logical operators)
        if '∀' in type_info or '∃' in type_info:
            return 'hypothesis', name_part.split(), type_info
        return 'function', name_part.split(), type_info

    # Check for propositions or logical statements involving quantifiers
    if 'Prop' in type_info or '¬' in type_info or '∈' in type_info or '∉' in type_info:
        return 'hypothesis', name_part.split(), type_info

    # Handle multiple variables on the same line
    names = name_part.split()
    return 'unknown', names, type_info

def classify_lean_elements(lean_state):
    category_of_name = {}
    type_info_of_name = {}
    lines = lean_state.strip().split('\n')
    for line in lines:
        category, names, type_info = classify_lean_element(line)
        if category is None or names is None:
            continue
        for name in names:
            category_of_name[name] = category
            type_info_of_name[name] = type_info
    
    names = list(category_of_name.keys())
    for name in names:
        if category_of_name[name] == 'unknown' and\
            type_info_of_name[name] in names and\
            category_of_name[type_info_of_name[name]] == 'type':
            category_of_name[name] = 'variable'

    return category_of_name, type_info_of_name

# Example usage
lean_state = """
case h₁
f g : ℝ → ℝ
⊢ f x ≤ a

α : Type u_1
P : α → Prop
Q : Prop
h : ¬∀ (x : α), P x
h' : ¬∃ x, ¬P x
x : α
h'' : ¬P x
a b : ℝ
f✝ : ℝ → ℝ
h1 : ∀ {f : ℝ → ℝ}, Monotone f → ∀ {a b : ℝ}, f a ≤ f b → a ≤ b
f : ℝ → ℝ := fun x => 0
α1 : Type u_1
s t : Set α
xstu : x ∈ (s \ t) \ u
xs : x ∈ s
xnt : x ∉ t
xnu : x ∉ u
xu : x ∈ u
ubf : FnHasUb f
ι : Type u_1
R : Type u_2
inst✝ : CommRing R
I : Ideal R
J : ι → Ideal R
i : ι
hs : (∀ j ∈ s, I + J j = 1) → I + ⨅ j ∈ s, J j = 1
hf : ∀ j ∈ insert i s, I + J j = 1
inst✝ : CommRing R
"""

category_of_name, _ = classify_lean_elements(lean_state)
for name, category in category_of_name.items():
    print(name, category)


f function
g function
α type
P function
Q hypothesis
h hypothesis
h' hypothesis
x variable
h'' hypothesis
a variable
b variable
f✝ function
h1 hypothesis
α1 type
s set
t set
xstu hypothesis
xs hypothesis
xnt hypothesis
xnu hypothesis
xu hypothesis
ubf unknown
ι type
R type
inst✝ unknown
I unknown
J function
i variable
hs hypothesis
hf hypothesis


In [59]:
import re

def tokenize_lean_tactic(tactic):
    # Define the regex patterns
    variable_pattern = r'[a-zA-Z_\'\u03b1-\u03c9][a-zA-Z_\'\u03b1-\u03c9\d₀-₉]*'
    operator_pattern = r'[:=<>∈∉≤≥+\-*/^∀∃¬⁻¹]+'
    punctuation_pattern = r'[,⟨⟩\[\](){}|]'
    comment_pattern = r'--.*'

    # Remove comments from the tactic
    tactic = re.sub(comment_pattern, '', tactic)

    # Tokenize the tactic
    tokens = []
    while tactic:
        # Check for variables
        match = re.match(variable_pattern, tactic)
        if match:
            tokens.append(match.group())
            tactic = tactic[match.end():]
            continue

        # Check for operators
        match = re.match(operator_pattern, tactic)
        if match:
            tokens.append(match.group())
            tactic = tactic[match.end():]
            continue

        # Check for punctuation
        match = re.match(punctuation_pattern, tactic)
        if match:
            tokens.append(match.group())
            tactic = tactic[match.end():]
            continue

        # Add remaining characters as separate tokens
        tokens.append(tactic[0])
        tactic = tactic[1:]

    return tokens

test_cases = [
    "intro h''",
    "rintro y ⟨x, ⟨xs, xt⟩, rfl⟩",
    "x = exp (log x) := by rw [exp_log xpos]",
    "have ε2pos : 0 < ε / 2 := by linarith",
    "rw [← h1 fx₂eq]",
    "rintro y ⟨⟨x₁, x₁s, rfl⟩, ⟨x₂, x₂t, fx₂eq⟩⟩",
    "use y*h1*y⁻¹",
    "have : j ∉ f j := by rwa [h2] at h'",
    "rw [mem_map] at * -- Lean does not need this line",
    "have h₀ : 0 < 1 + exp a := by linarith [exp_pos a]",
    "_ ≤ |s n - a| + |a| := (abs_add _)_",
    "|a - b| = |(-(s N - a)), + (s N - b)| := by",
    "∃ a, FnLb f a", 
    "¬¬∀ x, a ≤ f x",
    "s ∩ t ∪ s ∩ u ⊆ s ∩ (t ∪ u)"
]

expected_outputs = [
    ['intro', "h''"],
    ['rintro', 'y', '⟨', 'x', ',', '⟨', 'xs', ',', 'xt', '⟩', ',', 'rfl', '⟩'],
    ['x', '=', 'exp', '(', 'log', 'x', ')', ':=', 'by', 'rw', '[', 'exp_log', 'xpos', ']'],
    ['have', 'ε2pos', ':', '0', '<', 'ε', '/', '2', ':=', 'by', 'linarith'],
    ['rw', '[', '←', 'h1', 'fx₂eq', ']'],
    ['rintro', 'y', '⟨', '⟨', 'x₁', ',', 'x₁s', ',', 'rfl', '⟩', ',', '⟨', 'x₂', ',', 'x₂t', ',', 'fx₂eq', '⟩', '⟩'],
    ['use', 'y', '*', 'h1', '*', 'y', '⁻', '¹'],
    ['have', ':', 'j', '∉', 'f', 'j', ':=', 'by', 'rwa', '[', 'h2', ']', 'at', "h'"],
    ['rw', '[', 'mem_map', ']', 'at', '*'],
    ['have', 'h₀', ':', '0', '<', '1', '+', 'exp', 'a', ':=', 'by', 'linarith', '[', 'exp_pos', 'a', ']'],
    ['_', '≤', '|', 's', 'n', '-', 'a', '|', '+', '|', 'a', '|', ':=', '(', 'abs_add', '_', ')', '_'],
    ['|', 'a', '-', 'b', '|', '=', '|', '(', '-', '(', 's', 'N', '-', 'a', ')', ')', ',', '+', '(', 's', 'N', '-', 'b', ')', '|', ':=', 'by'],
    ['∃', 'a', ',', 'FnLb', 'f', 'a'],
    ['¬', '¬', '∀', 'x', ',', 'a', '≤', 'f', 'x'],
    ['s', '∩', 't', '∪', 's', '∩', 'u', '⊆', 's', '∩', '(', 't', '∪', 'u', ')']
]

num_mismatch = 0
for i in range(len(test_cases)):
    output = tokenize_lean_tactic(test_cases[i])
    output = [x for x in output if x != ' ']
    if output != expected_outputs[i]:
        print("Case",i,"mismatches:",test_cases[i])
        print("Expected:", expected_outputs[i])
        print("Actual:  ", output)
        num_mismatch += 1
        
print(num_mismatch, "mismatches")

Case 6 mismatches: use y*h1*y⁻¹
Expected: ['use', 'y', '*', 'h1', '*', 'y', '⁻', '¹']
Actual:   ['use', 'y', '*', 'h1', '*', 'y', '⁻¹']
Case 13 mismatches: ¬¬∀ x, a ≤ f x
Expected: ['¬', '¬', '∀', 'x', ',', 'a', '≤', 'f', 'x']
Actual:   ['¬¬∀', 'x', ',', 'a', '≤', 'f', 'x']
2 mismatches


In [63]:
import re

def transform_bracket_contents(contents, definition_dict, bracket='[]'):
    # Split the contents within the brackets by commas and strip spaces
    items = map(str.strip, contents.split(","))
    transformed = []
    for item in items:
        if item in definition_dict:
            transformed.append("{" + definition_dict[item] + "}")
        else:
            transformed.append(item)
    return bracket[0] + ", ".join(transformed) + bracket[-1]

def parse_lean_state_and_tactic(state_before, tactic, custom_defs=None):

    # Step 1: Parse the state_before to create a dictionary of definitions
    definition_dict, _ = classify_lean_elements(state_before)
    print(definition_dict)
    #     definition_dict = {}
    #     lines = state_before.split("\n")
    #     for line in lines:
    #         line = line.strip()
    #         if ":" in line and not line.startswith("⊢"):  # ignore the goal line
    #             name_type_pair = line.split(" : ")
    #             name = name_type_pair[0].strip()
    #             type_def = name_type_pair[1].strip()
    #             definition_dict[name] = type_def

    # Step 2: Parse the tactic
    tactic_parts = tokenize_lean_tactic(tactic)
    if len(tactic_parts) == 1:  # tactic without arguments
        return tactic  # no transformation needed

    # Step 3: Check and transform arguments
    transformed_parts = []
    for tp in tactic_parts:
        if tp in definition_dict:
            transformed_parts.append('{' + definition_dict[tp] + '}')
        else:
            transformed_parts.append(tp)
#     if "[" in arguments:  # Handle bracketed arguments like in 'rw [ef, eg]'
#         transformed_arguments = re.sub(r'\[(.*?)\]', lambda m: transform_bracket_contents(m.group(1), definition_dict, '[]'), arguments)
#     elif "⟨" in arguments:  # Handle bracketed arguments like in 'rintro y ⟨x, ⟨xs, xt⟩, rfl⟩'
#         transformed_arguments = re.sub(r'\⟨(.*?)\⟩', lambda m: transform_bracket_contents(m.group(1), definition_dict, '⟨⟩'), arguments)
#     else:  # Handle single or space-separated arguments like in 'apply hfa'
#         transformed_arguments = " ".join("{" + definition_dict[arg] + "}" if arg in definition_dict else arg for arg in arguments.split())

#     # Join the transformed arguments with the keyword
#     transformed_tactic = keyword + " " + transformed_arguments
    return ''.join(transformed_parts)


# Example usage for both scenarios
state_before = """
case h₁
f g : ℝ → ℝ
a b : ℝ
hfa : FnUb f a
hgb : FnUb g b
x : ℝ
⊢ f x ≤ a

case h₂
f g : ℝ → ℝ
a b : ℝ
hfa : FnUb f a
hgb : FnUb g b
x : ℝ
⊢ g x ≤ b
"""
tactic = "apply hfa x"
print(parse_lean_state_and_tactic(state_before, tactic))  # should be "apply {FnUb f a}"

state_before = """
f g : ℝ → ℝ
ef : FnEven f
ε2pos : FnEven g
x : ℝ
⊢ f x + g x = f (-x) + g (-x)
"""
tactic = "rw [f, eg]"
print(parse_lean_state_and_tactic(state_before, tactic))  # should be "rw [{FnEven f}, {FnEven g}]"


state_before = """
f g : ℝ → ℝ
xs : ¬∀ (x : α), P x
xt : ¬∃ x, ¬P x
x : ℝ
y : ℝ
⊢ f x + g x = f (-x) + g (-x)
"""
tactic = "rintro y ⟨x, ⟨xs,xt⟩, rfl⟩"
print(parse_lean_state_and_tactic(state_before, tactic))  # should be "rw [{FnEven f}, {FnEven g}]"



{'f': 'function', 'g': 'function', 'a': 'variable', 'b': 'variable', 'hfa': 'unknown', 'hgb': 'unknown', 'x': 'variable'}
apply {unknown} {variable}
{'f': 'function', 'g': 'function', 'ef': 'unknown', 'ε2pos': 'unknown', 'x': 'variable'}
rw [{function}, eg]
{'f': 'function', 'g': 'function', 'xs': 'hypothesis', 'xt': 'hypothesis', 'x': 'variable', 'y': 'variable'}
rintro {variable} ⟨{variable}, ⟨{hypothesis},{hypothesis}⟩, rfl⟩


In [2]:
from utils.constants import *

tac_templates = [('simp', 2407),
 ('rfl', 1306),
 ('constructor', 768),
 ('simp [{unknown}]', 620),
 ('rw [{unknown}]', 534),
 ('· simp', 449),
 ('ext', 441),
 ('cases {unknown}', 374),
 ('exact {unknown}', 370),
 ('congr', 332),
 ('intro h', 325),
 ('ring', 253),
 ('subst {unknown}', 244),
 ('decide', 235),
 ('ext x', 230),
 ('rintro rfl', 224),
 ('linarith', 207),
 ('norm_num', 191),
 ('intro x hx', 177),
 ('apply {hypothesis}', 172),
 ('rw [← {unknown}]', 170),
 ('gcongr', 169),
 ('simp [{hypothesis}]', 155),
 ('simpa using {unknown}', 155),
 ('assumption', 154),
 ('contradiction', 148),
 ('exact {hypothesis}', 143),
 ('· rfl', 142),
 ('intro x', 139),
 ('· simp [{hypothesis}]', 135),
 ('simpa', 132),
 ('congr 1', 125),
 ('apply {unknown}', 117),
 ('simp at {unknown}', 117),
 ('norm_cast', 114),
 ('dsimp', 113),
 ('simpa using {hypothesis}', 107),
 ('contrapose! {unknown}', 105),
 ('swap', 105),
 ('apply le_antisymm', 99),
 ('positivity', 98),
 ('intro n', 98),
 ('contrapose! {hypothesis}', 94),
 ('intros', 92),
 ('right', 90),
 ('rw [{unknown}] at {unknown}', 88),
 ('left', 85),
 ('split', 83),
 ('symm', 81),
 ('aesop', 79),
 ('· exact {unknown}', 79),
 ('infer_instance', 79),
 ('abel', 79),
 ('intro', 78),
 ("induction' {unknown} with {unknown} ih", 78),
 ('nontriviality {type}', 78),
 ('tauto', 75),
 ('push_neg at {hypothesis}', 75),
 ('dsimp only', 73),
 ('split_ifs with h', 73),
 ('intro i', 70),
 ('push_cast', 70),
 ('rw [{function}]', 69),
 ('cases nonempty_fintype {type}', 69),
 ('ext {variable}', 67),
 ('simp_all', 66),
 ('rw [mul_comm]', 64),
 ('use {variable}', 63),
 ('intro H', 61),
 ('ext i', 60),
 ('use {unknown}', 60),
 ('simp [{unknown}, {unknown}]', 58),
 ('constructor <;> intro h', 58),
 ('simp [*]', 58),
 ('· simp [{unknown}]', 57),
 ("cases' {unknown} with {unknown}", 57),
 ('exact {hypothesis} {unknown}', 53),
 ('field_simp', 53),
 ('intro {unknown}', 52),
 ('exfalso', 50),
 ('intro y hy', 50),
 ('intro a', 49),
 ('ext1 x', 49),
 ('rw [{unknown}, {unknown}]', 47),
 ('cases {unknown} <;> rfl', 46),
 ('revert {unknown}', 45),
 ('convert {unknown}', 45),
 ('aesop_cat', 45),
 ('by_contra h', 44),
 ('split_ifs', 44),
 ('rw [add_comm]', 44),
 ('rwa [{unknown}]', 43),
 ('intro i hi', 43),
 ('subst_vars', 43),
 ('ext i j', 42),
 ('ext1', 40),
 ('ext y', 40),
 ('ac_rfl', 39),
 ('exact {unknown} {unknown}', 37),
 ('simp only', 36),
 ('rw [{hypothesis}]', 35),
 ('clear {unknown}', 34),
 ('ext {unknown}', 33),
 ('by_cases h : {variable} = {variable}', 33),
 ('borelize {type}', 33),
 ('ext j', 32),
 ('exact {hypothesis} {hypothesis}', 32),
 ("intro h'", 31),
 ('ext a', 31),
 ('intro s hs', 31),
 ('exact {function}', 30),
 ('simp at {hypothesis}', 30),
 ('push_neg', 29),
 ('cases isEmpty_or_nonempty {type}', 29),
 ('simp [{unknown}] at {unknown}', 28),
 ('lift {unknown} to ℝ≥0 using {unknown}', 28),
 ('rw [{unknown}] at {hypothesis}', 28),
 ('intro a ha', 28),
 ('· exact {hypothesis}', 28),
 ('exact mod_cast {unknown}', 27),
 ('subst {variable}', 27),
 ('simp only [{unknown}]', 27),
 ('cases {unknown} <;> simp', 27),
 ('exact Or.inl {unknown}', 27),
 ('intro {variable}', 26),
 ('· simpa using {unknown}', 26),
 ('trivial', 25),
 ('rw [← {unknown}] at {unknown}', 25),
 ('simp [{function}]', 25),
 ("cases' {unknown} with {unknown} {unknown}", 25),
 ('simp only [{hypothesis}]', 25),
 ("induction' {unknown} with {unknown} IH", 25),
 ('rw [one_mul]', 25),
 ('inhabit {type}', 25),
 ('exact {hypothesis} {variable} {hypothesis}', 25),
 ('exact {hypothesis} {variable}', 25),
 ('ext n', 24),
 ('by_cases h : {unknown} = 0', 24),
 ('intro j hj', 24),
 ('simpa [{unknown}] using {unknown}', 23),
 ('intro z hz', 23),
 ('cases {hypothesis}', 23),
 ('induction {unknown} using Real.Angle.induction_on', 23),
 ('simp (config := { contextual := true })', 22),
 ('simpa using {function}', 22),
 ('contrapose!', 22),
 ("refine' le_antisymm _ _", 22),
 ('revert {hypothesis}', 22),
 ('· simp at {unknown}', 22),
 ('by_contra! h', 22),
 ('exact {unknown} {hypothesis}', 22),
 ('convert {unknown} using 1', 22),
 ('intro {variable} hx', 22),
 ('rw [add_comm] at {unknown}', 21),
 ('rw [pow_succ]', 21),
 ('intro p hp', 21),
 ('apply Subset.antisymm', 21),
 ('intro hx', 21),
 ('haveI := Classical.decEq {type}', 21),
 ('intro b hb', 21),
 ('exact {unknown}.2', 21),
 ('haveI := {unknown}', 21),
 ('simpa only [sub_eq_add_neg] using {unknown}.add {unknown}.neg', 21),
 ('exact {unknown}.symm', 20),
 ('exact {hypothesis} {unknown} {unknown}', 20),
 ('exact le_top', 20),
 ('intro j', 20),
 ('letI := Classical.decEq {type}', 20),
 ('continuity', 20),
 ('· contradiction', 19),
 ("refine' ⟨fun h => _, fun h => _⟩", 19),
 ('use 0', 19),
 ('exact zero_le _', 19),
 ('exact {hypothesis} _', 19),
 ('exact {hypothesis}.1', 19),
 ('ring_nf', 19),
 ('rw [← image_univ]', 19),
 ('congr 2', 19),
 ('apply le_trans', 19),
 ("induction' {unknown} with {unknown} hn", 18),
 ('intro n hn', 18),
 ('haveI := Fact.mk {unknown}', 18),
 ('intro {unknown} hm', 18),
 ('rwa [{unknown}] at {unknown}', 18),
 ('rw [mul_comm] at {unknown}', 18),
 ('exact le_rfl', 18),
 ('rwa [← {unknown}]', 18),
 ('intro t', 18),
 ('cases {set}', 18),
 ('ext {function}', 18),
 ('nlinarith', 18),
 ('by_cases hG : CompleteSpace {type}', 18),
 ('cases {function}', 17),
 ('injection {unknown}', 17),
 ('intro s', 17),
 ('by_cases hp : {unknown} = 0', 17),
 ('· linarith', 17),
 ('convert {hypothesis}', 17),
 ('intro b', 17),
 ('simp [← {unknown}]', 17),
 ('tfae_finish', 17),
 ('exact {hypothesis} _ {unknown}', 16),
 ('intro y', 16),
 ('· assumption', 16),
 ('intro i j hij', 16),
 ('rw [sub_eq_add_neg]', 16),
 ('rintro ⟨rfl, rfl⟩', 16),
 ('dsimp at {unknown}', 16),
 ('exact ⟨{unknown}, {unknown}⟩', 16),
 ('· exact Or.inl {unknown}', 16),
 ('rw [mul_one]', 16),
 ('exact Iff.rfl', 16),
 ('ext1 {unknown}', 16),
 ('intro f', 15),
 ('exact {unknown}.1', 15),
 ('exact {unknown} rfl', 15),
 ('intro k hk', 15),
 ('· simpa', 15),
 ('simp_rw [{unknown}]', 15),
 ("cases' {hypothesis} with {hypothesis} {hypothesis}", 15),
 ('rw [← {unknown}] at {hypothesis}', 15),
 ('cases {unknown} <;> cases {unknown} <;> rfl', 15),
 ('rw [← {function}]', 15),
 ('exact {hypothesis}.2', 15),
 ('isBoundedDefault', 15),
 ('apply lintegral_congr_ae', 15),
 ('linarith only [{unknown}]', 15),
 ('simp [{unknown}, {hypothesis}]', 14),
 ("refine' ⟨fun h => _, _⟩", 14),
 ('cases subsingleton_or_nontrivial {type}', 14),
 ('use 1', 14),
 ('by_cases h : {variable} = 0', 14),
 ('· simp_all', 14),
 ('exact ⟨{variable}, {hypothesis}⟩', 14),
 ('contrapose {unknown}', 14),
 ('rw [mul_assoc]', 14),
 ('by_cases hx : {variable} = 0', 14),
 ('filter_upwards [self_mem_nhdsWithin]', 14),
 ('intro _', 13),
 ('revert {unknown} {unknown}', 13),
 ('simp [{unknown}] at {hypothesis}', 13),
 ('apply dvd_mul_right', 13),
 ('exact bot_le', 13),
 ('intro hf', 13),
 ('norm_num at {unknown}', 13),
 ('rotate_left', 13),
 ('intro i _', 13),
 ('funext x y', 13),
 ('simp_rw [{hypothesis}]', 13),
 ('intro t ht', 13),
 ('use {hypothesis}', 13),
 ('haveI := Fintype.ofFinite {type}', 13),
 ('funext x', 13),
 ("cases' {unknown} with hd tl", 13),
 ('intro z', 13),
 ('ext z', 13),
 ('by_cases hf : Summable {function}', 13),
 ('intro e', 13),
 ('lift {function} to {type} → ℝ≥0 using {hypothesis}', 13),
 ('haveI := {set}.to_subtype', 13),
 ('intro c', 13),
 ('filter_upwards [{hypothesis}] with x hx', 13),
 ('by_cases hp_top : {unknown} = ∞', 13),
 ('by_cases hf : Integrable {function} {unknown}', 13),
 ('simp only [integral, {unknown}, L1.integral]', 13),
 ('simp [integral, {hypothesis}]', 13),
 ('simp [{unknown}.symm]', 12),
 ('use {unknown}, {unknown}', 12),
 ('rw [{unknown}, zero_mul]', 12),
 ('intro k', 12),
 ('exact one_ne_zero', 12),
 ('by_cases hp0 : {unknown} = 0', 12),
 ('by_cases hn : {unknown} = 0', 12),
 ('rcases {unknown}.eq_zero_or_pos with (rfl | hn)', 12),
 ('classical!', 12),
 ('exact ⟨{variable}, {hypothesis}, rfl⟩', 12),
 ('intro h x', 12),
 ('fconstructor', 12),
 ('ext c', 12),
 ('rintro ⟨x, hx⟩', 12),
 ('intro u hu', 12),
 ("apply forall_congr'", 12),
 ('ext1 i', 12),
 ('lift {variable} to ℝ≥0 using {unknown}', 12),
 ('simp [mul_comm]', 12),
 ('exact {hypothesis} _ {hypothesis}', 12),
 ('specialize {hypothesis} {variable}', 12),
 ('rwa [{unknown}.cof_eq]', 12),
 ('filter_upwards [{hypothesis}]', 12),
 ('intro r hr', 12),
 ('ext x y', 12),
 ('ext1 s hs', 12),
 ('rw [← hasDerivWithinAt_univ] at *', 12),
 ('· rw [{unknown}]', 11),
 ('simp [eq_comm]', 11),
 ('exact le_max_left _ _', 11),
 ('rcases eq_or_ne {unknown} 0 with (rfl | ha)', 11),
 ('rcases {unknown} with ⟨⟩', 11),
 ('rw [{unknown}, mul_zero]', 11),
 ('by_cases ha : {variable} = 0', 11),
 ('use {function}', 11),
 ('nontriviality', 11),
 ('exact dvd_mul_right _ _', 11),
 ('funext', 11),
 ('rw [← {unknown}, {unknown}]', 11),
 ('simpa [{unknown}]', 11),
 ("induction' {unknown} with hd tl IH", 11),
 ('intro m hm', 11),
 ('exact ({unknown} rfl).elim', 11),
 ('ext p', 11),
 ('congr with x', 11),
 ('apply toFinite', 11),
 ('· simp [*]', 11),
 ('induction {unknown} <;> simp [*]', 11),
 ('by_contra! H', 11),
 ('cases {unknown} <;> cases {unknown} <;> simp [{hypothesis}]', 11),
 ('apply Finset.sum_congr rfl', 11),
 ('ext1 {variable}', 11),
 ('exact {unknown}.le', 11),
 ('use {variable}, {hypothesis}', 11),
 ('apply le_max_left', 11),
 ('intro {hypothesis}', 11),
 ('intro c hc', 11),
 ('intro x _', 11),
 ('apply min_le_left', 11),
 ("cases' {unknown} with _ _ hprime _", 11),
 ('simp!', 10),
 ('have {unknown} := {unknown} ({unknown} := {unknown})', 10),
 ('simp [{unknown}] at {unknown} {unknown}', 10),
 ('split_ifs with {unknown}', 10),
 ('simp [add_comm]', 10),
 ('· exact {hypothesis} {unknown}', 10),
 ('rintro ⟨i, hi⟩', 10),
 ('clear {hypothesis}', 10),
 ('split_ifs with h <;> simp [h]', 10),
 ('split_ifs <;> rfl', 10),
 ('norm_cast at {unknown}', 10),
 ('exact id', 10),
 ('simp [{hypothesis}, {unknown}]', 10),
 ("refine' ⟨{unknown}, _⟩", 10),
 ('· cases {unknown}', 10),
 ('rcases eq_or_ne {variable} {variable} with (rfl | h)', 10),
 ('exact ⟨{hypothesis}, {hypothesis}⟩', 10),
 ('rw [disjoint_iff_inf_le]', 10),
 ('subst {function}', 10),
 ('exact {unknown}.trans {unknown}', 10),
 ('intro {function}', 10),
 ('· exact le_rfl', 10),
 ('simp [mul_assoc]', 10),
 ('by_contra!', 10),
 ('haveI := {set}.toEncodable', 10),
 ('intro U hU', 10),
 ('rw [nhds_prod_eq]', 10),
 ('by_cases h : {variable} ∈ {set}', 10),
 ('exact {hypothesis} {variable} {unknown}', 10),
 ('intro x y', 10),
 ('convert {function}', 10),
 ('cases nonempty_encodable {type}', 10),
 ('intro _ _', 10),
 ('apply Finset.sum_eq_zero', 10),
 ('rw [← nonpos_iff_eq_zero]', 10),
 ('apply eventually_of_forall', 10),
 ('intro ε εpos', 10),
 ('lift {variable} to ℝ≥0 using {unknown}.le', 10),
 ('rw [← memℒp_one_iff_integrable] at {unknown} ⊢', 10),
 ('intro {set} hs hμs', 10),
 ('simp only [norm_smul]', 10),
 ('sorry', 10),
 ('exact {function} {unknown}', 9),
 ('apply {function}', 9),
 ('apply absurd {unknown}', 9),
 ('apply Eq.symm', 9),
 ('exact funext {hypothesis}', 9),
 ('split_ifs <;> simp [*]', 9),
 ('· aesop', 9),
 ('nth_rw 1 [{unknown}]', 9),
 ('cases isEmpty_or_nonempty {unknown}', 9),
 ('intro x y h', 9),
 ('rw [← sub_eq_zero]', 9),
 ('exact one_ne_zero {unknown}', 9),
 ("refine' {unknown}.induction_on _ _", 9),
 ('rcases eq_or_ne {unknown} 0 with (rfl | hn)', 9),
 ('rintro ⟨⟩', 9),
 ('simp [{hypothesis}, {hypothesis}]', 9),
 ("induction' {unknown} with {unknown} ihn", 9),
 ('exact le_of_lt {unknown}', 9),
 ('conv_lhs => rw [{unknown}]', 9),
 ('rw [inter_comm]', 9),
 ('by_contra H', 9),
 ('ext ⟨x, y⟩', 9),
 ('· exact Or.inl {hypothesis}', 9),
 ('rw [mem_map]', 9),
 ('by_cases h : {variable} ∈ {unknown}', 9),
 ('intro eq', 9),
 ('apply Subsingleton.elim', 9),
 ('· simpa using {hypothesis}', 9),
 ('ext m', 9),
 ('split_ifs <;> simp', 9),
 ('intro x y hxy', 9),
 ('simpa using {hypothesis} {variable} {hypothesis}', 9),
 ("obtain rfl | ha' := {unknown}.eq_or_lt", 9),
 ('intro x hx y hy', 9),
 ('congr with i', 9),
 ('contrapose', 9),
 ('obtain rfl | {unknown} := {unknown}.eq_or_lt', 9),
 ("cases' {unknown} with k hk", 9),
 ('dsimp at {function}', 9),
 ('intro g', 9),
 ('by_cases hx : {variable} ∈ {set}', 9),
 ('exact tendsto_const_nhds', 9),
 ('simpa using {unknown}.symm', 9),
 ('rw [zero_smul]', 9),
 ('split_ifs with h₁ h₂', 9),
 ('exact ⟨{variable}, rfl⟩', 9),
 ('fin_cases {unknown}', 9),
 ('haveI := nontrivial_of_invariantBasisNumber {type}', 9),
 ('intro w hw', 9),
 ('apply min_le_right', 9),
 ('by_cases hx : {unknown} = 0', 9),
 ('rcases {unknown}.eq_or_lt with (rfl | {unknown})', 9),
 ('by_cases h0 : {variable} = 0', 9),
 ('intro {set} hs', 9),
 ('ext s hs', 9),
 ('simp_arith', 8),
 ('have haux : {unknown} * Var.denote {unknown} {unknown} ≤ {unknown} * Var.denote {unknown} {unknown} := Nat.mul_le_mul_right _ (Nat.le_of_lt (Nat.blt_eq.mp {unknown}))',
  8),
 ('simp at {function}', 8),
 ('cases {unknown} <;> decide', 8),
 ('apply {hypothesis} {unknown}', 8),
 ('exact dvd_zero _', 8),
 ('by_cases hq : {unknown} = 0', 8),
 ('rw [if_neg]', 8),
 ('rw [pow_zero]', 8),
 ('convert rfl', 8),
 ('rw [if_neg {hypothesis}]', 8),
 ('exact Nat.zero_le _', 8),
 ('simp [{hypothesis} {variable} {hypothesis}]', 8),
 ("refine' {unknown}.trans _", 8),
 ('constructor <;> intro H', 8),
 ('rwa [{unknown}] at {hypothesis}', 8),
 ('rw [← mul_assoc]', 8),
 ('rw [two_mul]', 8),
 ('intro hy', 8),
 ('specialize {hypothesis} {unknown}', 8),
 ('rw [not_le] at {hypothesis}', 8),
 ('induction {unknown} using Quotient.inductionOn', 8),
 ('· simp at {hypothesis}', 8),
 ('have := {set}.to_subtype', 8),
 ('intro x hx y hy hxy', 8),
 ('rintro _ ⟨x, hx, rfl⟩', 8),
 ('ext f', 8),
 ('ext t', 8),
 ('rw [biUnion_eq_iUnion]', 8),
 ('simp [and_comm]', 8),
 ('simpa using {hypothesis} {variable}', 8),
 ('ext s', 8),
 ('ext b', 8),
 ('obtain ⟨k, hk, rfl⟩ := nthLe_of_mem {hypothesis}', 8),
 ('· simp [← {unknown}]', 8),
 ('intro r', 8),
 ('simp only [{unknown}, {unknown}]', 8),
 ('exact {hypothesis} _ _ {unknown}', 8),
 ('rw [← {hypothesis}]', 8),
 ('rw [zero_add] at {unknown}', 8),
 ('exact {function} {hypothesis}', 8),
 ("simp [ofReal']", 8),
 ('apply congr_arg', 8),
 ('rintro ⟨y, rfl⟩', 8),
 ('rw [add_zero]', 8),
 ('· congr\n  funext x y\n  cases x <;> cases y <;> rfl', 8),
 ('rw [eq_top_iff]', 8),
 ('rw [continuous_iff_continuousOn_univ]', 8),
 ('intro v hv', 8),
 ('haveI := {unknown} ⊓ {unknown}', 8),
 ('haveI : Nonempty {type} := ⟨{variable}⟩', 8),
 ('rw [dist_eq_norm]', 8),
 ('intro w', 8),
 ('ext a b', 8),
 ('rw [Submodule.span_le]', 8),
 ('rw [one_smul]', 8),
 ('apply ne_of_lt', 8),
 ('intro {unknown} hx', 8),
 ('intro σ', 8),
 ('by_cases hp_zero : {unknown} = 0', 8),
 ('apply measure_mono', 8),
 ('rw [mem_filter] at {function}', 8),
 ("refine' le_iInf _", 8),
 ("refine' Eq.trans _ map_refl.symm", 8),
 ('exact and_comm', 8),
 ('· field_simp', 8),
 ('rcases {hypothesis} with ⟨P, rfl⟩', 8),
 ('apply inf_le_left', 8),
 ('apply ext', 7),
 ('rw [add_mul]', 7),
 ('rw [or_iff_not_imp_right]', 7),
 ('ext : 1', 7),
 ('by_cases h0 : {unknown} = 0', 7),
 ('all_goals positivity', 7),
 ('intro hn', 7),
 ('exact Set.mem_range_self _', 7),
 ('rintro ⟨k, rfl⟩', 7),
 ('rw [zero_add]', 7),
 ('dsimp [{unknown}]', 7),
 ('exact zero_le_one', 7),
 ('dsimp only at {function}', 7),
 ('simp only at {function}', 7),
 ('rw [← not_iff_not]', 7),
 ("by_contra h'", 7),
 ('apply Nat.succ_pos', 7),
 ('· cases {unknown} rfl', 7),
 ('cases {unknown} rfl', 7),
 ('rw [mem_antidiagonal] at {hypothesis}', 7),
 ('exact {hypothesis} {unknown} {unknown} {unknown}', 7),
 ('· exact ({unknown} rfl).elim', 7),
 ('intro q hq', 7),
 ('rcases eq_empty_or_nonempty {set} with (rfl | hne)', 7),
 ('· rwa [{unknown}]', 7),
 ('rintro ⟨c, rfl⟩', 7),
 ("refine' ⟨{unknown}, {unknown}, _⟩", 7),
 ('ext {set}', 7),
 ('rcases eq_or_ne {variable} {variable} with (rfl | hne)', 7),
 ('intro h x hx', 7),
 ('haveI : Nonempty {set} := {set}.to_subtype', 7),
 ('rintro ⟨{variable}, rfl⟩', 7),
 ('exact Or.inr {hypothesis}', 7),
 ('induction {unknown} generalizing {unknown}', 7),
 ('intro hs', 7),
 ('simp [le_refl]', 7),
 ('cases {unknown} <;> simp [*]', 7),
 ('split <;> simp [*]', 7),
 ('choose f hf using {hypothesis}', 7),
 ('ext <;> simp', 7),
 ('lift {unknown} to ℝ using ⟨{unknown}, {unknown}⟩', 7),
 ('apply {unknown}.trans', 7),
 ('rw [← not_lt]', 7),
 ("cases' {unknown} with x y", 7),
 ('congr with j', 7),
 ('simp only [Function.comp_apply]', 7),
 ('rintro x hx', 7),
 ('rcases eq_or_lt_of_le {unknown} with (rfl | {unknown})', 7),
 ('apply le_max_right', 7),
 ('exact {unknown} _ {hypothesis}', 7),
 ('intro hc', 7),
 ('simp only [{hypothesis}, Finset.sum_insert, not_false_iff]', 7),
 ('tfae_have 1 → 2', 7),
 ('tfae_have 2 → 3', 7),
 ('dsimp only [SProd.sprod]', 7),
 ('by_cases ha : {unknown} = 0', 7),
 ("refine' Finset.induction_on {unknown} _ _", 7),
 ('intro m', 7),
 ('exact ⟨{variable}, {hypothesis}, {unknown}⟩', 7),
 ('ext1 n', 7),
 ('rw [add_zero] at {function}', 7),
 ('letI : UniformSpace {type} := TopologicalAddGroup.toUniformSpace {type}',
  7),
 ('intro i j', 7),
 ('· exact {hypothesis} {variable} {hypothesis}', 7),
 ('rw [eq_bot_iff]', 7),
 ('convert {unknown} using 2', 7),
 ('rw [smul_add]', 7),
 ('field_simp [{unknown}]', 7),
 ('by_cases hi : {variable} = {variable}', 7),
 ('apply le_min', 7),
 ('rw [← {unknown}, ← {unknown}]', 7),
 ('rw [star_mul, star_star]', 7),
 ('simp only [pow_one]', 7),
 ('rw [div_eq_mul_inv]', 7),
 ('ext1 s', 7),
 ('haveI : Encodable {set} := {set}.toEncodable', 7),
 ('apply ENNReal.tsum_le_tsum fun n => ?_', 7),
 ('rw [add_zero] at {unknown}', 7),
 ('rw [← dist_eq_norm]', 7),
 ('· simp [integral, {hypothesis}]', 7),
 ("refine' integral_congr_ae _", 7),
 ('· norm_num', 7),
 ('by_cases hx0 : {variable} = 0', 7),
 ('by_cases hE : CompleteSpace {type}', 7),
 ('by_cases hx : {unknown} {variable} = {variable}', 7),
 ('rintro ⟨{variable}, ha⟩ ⟨{variable}, hb⟩', 7),
 ('apply sup_le', 7),
 ('apply mul_comm', 7),
 ('rintro ⟨i, j⟩ hij', 7),
 ('let ⟨l, m, r, {unknown}⟩ := {unknown}.validFor', 7),
 ('simp [concat] at {unknown}', 6),
 ('intro h₁ h₂', 6),
 ('split_ifs <;> simp_all', 6),
 ('exact ⟨rfl, rfl⟩', 6),
 ('apply and_congr', 6),
 ('rw [abs_of_nonneg {unknown}]', 6),
 ('· rw [{hypothesis}]\n  simp', 6),
 ('rw [{unknown}, {unknown}, {unknown}]', 6),
 ("refine' ⟨{unknown}, _, _⟩", 6),
 ('rw [Finset.forall_mem_insert] at {hypothesis}', 6),
 ('simp only [eq_iff_true_of_subsingleton]', 6),
 ('rw [if_pos {unknown}]', 6),
 ('· simp [{hypothesis}, {unknown}]', 6),
 ("refine' ⟨{variable}, {hypothesis}, _⟩", 6),
 ('rintro ⟨{unknown}, rfl⟩', 6),
 ('intro con', 6),
 ('apply {unknown} {hypothesis}', 6),
 ('simp [eq_iff_true_of_subsingleton]', 6),
 ('obtain ⟨k, hk⟩ := {unknown}', 6),
 ('· rintro rfl\n  simp', 6),
 ('rw [zero_mul]', 6),
 ('exact Nat.lt_succ_self _', 6),
 ("induction' {unknown} with {unknown} {unknown} ih", 6),
 ("induction' {unknown} with hd tl hl", 6),
 ("induction' {unknown} with hd tl ih", 6),
 ('funext {unknown}', 6),
 ('exact lt_irrefl _ {unknown}', 6),
 ('obtain ⟨n, hn⟩ := {unknown}', 6),
 ('exact ⟨{unknown}, {unknown}, {unknown}⟩', 6),
 ('exact em _', 6),
 ('apply zero_le', 6),
 ("induction' {unknown} with {unknown} ih generalizing {unknown}", 6),
 ("cases' {unknown}.lt_or_lt with {unknown} {unknown}", 6),
 ('exact mem_range_self _', 6),
 ('exact Or.inr {unknown}', 6),
 ('obtain rfl | hij := eq_or_ne {variable} {variable}', 6),
 ('exact ⟨{unknown}, {hypothesis}, rfl⟩', 6),
 ('rcases {hypothesis} with ⟨{variable}, rfl⟩', 6),
 ('clear {unknown} {unknown}', 6),
 ('exact mem_univ _', 6),
 ('rw [dvd_iff]', 6),
 ('rw [gcd_comm]', 6),
 ('exact ⟨{unknown}, {hypothesis}⟩', 6),
 ('obtain rfl | hs := {set}.eq_empty_or_nonempty', 6),
 ('· simp [{hypothesis}, {hypothesis}]', 6),
 ('split_ifs with {hypothesis}', 6),
 ('rwa [← {unknown}] at {unknown}', 6),
 ('exact Classical.choose_spec {hypothesis}', 6),
 ('rw [{unknown}] at {function}', 6),
 ('intro f g h', 6),
 ('trans', 6),
 ('simpa [{hypothesis}] using {hypothesis}', 6),
 ('· simp [{unknown}, {hypothesis}]', 6),
 ('exact {hypothesis} {variable} {hypothesis} {variable} {hypothesis}', 6),
 ('simp [lt_irrefl]', 6),
 ('induction {unknown}', 6),
 ('congr!', 6),
 ('revert {variable}', 6),
 ('· apply {hypothesis}', 6),
 ('clear_value {function}', 6),
 ('rw [{unknown}, mul_comm]', 6),
 ('rcases {unknown} with {unknown} | {unknown}', 6),
 ('haveI := Classical.propDecidable', 6),
 ('conv_rhs => rw [{unknown}]', 6),
 ('rintro ⟨h₁, h₂⟩', 6),
 ('exact ⟨{variable}, {hypothesis}, {hypothesis}⟩', 6),
 ('rw [{unknown}, zero_add]', 6),
 ('exact ⟨_, rfl⟩', 6),
 ('rw [mul_assoc] at {unknown}', 6),
 ('· cases NeZero.ne 0 rfl', 6),
 ('cases NeZero.ne 0 rfl', 6),
 ('rcases {unknown} with ⟨d, rfl⟩', 6),
 ('exact ⟨_, {unknown}⟩', 6),
 ('simp [div_eq_mul_inv]', 6),
 ('rw [mul_zero]', 6),
 ('rwa [type_lt]', 6),
 ('exact inter_subset_right _ _', 6),
 ('rw [nonempty_iff_ne_empty]', 6),
 ('simp only [not_exists, not_lt] at {hypothesis}', 6),
 ('rw [sUnion_eq_iUnion]', 6),
 ('rw [dist_comm] at {unknown}', 6),
 ('tfae_have 1 ↔ 2', 6),
 ('rwa [zero_add]', 6),
 ('tfae_have 3 → 1', 6),
 ('rw [smul_zero]', 6),
 ('rcases le_total {variable} {variable} with h | h', 6),
 ('ext1 {function}', 6),
 ('intro {set} ht', 6),
 ('rintro ⟨n, hn⟩', 6),
 ('intro h2', 6),
 ('exact ⟨{variable}, {unknown}⟩', 6),
 ('ext v', 6),
 ('rw [{unknown}] at {unknown} {unknown}', 6),
 ('push_cast at {unknown}', 6),
 ('linarith only [{unknown}, {unknown}]', 6),
 ('obtain rfl | hc := eq_or_ne {variable} 0', 6),
 ('rw [zpow_ofNat]', 6),
 ('obtain ⟨{variable}, rfl⟩ := exists_add_of_le {unknown}', 6),
 ('simp only [neg_pos, neg_nonneg]', 6),
 ('measurability', 6),
 ("intro x hx h'x", 6),
 ("apply {unknown}.congr' _", 6),
 ('apply coeFn_mk', 6),
 ('· exact (measurable_rnDeriv _ _).aemeasurable', 6),
 ('exact (measurable_rnDeriv _ _).aemeasurable', 6),
 ('rw [measure_iUnion]', 6),
 ('apply add_le_add', 6),
 ('simp [{unknown}.le]', 6),
 ('rw [← Ne.def] at {hypothesis}', 6),
 ("refine' {hypothesis}.mono fun x hx => _", 6),
 ('· simp [integral, {unknown}]', 6),
 ('exact {hypothesis} {set} {unknown} {unknown}', 6),
 ('cases {unknown}.out', 6),
 ('rw [measure_singleton]', 6),
 ('exact ENNReal.zero_ne_top', 6),
 ('intro {set}', 6),
 ('rw [sub_nonneg]', 6),
 ('exact {unknown}.mono {unknown}', 6),
 ("cases' lt_or_gt_of_ne {unknown} with {unknown} {unknown}", 6),
 ('rintro r (rpos : 0 < r)', 6),
 ('rintro ⟨p, {variable}⟩ ⟨hp, -⟩', 6),
 ('rwa [interior_interior]', 6),
 ('rcases NormedField.exists_one_lt_norm {type} with ⟨c, hc⟩', 6),
 ('linarith [{hypothesis}.2]', 6),
 ('rw [← contDiffWithinAt_univ] at *', 6),
 ('· nlinarith', 6),
 ('ring_nf at {unknown} ⊢', 6),
 ("have hxz' : 0 < {variable} - {variable} := by linarith", 6),
 ("rw [← add_div, div_self {unknown}.ne']", 6),
 ('exact_mod_cast of_decide_eq_true {unknown}', 6),
 ('apply Nat.lt_of_lt_of_le {unknown}', 6),
 ('unfold merge', 6),
 ('simp only [Heap.tail?]', 6),
 ('push_neg  at {hypothesis}', 6),
 ('apply le_inf', 6),
 ('apply inf_le_right', 6),
 ('apply le_sup_left', 6),
 ('split at {unknown}', 5),
 ('next => intro; contradiction', 5),
 ('exact rfl', 5),
 ('apply Nat.le_refl', 5),
 ('simp [eq_of_beq {unknown}]', 5),
 ('simp [insert_nonEmpty]', 5),
 ('have heqv : {unknown} = {unknown} := eq_of_not_blt_eq_true {hypothesis} {hypothesis}',
  5),
 ('rename_i {function} lhs rhs', 5),
 ('intro ⟨h₁, h₂⟩', 5),
 ('simpa [{hypothesis}]', 5),
 ('by_contra hn', 5),
 ('rintro ⟨x, rfl⟩', 5),
 ('rcases {unknown} with ⟨c, rfl⟩', 5),
 ('haveI := Nontrivial.of_polynomial_ne {unknown}', 5),
 ('rwa [mul_comm]', 5),
 ('obtain ⟨{unknown}, rfl⟩ := {hypothesis}', 5),
 ('use {unknown} + {unknown}', 5),
 ('rw [← Finset.card_range {unknown}, ← Finset.prod_const]', 5),
 ('obtain ⟨{unknown}, rfl⟩ := {unknown}', 5),
 ('rcases {unknown} with ⟨{unknown}⟩', 5),
 ('· rw [{hypothesis}]', 5),
 ('rw [coeff_map]', 5),
 ('exact Nat.le_add_right _ _', 5),
 ('intro {variable} hb', 5),
 ('rw [mem_range] at {hypothesis}', 5),
 ('by_cases hij : {variable} = {variable}', 5),
 ('intro hp', 5),
 ('rw [isPrimePow_nat_iff]', 5),
 ('simpa [mul_comm] using {unknown}', 5),
 ('dsimp only []', 5),
 ("induction' {unknown} with d {unknown} ih", 5),
 ('dsimp [{function}]', 5),
 ('simp [ofDigits]', 5),
 ("induction' {unknown} with _ _ ih", 5),
 ('rcases {unknown} with (_ | _ | {unknown})', 5),
 ('exact le_self_add', 5),
 ('rw [sq, sq]', 5),
 ('rintro ⟨n, rfl⟩', 5),
 ('intro h₁', 5),
 ('rw [Set.not_nonempty_iff_eq_empty] at {hypothesis}', 5),
 ('funext m n', 5),
 ('cases {unknown} <;> simp_all', 5),
 ('· subst {hypothesis}\n  simp', 5),
 ('exact inf_le_right', 5),
 ('rintro -', 5),
 ('rw [eq_comm]', 5),
 ('rcases eq_or_ne {unknown} 0 with (rfl | hb)', 5),
 ('rw [{hypothesis} {variable} {hypothesis}]', 5),
 ('exact {hypothesis} {variable} {hypothesis} {unknown}', 5),
 ('rcases mem_iUnion.1 {hypothesis} with ⟨n, hn⟩', 5),
 ('rintro i hi', 5),
 ('lift {unknown} to ℕ using {unknown}', 5),
 ('exact subset_union_right _ _', 5),
 ('exact mem_cons_self _ _', 5),
 ('funext a', 5),
 ('funext _', 5),
 ('simp only [mem_cons]', 5),
 ('rintro ⟨f, hf⟩', 5),
 ('· cases {hypothesis}', 5),
 ('simpa [{unknown}] using {hypothesis}', 5),
 ('rw [nodup_iff_nthLe_inj] at {unknown}', 5),
 ('intro {unknown} {unknown}', 5),
 ('unfold {unknown}', 5),
 ("induction' {unknown} with hd tl hl generalizing {unknown}", 5),
 ("cases' {unknown} with x {unknown}", 5),
 ('rintro (h | h)', 5),
 ('obtain ⟨k, rfl⟩ := {unknown}', 5),
 ('apply propext', 5),
 ('intro ε ε0', 5),
 ("cases' {unknown} with {unknown} hi", 5),
 ('intro {function} hx', 5),
 ('funext {function}', 5),
 ('intro l hl', 5),
 ('apply Finset.prod_congr rfl', 5),
 ('exact zero_mem _', 5),
 ('intro d hd', 5),
 ('rw [Finsupp.sum_mapDomain_index]', 5),
 ('simp_rw [mul_comm]', 5),
 ('intro {unknown} {hypothesis}', 5),
 ('tfae_have 1 → 4', 5),
 ('rw [image_subset_iff]', 5),
 ('simp [map_eq_image]', 5),
 ('exact ⟨{unknown}, {hypothesis}, {hypothesis}⟩', 5),
 ('fin_cases {unknown} <;> fin_cases {unknown} <;> rfl', 5),
 ('ext ⟨a, b⟩', 5),
 ('rw [{unknown}, add_zero]', 5),
 ('exact {unknown}.ne', 5),
 ('rcases le_or_lt {variable} 0 with hr | hr', 5),
 ('rcases {unknown} with ⟨u, rfl⟩', 5),
 ('apply dvd_zero', 5),
 ('rintro rfl x', 5),
 ('exact {unknown} {unknown} {unknown}', 5),
 ('rintro _ ⟨s, hs, rfl⟩', 5),
 ('subst {unknown} {unknown}', 5),
 ('apply dvd_add', 5),
 ('intro a {unknown}', 5),
 ('use {unknown} + 1', 5),
 ('apply typein_lt_type', 5),
 ('rw [← familyOfBFamily_enum {unknown} {function}]', 5),
 ('apply add_le_add_left {unknown}', 5),
 ('assumption_mod_cast', 5),
 ('exact {unknown}.inter {unknown}', 5),
 ('simp [{set}]', 5),
 ('rw [eventually_atTop]', 5),
 ('rw [hasCompactSupport_iff_eventuallyEq] at {unknown} ⊢', 5),
 ('rintro ⟨s, hs⟩', 5),
 ('rwa [lift_id]', 5),
 ('intro {variable} hy', 5),
 ('exact {unknown} ⟨{unknown}, {unknown}⟩', 5),
 ('simp_rw [if_neg {hypothesis}]', 5),
 ('exact le_trans {unknown} {unknown}', 5),
 ('use ∅', 5),
 ('intro x xs', 5),
 ('letI := {unknown}.induced {function}', 5),
 ('exact {hypothesis} ⟨{variable}, {hypothesis}⟩', 5),
 ('intro i j h', 5),
 ('rw [Function.update_apply]', 5),
 ('exact ⟨{hypothesis}, {unknown}⟩', 5),
 ('simp only [Prod.forall]', 5),
 ('rw [tsum_eq_zero_of_not_summable {hypothesis}]', 5),
 ('choose g hg using {hypothesis}', 5),
 ('· simp [{set}]', 5),
 ('intro ε ε_pos', 5),
 ('rcases {set}.eq_empty_or_nonempty with hs | hs', 5),
 ('rw [hausdorffEdist_comm] at {unknown}', 5),
 ('rw [← cthickening_zero]', 5),
 ('letI := pseudoMetrizableSpacePseudoMetric {type}', 5),
 ('haveI : UniformAddGroup {type} := comm_topologicalAddGroup_is_uniform', 5),
 ("rcases eq_or_ne (𝓝[<] {variable}) ⊥ with (h' | h')", 5),
 ('by_cases hc : {variable} = 0', 5),
 ('exact {unknown}.add {unknown}.neg', 5),
 ('refine ⟨fun h ↦ ?_, ?_⟩', 5),
 ('rcases {unknown} with ({unknown} | {unknown})', 5),
 ('rintro j -', 5),
 ('ext ⟨a, b⟩ : 1', 5),
 ('ext k', 5),
 ('· congr', 5),
 ("rw [iterate_succ_apply']", 5),
 ("induction' {unknown} using Finset.induction with a {unknown} has ih h", 5),
 ('apply Submodule.subset_span', 5),
 ('rw [Submodule.mem_sup]', 5),
 ('ext {hypothesis}', 5),
 ('rw [Submodule.eq_bot_iff]', 5),
 ("refine' Finset.sum_congr rfl fun {unknown} _ => _", 5),
 ('exact mem_top', 5),
 ("rw [linearIndependent_iff']", 5),
 ('apply sum_mem', 5),
 ('apply sum_congr rfl', 5),
 ('intro g hg', 5),
 ('revert {function}', 5),
 ('rcases {hypothesis} with ⟨r, hr⟩', 5),
 ('all_goals decide', 5),
 ('exact congr_arg Int.cast rfl', 5),
 ('by_cases hy : {unknown} = 0', 5),
 ('simp [mem_iSup_of_directed {function}]', 5),
 ('simp [mem_sSup_of_directedOn {set} {function}]', 5),
 ('exact algebraMap_ne_zero (denom_ne_zero {unknown})', 5),
 ('simpa only [sub_eq_add_neg] using {unknown}.add_const (-{variable})', 5),
 ('apply dvd_mul_left', 5),
 ('simp only [iInf, mem_sInf, Set.forall_range_iff]', 5),
 ('rw [pow_zero, pow_zero]', 5),
 ('· simp only [pow_one]', 5),
 ('· simp only [one_pow, pow_one]', 5),
 ('simp only [one_pow, pow_one]', 5),
 ('rw [Ideal.span_le]', 5),
 ('exact {hypothesis} {variable} {variable}', 5),
 ('rw [← one_add_one_eq_two]', 5),
 ('· positivity', 5),
 ('rw [tendsto_iff_norm_sub_tendsto_zero]', 5),
 ('by_cases hl : HaveLebesgueDecomposition {unknown} {unknown}', 5),
 ('ext1 A hA', 5),
 ('intro A hA', 5),
 ('rcases {hypothesis} with ⟨n, hn⟩', 5),
 ('rw [← inter_iUnion]', 5),
 ('apply le_antisymm _ (zero_le _)', 5),
 ('exact {unknown}.continuousOn', 5),
 ('simpa using {unknown}.ne', 5),
 ("refine' lintegral_congr_ae _", 5),
 ('gcongr with a', 5),
 ('by_cases hxs : {variable} ∈ {set}', 5),
 ('filter_upwards [{unknown}.ae_eq_mk] with x hx', 5),
 ('apply Subset.trans _ {unknown}', 5),
 ('simp_rw [setToL1S]', 5),
 ("refine' ContinuousLinearMap.extend_unique (setToL1SCLM {type} {type} {unknown} {unknown}) _ _ _ _ _",
  5),
 ('apply integral_congr_ae', 5),
 ('ext1 {set} hs', 5),
 ('ext1 t ht', 5),
 ('apply {unknown}.mono', 5),
 ('rintro _ ⟨x, rfl⟩', 5),
 ('rw [{function} {unknown}]', 5),
 ('intro xs', 5),
 ("refine' {hypothesis} _", 5),
 ('simp [{unknown}.map_rel_iff]', 5),
 ('rw [fderivWithin_zero_of_not_differentiableWithinAt {hypothesis}]', 5),
 ("induction' {unknown} with {unknown} IH generalizing {variable}", 5),
 ('linarith [{hypothesis}.1]', 5),
 ('rw [{function}.map_add, {function}.map_smul, {function}.map_smul]', 5),
 ('intro {variable} hy a b ha hb hab', 5),
 ("intro h''", 5),
 ('simp only [← iteratedFDerivWithin_univ]', 5),
 ('rintro ⟨a, b, ha, hb, hab, rfl⟩', 5),
 ('refine EqOn.image_eq fun a ha ↦ ?_', 5),
 ("· refine' {unknown}.eq_or_eq_neg_of_isEmpty.elim _ _ <;> rintro rfl <;> simp",
  5),
 ("refine' {unknown}.eq_or_eq_neg_of_isEmpty.elim _ _ <;> rintro rfl <;> simp",
  5),
 ('have := {unknown}.topologicalAddGroup', 5),
 ('rw [div_eq_inv_mul]', 5),
 ('simp [log]', 5),
 ('letI := {function}.toAlgebra', 5),
 ('ext g', 5),
 ('intro J hJ', 5),
 ('have := Submodule.coe_mem ({function} {variable})', 5),
 ('rw [{unknown}, pow_add]', 5),
 ('intro {unknown} hl', 5),
 ('simp [{unknown}, mul_coeff]', 5),
 ('simp only [mul_def]', 5),
 ('simp only [Heap.tail]', 5),
 ('apply Nat.dvd_antisymm', 5),
 ('apply Nat.succ_le_succ', 5),
 ('intro.', 5),
 ('refine have ⟨_, _, h₁, _, eq⟩ := Buckets.exists_of_update ..; eq ▸ ?_', 5),
 ('unfold ins', 5),
 ('simp [{unknown}.toString] at {unknown}', 5),
 ('rw [Int.mul_comm]', 5),
 ('induction {unknown} generalizing {variable} <;> simp [*]', 5),
 ('cases {unknown}.symm.trans {unknown}', 5),
 ('rintro ⟨h0, h1⟩', 5),
 ('rw [abs_of_neg {unknown}]', 5),
 ('all_goals (intro; contradiction)', 4),
 ('(intro; contradiction)', 4),
 ('exact absurd {unknown} {hypothesis}', 4),
 ('simp [sort.loop]', 4),
 ('exact {function}.1', 4),
 ('apply Nat.zero_lt_succ', 4),
 ('apply @le.intro _ _ {unknown}', 4),
 ('rename_i k₁ v₁ m₁ k₂ v₂ {unknown}', 4),
 ('· apply {unknown}; simp [{unknown}] at {unknown} |{unknown}; assumption',
  4),
 ('simp [{unknown}] at {unknown} |-', 4),
 ('simp [{unknown}] at {unknown} |{unknown}', 4),
 ('rw [Nat.mul_sub_right_distrib, ← Nat.add_assoc, ← Nat.add_sub_assoc {unknown}]',
  4),
 ('have heqk : {unknown} = {unknown} := eq_of_not_blt_eq_true {hypothesis} {hypothesis}',
  4),
 ('· have {unknown} := {unknown} ({unknown} := {unknown}); simp [{unknown}] at {unknown} {unknown}; assumption',
  4),
 ('rw [Nat.mul_sub_right_distrib, ← Nat.add_assoc, ← Nat.add_sub_assoc {unknown}] at {unknown}',
  4),
 ('split <;> simp', 4),
 ('rw [Poly.denote_le_cancel_eq]', 4),
 ('simp [Poly.of_isZero, {unknown}]', 4),
 ('apply congrArg', 4),
 ('rw [List.length_cons]', 4),
 ('split_ifs with hx', 4),
 ('split_ifs at {unknown}', 4),
 ('· simp (config := { contextual := true })', 4),
 ('apply Nat.gcd_dvd_right', 4),
 ('lift {unknown} to ℝ≥0 using ne_top_of_lt {unknown}', 4),
 ('by_cases hf : {unknown} = 0', 4),
 ("cases' subsingleton_or_nontrivial {type} with hR hR", 4),
 ('rw [Finset.prod_insert {hypothesis}]', 4),
 ('rw [map_id]', 4),
 ('split_ifs with hi', 4),
 ('simp [ext_iff]', 4),
 ('rw [degree_lt_iff_coeff_zero] at {unknown}', 4),
 ('rcases {unknown} with ⟨{unknown}, rfl⟩', 4),
 ('rw [mul_ne_zero_iff] at {unknown}', 4),
 ('refine le_antisymm ?_ ?_', 4),
 ('rw [dif_neg {unknown}]', 4),
 ('exact zero_ne_one {unknown}', 4),
 ('rw [← C_1]', 4),
 ('rw [mul_zero] at {function}', 4),
 ('rcases le_or_lt {unknown} 1 with hb | hb', 4),
 ('rcases {unknown}, {unknown} with ⟨_ | _, _ | _⟩', 4),
 ("repeat' erw [WithBot.coe_eq_coe]", 4),
 ('erw [WithBot.coe_eq_coe]', 4),
 ('· exact bot_le', 4),
 ('have := minSqFac_has_prop {unknown}', 4),
 ('rw [{hypothesis}] at {unknown}', 4),
 ('by_contra hx', 4),
 ('rw [{unknown}, one_mul]', 4),
 ('unfold Monic', 4),
 ('rintro ⟨{unknown}, hx, rfl⟩', 4),
 ('refine Finsupp.ext fun _ => ?_', 4),
 ("split_ifs with h'", 4),
 ('· norm_num at {unknown}', 4),
 ('by_cases hm : {unknown} = 0', 4),
 ('rcases {unknown} with (_ | _ | {unknown}) <;> try simp_all', 4),
 ("induction' {unknown} using PartENat.casesOn with n", 4),
 ('rw [{unknown}, mul_one]', 4),
 ('rw [mul_one] at {unknown}', 4),
 ('simp [← two_mul, {unknown}]', 4),
 ('exact {unknown}.ne_zero', 4),
 ('exact card_le_card {unknown}', 4),
 ('exact dvd_zero {unknown}', 4),
 ('obtain ⟨a, rfl⟩ := {unknown}', 4),
 ('exact one_dvd _', 4),
 ('all_goals assumption', 4),
 ("rw [image_id']", 4),
 ('rw [image_eq_range]', 4),
 ('ext q', 4),
 ('· simp [{unknown}, {unknown}]', 4),
 ('exact inf_le_left', 4),
 ('intro i m', 4),
 ('simp [{unknown}, Finset.filter_singleton, not_isPrimePow_one]', 4),
 ('exact Or.inl rfl', 4),
 ('exact Or.inl {hypothesis}', 4),
 ('rw [if_pos rfl]', 4),
 ('intro _ h', 4),
 ('rcases {set}.eq_empty_or_nonempty with (rfl | hne)', 4),
 ("refine' disjoint_iff_inf_le.mpr _", 4),
 ('obtain ⟨x, hx⟩ := {set}', 4),
 ('apply eq', 4),
 ('ext ⟨x, hx⟩', 4),
 ('by_cases h : {function} {variable}', 4),
 ('simp [{unknown}.eq_iff]', 4),
 ('rcases {set}.eq_empty_or_nonempty with h | h', 4),
 ('cases finite_or_infinite {type}', 4),
 ('rintro ⟨h, rfl⟩', 4),
 ("refine' ⟨{variable}, {unknown}, _⟩", 4),
 ('intro y y_in', 4),
 ('simp [inter_comm]', 4),
 ('simp [Set.ext_iff]', 4),
 ('simp_rw [image2_subset_iff, mem_iInter]', 4),
 ('· exact {unknown}.trans {unknown}', 4),
 ("refine' ⟨_, fun h => _⟩", 4),
 ('intro a b h', 4),
 ('intro {variable} hi', 4),
 ('simp only [cons_append] at {unknown}', 4),
 ('rcases {unknown} with ⟨rfl, rfl⟩', 4),
 ("refine' permutationsAux.rec (by simp) _", 4),
 ('rintro _ ⟨y, hy, rfl⟩', 4),
 ('by_cases hx : {variable} ∈ {set} <;> simp [hx]', 4),
 ('· exact Or.inr {hypothesis}', 4),
 ("induction' {unknown} with y {unknown} IH", 4),
 ("cases' {unknown} with b {unknown}", 4),
 ('rw [prod_cons]', 4),
 ('intro l', 4),
 ('exact {unknown}.succ_pos', 4),
 ("cases' {unknown} with z zs", 4),
 ('· simpa [{hypothesis}] using {hypothesis}', 4),
 ('simp [{unknown}, {unknown}] at {hypothesis}', 4),
 ('simpa using {unknown}.2', 4),
 ('· exact {function}', 4),
 ('· simp only [{hypothesis}, or_true]', 4),
 ('simp only [{unknown}, or_true]', 4),
 ('simp only [{unknown}, and_true]', 4),
 ('by_cases hc : {variable} ≤ {variable}', 4),
 ('· simp only [{hypothesis}, {hypothesis}, and_self, or_true]', 4),
 ('· simp only [{hypothesis}, {hypothesis}, and_self, or_false]', 4),
 ('simp only [{unknown}, {unknown}, and_self, or_true]', 4),
 ('simp only [{unknown}, true_and, {unknown}, or_self]', 4),
 ('simp only [{unknown}, and_true, {hypothesis}, false_and, or_false, true_or]',
  4),
 ('simp only [{hypothesis}, {hypothesis}, and_self, or_false]', 4),
 ('apply {unknown}.subset', 4),
 ('rcases {unknown}.eq_or_lt with (rfl | hlt)', 4),
 ('substs {unknown} {unknown}', 4),
 ('simp [mergeSort]', 4),
 ('obtain ⟨n, rfl⟩ := {unknown}', 4),
 ('simp [reverse_involutive.comp_self, reverse_map]', 4),
 ('apply MvPolynomial.induction_on {unknown} <;>\n  · intros\n    simp [*]',
  4),
 ('apply induction_on {unknown}', 4),
 ('use {unknown} ∪ {unknown}', 4),
 ('apply swap', 4),
 ("induction' {unknown} with {unknown} IH generalizing {unknown}", 4),
 ('simpa using {hypothesis} _ {unknown}', 4),
 ('exact_mod_cast {unknown}', 4),
 ('rintro ⟨u, rfl⟩', 4),
 ('· simp [{function}]', 4),
 ('rw [{function}] at {function}', 4),
 ('· simp [{unknown}] at {unknown}', 4),
 ("refine' ⟨_, _, _⟩", 4),
 ('rw [lift_id]', 4),
 ('rw [ext_iff]', 4),
 ('apply {hypothesis} {hypothesis}', 4),
 ('intro {unknown} {unknown} hp hq', 4),
 ("refine' {unknown}.induction _ _", 4),
 ('apply exists_congr', 4),
 ('rw [degreeOf]', 4),
 ('rw [EReal.mul_comm]', 4),
 ("exact ciSup_le' {hypothesis}", 4),
 ('rintro ⟨_⟩ ⟨_⟩', 4),
 ('tfae_have 2 → 1', 4),
 ('obtain rfl | hx := eq_or_ne {variable} 0', 4),
 ('apply Fintype.card_congr', 4),
 ('by_contra! hf', 4),
 ('rw [zero_smul_set {set}, zero_smul]', 4),
 ('intro x {unknown} {unknown}', 4),
 ('dsimp only at {unknown}', 4),
 ('have : Nonempty {type} := ⟨{variable}⟩', 4),
 ('funext i', 4),
 ('exact {hypothesis} {function}', 4),
 ("exact {unknown}.ne'", 4),
 ('apply Nat.le_add_right', 4),
 ('dsimp [ZMod, ZMod.cast]', 4),
 ('rw [valMinAbs_def_pos]', 4),
 ('rcases {unknown} with (rfl | rfl)', 4),
 ('exact Finset.fold_congr {hypothesis}', 4),
 ('cases {hypothesis} rfl', 4),
 ('exact congr_fun Option.map_id _', 4),
 ('exists {unknown}', 4),
 ('simp [← one_add_one_eq_two]', 4),
 ('have h1 : {unknown} + 2 * {unknown} = 3 * {unknown} := by ring', 4),
 ('norm_num [Nat.factorial]', 4),
 ('· exact Complex.exp_bound (by simpa) (by decide)', 4),
 ('exact Complex.exp_bound (by simpa) (by decide)', 4),
 ('rw [mul_add]', 4),
 ('fapply Prefunctor.ext', 4),
 ('exact ⟨{unknown}, rfl⟩', 4),
 ('simp [{unknown}, {unknown}, {unknown}]', 4),
 ('rw [← Nat.div_add_mod {unknown} 8, {unknown}, Odd]', 4),
 ('rintro ⟨⟨⟩⟩', 4),
 ('refine ⟨{unknown}, ?_⟩', 4),
 ('exact mul_pos {unknown} {unknown}', 4),
 ('simp [mul_add]', 4),
 ('simp [sub_eq_add_neg]', 4),
 ('rw [NormNum.IsNat.to_eq {unknown} rfl]', 4),
 ('apply ne_of_gt', 4),
 ("rw [Arrow.iso_w' {unknown}]", 4),
 ('constructor <;> intro', 4),
 ('intro {variable} a b w', 4),
 ("rw [Function.iterate_succ']", 4),
 ('exact principal_zero', 4),
 ('rw [← {hypothesis} {unknown} {unknown}]', 4),
 ('rcases lt_omega.1 {unknown} with ⟨n, rfl⟩', 4),
 ('rcases lt_or_eq_of_le {unknown} with ({unknown} | rfl)', 4),
 ('rintro ⟨c, hc⟩', 4),
 ('rcases eq_or_ne {unknown} 0 with (rfl | hx)', 4),
 ('rw [biInter_eq_iInter]', 4),
 ('rintro x -', 4),
 ('intro a b', 4),
 ('rcases le_or_lt ℵ₀ {unknown} with ha | ha', 4),
 ('rw [and_assoc]', 4),
 ('apply Subtype.eq', 4),
 ('apply {unknown}.2', 4),
 ("refine' ⟨{function}, _⟩", 4),
 ('cases fintypeOrInfinite {type}', 4),
 ('letI := upgradePolishSpace {type}', 4),
 ('intro x x_in', 4),
 ('rw [cof_eq_sInf_lsub]', 4),
 ('rwa [(#{type}).lift_id]', 4),
 ('rw [{unknown}.cof_eq]', 4),
 ('apply lt_blsub', 4),
 ('rcases lt_aleph0.1 {unknown} with ⟨n, rfl⟩', 4),
 ('by_cases H : {function} {variable} ≤ {function} {variable}', 4),
 ('· simp [{hypothesis}]\n  exact {unknown} ⟨{unknown}, {hypothesis}⟩', 4),
 ('· simp [le_of_not_le {hypothesis}]\n  exact {unknown} ⟨{unknown}, le_rfl⟩',
  4),
 ('simp [le_of_not_le {hypothesis}]', 4),
 ('exact {unknown} ⟨{unknown}, le_rfl⟩', 4),
 ('apply Filter.eventually_of_forall', 4),
 ('simp_rw [← lowerSemicontinuousWithinAt_univ_iff] at *', 4),
 ('filter_upwards [{hypothesis}] with i hi', 4),
 ('exact isClosed_univ', 4),
 ('ext U', 4),
 ('rw [mem_nhds_iff]', 4),
 ('exact Continuous fun p : {type} × {type} => {function} p.1 p.2', 4),
 ('haveI := sInf {set}', 4),
 ('rw [Subtype.image_preimage_coe]', 4),
 ('have := {hypothesis} {variable}', 4),
 ('rintro ⟨a, rfl⟩', 4),
 ('apply nat_lt_aleph0', 4),
 ('rintro ⟨y, hy⟩', 4),
 ('lift {set} to Closeds {type} using {unknown}', 4),
 ('lift {variable} to {set} using {hypothesis}', 4),
 ('exact {unknown}.prod_map {unknown}', 4),
 ('rintro (rfl | rfl)', 4),
 ('apply isTopologicalBasis_of_isOpen_of_nhds', 4),
 ('rw [tsum_eq_zero_of_not_summable {hypothesis}, tsum_eq_zero_of_not_summable {hypothesis}]',
  4),
 ('exact {unknown}.false.elim', 4),
 ('lift {set} to Finset {type} using {set}', 4),
 ('exact ENNReal.summable.hasSum', 4),
 ('simp [Metric.diam, ENNReal.toReal_ofReal (sub_nonneg.2 {unknown})]', 4),
 ('simp_all only [ne_eq, ge_iff_le, min_eq_top, false_and, not_false_eq_true]',
  4),
 ('mono', 4),
 ('apply mem_of_superset {hypothesis}', 4),
 ('exact UniformContinuous fun p : {type} × {type} => {function} p.1 p.2', 4),
 ('have := {unknown}.1', 4),
 ('rintro ⟨x, y⟩', 4),
 ('intro i {unknown}', 4),
 ('· simpa only [image_preimage_inter] using {set}.image {function}', 4),
 ('simpa only [image_preimage_inter] using {set}.image {function}', 4),
 ("refine' ⟨fun hs => _, _⟩", 4),
 ('· exact {hypothesis} {variable} {unknown}', 4),
 ('apply dist_triangle', 4),
 ("refine' ⟨_, {unknown}⟩", 4),
 ('apply map_mem', 4),
 ('intro y hy z hz', 4),
 ('rw [tendsto_pi_nhds]', 4),
 ('rw [add_assoc, ENNReal.add_halves]', 4),
 ('intro {variable} xs', 4),
 ('haveI : PolishSpace {set} := {unknown}.polishSpace', 4),
 ('rcases eq_or_ne {variable} 0 with (rfl | hc)', 4),
 ('rw [← mem_cylinder_iff_eq]', 4),
 ('exact ⟨0, by simp⟩', 4),
 ('intro {set} hx', 4),
 ('specialize {hypothesis} {unknown} {unknown}', 4),
 ('intro ε hε', 4),
 ('rw [continuous_iff_continuousAt]', 4),
 ('apply edist_triangle', 4),
 ('erw [Pi.basis_apply]', 4),
 ('rw [SetLike.le_def]', 4),
 ('rw [linearIndependent_iff]', 4),
 ('use {variable}, {variable}', 4),
 ('exact Submodule.add_mem _ {hypothesis} {hypothesis}', 4),
 ('· infer_instance', 4),
 ('intro φ', 4),
 ('obtain ⟨vi, rfl⟩ := {unknown}.repr.symm.surjective {variable}', 4),
 ('rw [Finsupp.zero_apply]', 4),
 ('rcases {hypothesis} with ⟨{variable}, hy, {hypothesis}⟩', 4),
 ('rw [mem_graph_iff] at {hypothesis}', 4),
 ("cases' {hypothesis} with {variable} {hypothesis}", 4),
 ('rw [forall_mem_cons] at {hypothesis}', 4),
 ('intro x {unknown}', 4),
 ('refine ⟨fun h ↦ ?_, fun h ↦ ?_⟩', 4),
 ('rcases eq_or_ne {variable} 0 with (rfl | hx)', 4),
 ('use 0, 1', 4),
 ("refine' ⟨_, _⟩", 4),
 ('rwa [{function}]', 4),
 ('rintro {variable} hx', 4),
 ('haveI : DecidableEq {type} := Classical.decEq _', 4),
 ("induction' {unknown} with t {unknown} IH", 4),
 ('· rw [{hypothesis}]\n  simp [{hypothesis}]', 4),
 ('apply eq_of_sub_eq_zero', 4),
 ('exact Submodule.mem_top', 4),
 ("apply ciSup_le'", 4),
 ("refine' ⟨{function}, _, _⟩", 4),
 ('exact Subsingleton.elim _ _', 4),
 ('rw [← Finset.card_univ] at {unknown}', 4),
 ('interval_cases {unknown} <;> simp_all', 4),
 ('· tauto', 4),
 ('apply (algebraMap {type} {type}).injective', 4),
 ('by_cases hF : ringChar {type} = 2', 4),
 ('simp [Submodule.span_insert_eq_span]', 4),
 ("haveI := IsCyclotomicExtension.neZero' {unknown} {type} {type}", 4),
 ('rw [sub_add_cancel]', 4),
 ('rcases {unknown} with _ | {unknown}', 4),
 ('rcases {unknown}.eq_zero_or_pos with (rfl | hpos)', 4),
 ('rw [{unknown}, zero_mul] at {unknown}', 4),
 ('exact isIntegral_algebraMap', 4),
 ('intro h y', 4),
 ('induction {unknown} <;> simp', 4),
 ('trans Nat.gcd ({unknown} * {unknown}) ({unknown} * {unknown})', 4),
 ('rw [← {unknown}.1.1, ← {unknown}.2.1] at {unknown}', 4),
 ('dsimp at {hypothesis}', 4),
 ('rw [dif_pos {unknown}, ← Units.eq_iff, ← Units.coeHom_apply, map_prod, Units.val_mk0,\n  prod_congr rfl _]',
  4),
 ('by_cases hx : IsIntegral {type} {variable}', 4),
 ('exact zero_ne_one', 4),
 ('· exact algebraMap_ne_zero (denom_ne_zero {unknown})', 4),
 ('rintro _ ⟨i, rfl⟩', 4),
 ('intros z hz', 4),
 ('ext1 y', 4),
 ('rintro ⟨a, ha⟩', 4),
 ('rw [pow_succ, pow_succ]', 4),
 ('rwa [{hypothesis}]', 4),
 ('simp_rw [← {unknown}]', 4),
 ('intro a ha b hb', 4),
 ('exact {hypothesis} _ _', 4),
 ('constructor <;> intro H <;> ext', 4),
 ('rintro ⟨{variable}, ha⟩', 4),
 ('simp only [mem_iInf] at {hypothesis}', 4),
 ('· left\n  exact {hypothesis}', 4),
 ('rw [mul_assoc, {unknown}]', 4),
 ('induction {unknown} using Quot.inductionOn', 4),
 ("delta mk'", 4),
 ('simp only [{unknown}, zero_mul]', 4),
 ('exact LinearMap.congr_fun {unknown} {variable}', 4),
 ('rw [Multiset.prod_cons]', 4),
 ('apply dvd_mul_of_dvd_right', 4),
 ('letI : Nontrivial {type} := ⟨⟨{variable}, 0, {unknown}⟩⟩', 4),
 ('exact le_sup_left', 4),
 ('exact mul_mem_mul {hypothesis} {hypothesis}', 4),
 ('obtain {unknown} | {unknown} := {unknown}.eq_or_lt', 4),
 ('· exact {hypothesis} _ {unknown} _ _ {unknown}', 4),
 ('exact {hypothesis} _ {unknown} _ _ {unknown}', 4),
 ('· simp [← {unknown}] at {unknown}', 4),
 ('simp [← {unknown}] at {unknown}', 4),
 ('simp only [eq_iff_le_not_lt, {unknown}, {unknown}, true_and]', 4),
 ('obtain ⟨{variable}, rfl⟩ := exists_add_of_le {unknown}.le', 4),
 ('have := le_total {variable} 0', 4),
 ('rw [sub_add_sub_cancel]', 4),
 ('lift {unknown} to {type} using {unknown}', 4),
 ('rw [inv_eq_one_div]', 4),
 ('rcases lt_trichotomy {variable} 0 with (hb | rfl | hb)', 4),
 ('obtain ha | ha := {unknown}.prop.eq_or_lt', 4),
 ('rw [tendsto_iff_dist_tendsto_zero]', 4),
 ('all_goals isBoundedDefault', 4),
 ('lift {unknown} to ℝ≥0 using {hypothesis} {unknown} {hypothesis}', 4),
 ('apply inter_subset_right', 4),
 ('simp only [ENNReal.coe_ne_top, Ne.def, or_true_iff, not_false_iff]', 4),
 ('apply {hypothesis}.frequently.mono fun a ha => ?_', 4),
 ('rw [coe_nnreal_smul_apply]', 4),
 ('apply ge_of_tendsto {function}', 4),
 ('ext1 a', 4),
 ('apply lintegral_mono', 4),
 ('exact WithTop.zero_lt_top', 4),
 ('· exact (measurable_rnDeriv _ _).aemeasurable.const_smul _', 4),
 ('simp_rw [ENNReal.smul_def]', 4),
 ('rw [withDensity_smul _ (measurable_rnDeriv _ _)]', 4),
 ('exact (measurable_rnDeriv _ _).aemeasurable.const_smul _', 4),
 ('rwa [add_right_inj] at {unknown}', 4),
 ('rw [ne_eq, ENNReal.toNNReal_eq_zero_iff]', 4),
 ('intro B hB', 4),
 ('exact ne_of_lt (measure_lt_top _ _)', 4),
 ('apply subset_closure', 4),
 ("refine' ⟨{function}, _, {unknown}⟩", 4),
 ('specialize {hypothesis} {variable} {hypothesis}', 4),
 ('rw [← image_iUnion, ← inter_iUnion]', 4),
 ('choose δ hδ using {unknown}', 4),
 ('intro A', 4),
 ('exact measure_iUnion_le _', 4),
 ('apply lintegral_mono_ae', 4),
 ('rw [iff_iff_eq]', 4),
 ('rw [integrable_indicator_iff {unknown}.measurableSet]', 4),
 ('rw [MeasureTheory.locallyIntegrableOn_iff (Or.inr {unknown})] at {unknown} ⊢',
  4),
 ('by_cases h_top : {unknown} = ∞', 4),
 ("rw [snorm_eq_snorm' {unknown} {hypothesis}]", 4),
 ('simp_rw [{unknown}, snorm_exponent_top]', 4),
 ('simp_rw [← memℒp_one_iff_integrable]', 4),
 ('rw [lintegral_congr_ae]', 4),
 ('simp [{unknown}.2]', 4),
 ('haveI := Fact.mk {unknown}.lt_top', 4),
 ('rw [norm_sub_rev]', 4),
 ('rcases le_total {variable} {variable} with hab | hab', 4),
 ('simp [norm_smul]', 4),
 ('simp [Real.pi_pos.le]', 4),
 ('rw [tendsto_zero_iff_norm_tendsto_zero]', 4),
 ('· exact le_top', 4),
 ('exact {unknown} {set} {unknown}', 4),
 ('simp_rw [setToSimpleFunc]', 4),
 ('simp_rw [setToFun_undef _ {hypothesis}]', 4),
 ('rw [Measure.restrict_apply {unknown}] at {unknown}', 4),
 ('rw [← sub_ae_eq_zero]', 4),
 ('by_cases hi : MeasurableSet {set}', 4),
 ('exact {hypothesis} {set} {unknown}', 4),
 ('intro j _', 4),
 ('intro h s hs', 4),
 ('exact {unknown}.inter (MeasurableSet.disjointed {hypothesis} {unknown})',
  4),
 ('· exact not_measurable {unknown} {hypothesis}', 4),
 ('exact not_measurable {unknown} {hypothesis}', 4),
 ('exact ({unknown}.norm.const_mul _).mul_const _', 4),
 ('ext1 u hu', 4),
 ('rw [count_apply_infinite {hypothesis}]', 4),
 ('· rw [sub_nonneg]\n  exact {unknown}.mono {unknown}', 4),
 ('filter_upwards [{unknown}.ae_eq_mk, {unknown}.ae_eq_mk] with x hfx hgx', 4),
 ("rintro ⟨g', g'meas, hg'⟩", 4),
 ('apply measure_congr', 4),
 ('intro {set} hU r hr', 4),
 ('apply add_le_add_right', 4),
 ('rcases lt_iSup_iff.1 {unknown} with ⟨n, hn⟩', 4),
 ('constructor <;> intros', 4),
 ("refine' iSup_le _", 4),
 ('apply generateFrom_le', 4),
 ('have : ({variable}, {variable}) ∈ Function.support {function}.uncurry := {unknown}',
  4),
 ('exact mem_image_of_mem _ (subset_tsupport _ {hypothesis})', 4),
 ('exact measurable_fst {unknown}', 4),
 ('refine NullMeasurableSet.inter\n        ({unknown}.preimage quasiMeasurePreserving_fst) (NullMeasurableSet.inter ?_ ?_)',
  4),
 ('simp only [mem_setOf_eq, mem_compl_iff, not_lt]', 4),
 ('by_cases hx : {variable} ∈ {unknown}.support', 4),
 ('obtain ⟨{variable}, hx, -⟩ := id {unknown}', 4),
 ('rw [{unknown}] at {unknown} ⊢', 4),
 ('simp only [iUnion_singleton_eq_range, mem_range, image_univ, mem_image, exists_exists_eq_and]',
  4),
 ("simpa using support_formPerm_le' _ {hypothesis}", 4),
 ('rw [mem_cycleFactorsFinset_iff] at {hypothesis}', 4),
 ('rw [{unknown}, ← {unknown}]', 4),
 ('ext h', 4),
 ('rw [sup_inf_left]', 4),
 ("refine' Finset.subtype_map_of_mem fun x hx => _", 4),
 ('rw [← not_bounded_iff, ← not_bounded_iff, not_iff_not]', 4),
 ('cases PartialOrder.ext {hypothesis}', 4),
 ('obtain ⟨{variable}, rfl⟩ := {unknown} {variable}', 4),
 ('replace {unknown} := {unknown}.symm', 4),
 ('apply eq_of_forall_ge_iff', 4),
 ('obtain ⟨y, hy⟩ := not_forall_not.mp {hypothesis}', 4),
 ('simp [← eq_iff_iff]', 4),
 ('rintro - ⟨{function}, rfl⟩', 4),
 ('show ite _ _ _ ∈ _', 4),
 ('rintro (⟨⟩ | a) ha', 4),
 ("refine' ⟨fun {set} => _⟩", 4),
 ('exact {hypothesis}.comap _', 4),
 ('norm_cast at *', 4),
 ('apply ({hypothesis}.and {hypothesis}).mono', 4),
 ('intro hg', 4),
 ('simpa only [sub_eq_add_neg] using {unknown}.neg.const_add {variable}', 4),
 ('rw [← convolution_flip]', 4),
 ('rw [inter_comm] at {unknown}', 4),
 ("have hxyD' : Ioo {variable} {variable} ⊆ interior {set} :=\n  subset_sUnion_of_mem ⟨isOpen_Ioo, Ioo_subset_Icc_self.trans {unknown}⟩",
  4),
 ('rw [← sub_self {variable}]', 4),
 ('rw [iteratedFDeriv_succ_eq_comp_left]', 4),
 ('unfold derivWithin', 4),
 ('rw [inner_eq_zero_symm]', 4),
 ('exact {unknown}.of_le le_top', 4),
 ('ext y v', 4),
 ('intro {variable} _', 4),
 ("rw [Metric.cauchySeq_iff']", 4),
 ('have : ProperSpace {type} := .of_locallyCompactSpace {type}', 4),
 ('rw [map_zero]', 4),
 ("refine' eventually_atTop.2 ⟨1, fun x hx ↦ _⟩", 4),
 ('exact {unknown} {hypothesis} {unknown} {unknown} {unknown}', 4),
 ("obtain rfl | hb' := {unknown}.eq_or_lt", 4),
 ('rw [smul_add, smul_add, add_add_add_comm]', 4),
 ('rw [← one_mul {variable}, ← {unknown}, add_mul]', 4),
 ("simp_rw [div_eq_iff {unknown}.ne', ← {unknown}]", 4),
 ("have hxy' : 0 < {variable} - {variable} := by linarith", 4),
 ("have hyz' : 0 < {variable} - {variable} := by linarith", 4),
 ('intro {variable} hx y hy a b ha hb hab', 4),
 ('simp only [IsBigOWith_def]', 4),
 ('simp only [IsLittleO_def]', 4),
 ('simp only [IsLittleO_def] at *', 4),
 ('have cne0 : ‖{variable}‖ ≠ 0 := norm_ne_zero_iff.mpr {unknown}', 4),
 ('rcases {unknown}.basisSets_iff.mp {hypothesis} with ⟨s, r, hr, {hypothesis}⟩',
  4),
 ('specialize {unknown} {hypothesis}', 4),
 ('obtain ⟨a, b, ha, hb, hab, rfl⟩ := {hypothesis}', 4),
 ("obtain ⟨x', hx''⟩ := {unknown}.surjective.exists.mp ⟨{variable}, rfl⟩", 4),
 ("obtain ⟨y', hy''⟩ := {unknown}.surjective.exists.mp ⟨{variable}, rfl⟩", 4),
 ('simp only [{unknown}, {unknown}, OrderIso.symm_apply_apply, gt_iff_lt]', 4),
 ('exact ⟨⟨{variable}, {unknown}⟩, {unknown}⟩', 4),
 ('rw [pow_two]', 4),
 ('lift {unknown} to ℝ using {unknown}', 4),
 ('rw [{unknown}, invOf_eq_inv, ← div_eq_mul_inv]', 4),
 ('simpa [edist_eq_coe_nnnorm_sub] using {hypothesis}', 4),
 ('simp [sin_add]', 4),
 ('simp [cos_add]', 4),
 ('simp [sub_eq_add_neg, sin_add]', 4),
 ('nth_rw 1 [← coe_toReal {unknown}]', 4),
 ('rw [ringConGen_eq]', 4),
 ('rw [rpow_def_of_pos {unknown}]', 4),
 ('rw [← one_rpow {variable}]', 4),
 ('exact (rpow_zero {variable}).symm', 4),
 ('apply AddSubmonoid.add_mem_sup', 4),
 ('· exact Set.mem_range_self _', 4),
 ('rw [← abs_mul_cos_add_sin_mul_I {unknown}, {unknown}]', 4),
 ('obtain ⟨s, ⟨b⟩⟩ := {hypothesis}', 4),
 ('simp [Algebra.smul_def, ← {unknown}]', 4),
 ('apply Continuous.intervalIntegrable', 4),
 ('simp [mem_prod, MonoidHom.coe_fst]', 4),
 ('simp [mem_prod, MonoidHom.coe_snd]', 4),
 ('refine ⟨?_, fun ⟨i, hi⟩ ↦ le_iSup {function} i hi⟩', 4),
 ('simpa using @{unknown} {variable}', 4),
 ('apply AddSubgroup.zero_mem _', 4),
 ('apply AddSubgroup.add_mem _ {hypothesis} {hypothesis}', 4),
 ('apply AddSubgroup.neg_mem _ {hypothesis}', 4),
 ('obtain (rfl|hn) := {unknown}.eq_zero_or_pos', 4),
 ('apply monic_prod_of_monic', 4),
 ('· simp [{hypothesis}, mul_coeff]', 4),
 ('ext ⟨a1, a2⟩', 4),
 ('rintro ⟨s, ⟨f, hsur⟩⟩', 4),
 ("induction' {unknown} using induction_on with m {unknown} g ihf ihg r {unknown} ih",
  4),
 ('· rcases {hypothesis} with ⟨P, rfl⟩\n  exact ⟨{variable} • P, AlgHom.map_smul _ _ _⟩',
  4),
 ('have : {variable} ∈ closure {set} := {unknown}.symm ▸ mem_top _', 4),
 ('· exact ⟨1, AlgHom.map_one _⟩', 4),
 ('exact ⟨1, AlgHom.map_one _⟩', 4),
 ('rintro m₁ m₂ ⟨P₁, hP₁⟩ ⟨P₂, hP₂⟩', 4),
 ('rcases {hypothesis} with ⟨Q, rfl⟩', 4),
 ('exact ⟨{unknown} + {unknown}, AlgHom.map_add _ _ _⟩', 4),
 ('exact ⟨{variable} • {unknown}, AlgHom.map_smul _ _ _⟩', 4),
 ("induction' {unknown} using Nat.strong_induction_on with {unknown} ih", 4),
 ('intro hI', 4),
 ('have := {hypothesis}.1.1', 4),
 ('intros j hj', 4),
 ('exact {unknown} _ {unknown}', 4),
 ('rw [Nat.add_right_comm]', 4),
 ('intro | rfl => rfl', 4),
 ('have ⟨ax, ap⟩ := All_and.1 {unknown}', 4),
 ('have ⟨xb, bp⟩ := All_and.1 {unknown}', 4),
 ('unfold del', 4),
 ('simp [List.reverseAux_eq]', 4),
 ("cases {unknown}.out'", 4),
 ('constructor <;> assumption', 4),
 ('apply Int.le_refl', 4),
 ('apply Int.dvd_mul_right', 4),
 ('simp [lowerBound?]', 4),
 ('simp [← mem_toList, {unknown}] at {hypothesis}', 4),
 ('· exact Int.add_le_add_right {unknown} {unknown}', 4),
 ('exact Int.add_le_add_right {unknown} {unknown}', 4),
 ('· exact Int.neg_le_neg {unknown}', 4),
 ('exact Int.neg_le_neg {unknown}', 4),
 ('simp_all [{function}, div]', 4),
 ('apply Int.le_of_sub_nonneg', 4),
 ('exact aux {unknown}', 4),
 ('rcases le_or_gt 0 {variable} with h | h', 4),
 ('rcases eq_zero_or_eq_zero_of_mul_eq_zero {unknown} with h1 | h1', 4),
 ('· right\n  exact eq_neg_iff_add_eq_zero.mpr {unknown}', 4),
 ('. left\n  exact eq_of_sub_eq_zero {unknown}', 4),
 ('rw [{unknown}, sub_self]', 4),
 ('exact eq_neg_iff_add_eq_zero.mpr {unknown}', 4),
 ('exact eq_of_sub_eq_zero {unknown}', 4),
 ('linarith [{unknown}.two_le]', 4),
 ('apply le_sup_right', 4),
 ('intro f_eq', 3),
 ('rw [eq_of_cmp {unknown}, eq_of_cmp {unknown}]', 3),
 ('rw [div_eq]', 3),
 ('intro h₂', 3),
 ('split <;> simp_all', 3),
 ('exact {unknown}.down', 3),
 ('simp [insert]', 3),
 ('apply insert_nonEmpty', 3),
 ('. rfl', 3),
 ('rw [{unknown}.1, {unknown}.assoc.1, {unknown}.1 (evalList _ _ _)]', 3),
 ('simp only [{unknown}] at {unknown}', 3),
 ('simp [norm]', 3),
 ('simp_arith at {unknown}', 3),
 ('rw [Nat.add_comm]', 3),
 ('injection {unknown} with {unknown}', 3),
 ('apply Nat.pred_lt', 3),
 ('simp [Nat.succ_add]', 3),
 ('simp [Poly.denote_eq, Poly.denote_le] at {unknown} {unknown} |-', 3),
 ('apply Nat.lt_of_succ_le', 3),
 ('have := Nat.not_eq_zero_of_lt (Poly.of_isNonZero {unknown} {unknown})', 3),
 ('apply Nat.succ_lt_succ', 3),
 ('intro p', 3),
 ('rw [add_mul, one_mul]', 3),
 ('exact forall_swap', 3),
 ('rintro ⟨b, hb₁, hb₂⟩', 3),
 ('simp at *', 3),
 ('exact dif_pos {unknown}', 3),
 ('by_cases b0 : {unknown} = 0', 3),
 ('obtain rfl | hn := eq_or_ne {unknown} 0', 3),
 ('rw [mul_def, normalize_eq]', 3),
 ('apply And.intro', 3),
 ('rw [Finset.forall_mem_cons] at {hypothesis}', 3),
 ('lift {unknown} to ℝ≥0 using {unknown}.ne_top', 3),
 ('simpa only [Ne.def, coe_eq_zero] using {unknown}', 3),
 ('apply lt_of_lt_of_le _ {unknown}', 3),
 ('apply lt_of_le_of_ne', 3),
 ('intro a b hab', 3),
 ('intro N {unknown} {unknown} g Cf Cg Nf Og', 3),
 ('rw [← C_mul_X_pow_eq_monomial]', 3),
 ('by_contra', 3),
 ('exact Nat.succ_ne_zero _', 3),
 ("induction' {unknown} with {unknown} hn generalizing {unknown}", 3),
 ('rw [Polynomial.map_mul]', 3),
 ('nth_rw 1 [eq_prod_roots_of_monic_of_splits_id {unknown} {unknown}]', 3),
 ("by_contra! h'", 3),
 ('congr 3', 3),
 ("refine' (coeff_mul _ _ _).trans (Finset.sum_eq_zero fun x hx => _)", 3),
 ('rw [Finset.sum_eq_zero]', 3),
 ('rw [← WithBot.coe_eq_coe]', 3),
 ('rw [coeff_eq_zero_of_natDegree_lt]', 3),
 ('rw [pow_succ] at {unknown}', 3),
 ('simp [{hypothesis} {unknown} le_rfl]', 3),
 ('rw [degree_eq_natDegree {unknown}]', 3),
 ('unfold leadingCoeff', 3),
 ('· simp only [eq_iff_true_of_subsingleton]', 3),
 ('by_contra hc', 3),
 ('by_contra hcontra', 3),
 ('by_contra hp', 3),
 ('obtain ⟨u, rfl⟩ := {unknown}', 3),
 ('rw [← {unknown}, {unknown}] at {unknown}', 3),
 ('rw [WellFounded.fix_eq]', 3),
 ('simp only [if_neg {hypothesis}]', 3),
 ('apply Nat.pos_of_ne_zero', 3),
 ("induction' {unknown} with {unknown} hm", 3),
 ('simp [{unknown}, {function}]', 3),
 ("rw [Finset.sup'_le_iff]", 3),
 ('· simpa [{hypothesis}] using {unknown}', 3),
 ('obtain _ | _ | {unknown} := {unknown}', 3),
 ("any_goals refine' ⟨fun h => Option.noConfusion h, fun h => _⟩;\n            aesop (simp_config := { decide := true })",
  3),
 ("refine' ⟨fun h => Option.noConfusion h, fun h => _⟩", 3),
 ('aesop (simp_config := { decide := true })', 3),
 ('exact (not_lt_bot {unknown}).elim', 3),
 ('simp [bit0]', 3),
 ('simp [add_mul]', 3),
 ('unfold modByMonic divModByMonicAux', 3),
 ('apply coeff_eq_zero_of_natDegree_lt', 3),
 ('letI := {unknown}.toField', 3),
 ('exact by decide', 3),
 ('have := {hypothesis} {unknown}', 3),
 ('rw [← sq]', 3),
 ('exact add_comm _ _', 3),
 ('by_cases h : {unknown} = 1', 3),
 ("refine' {unknown} _", 3),
 ('rw [mem_image]', 3),
 ('rw [mem_Ico] at {hypothesis}', 3),
 ('rw [add_comm {unknown} {unknown}]', 3),
 ('rw [Nat.mul_comm]', 3),
 ('exact mod_add_div _ _', 3),
 ('exact div_add_mod _ _', 3),
 ('dsimp [ofDigits]', 3),
 ('apply Nat.strongInductionOn {unknown}', 3),
 ("induction' {unknown} with {unknown}", 3),
 ('simp only [map_zero]', 3),
 ('simp only [not_lt] at {hypothesis}', 3),
 ('apply add_lt_add_right', 3),
 ('apply le_top', 3),
 ('apply Nat.succ_le_of_lt', 3),
 ('exact Option.noConfusion {unknown}', 3),
 ('· exact {hypothesis} {unknown} {unknown} {unknown}', 3),
 ('apply Nat.le_add_left', 3),
 ('apply Iff.intro', 3),
 ('simp [sq]', 3),
 ('by_contra contra', 3),
 ('rw [tsub_add_cancel_of_le {unknown}]', 3),
 ('rw [bot_lt_iff_ne_bot]', 3),
 ("refine' mt _ {unknown}", 3),
 ('exact add_le_add_left {unknown} _', 3),
 ('intro a h', 3),
 ('rcases {unknown}.eq_zero_or_pos with (rfl | hb)', 3),
 ('apply mul_left_cancel₀ (factorial_ne_zero {unknown})', 3),
 ('exact lt_add_one _', 3),
 ('exact Nat.succ_lt_succ {unknown}', 3),
 ('bitwise_assoc_tac', 3),
 ('rw [div_add_mod]', 3),
 ('simp [and_assoc]', 3),
 ('simp [mod_zero]', 3),
 ('rw [← tsub_le_iff_right]', 3),
 ('rw [tsub_le_iff_right]', 3),
 ('intro q', 3),
 ("rcases em' {unknown}.Prime with (pp | pp)", 3),
 ('intro {unknown} hp', 3),
 ('apply induction_linear {unknown}', 3),
 ('rw [add_right_comm]', 3),
 ('rw [disjoint_left]', 3),
 ('simp [and_assoc, and_left_comm]', 3),
 ("induction' {unknown} with k ih", 3),
 ('simpa using {unknown}.2.2', 3),
 ('simpa using {unknown}.2.1', 3),
 ("refine' (iUnion₂_subset fun x hx => _).antisymm fun x hx => _", 3),
 ('exact mem_biUnion {hypothesis} {unknown}', 3),
 ('refine ⟨fun h => ?_, fun h => ?_⟩', 3),
 ('simp [range]', 3),
 ('rintro _ ⟨{variable}, ha, rfl⟩', 3),
 ('rcases ({set} ×ˢ {set}).eq_empty_or_nonempty with h | h', 3),
 ('exact mem_image_of_mem _ {hypothesis}.1', 3),
 ('rcases {set} with ⟨y, hy⟩', 3),
 ("repeat' rw [← coprime_coe]", 3),
 ('rw [mul_coe]', 3),
 ('rw [← coprime_coe]', 3),
 ('conv_lhs => rw [← {unknown}]', 3),
 ('rw [mem_iUnion₂]', 3),
 ('by_cases h : {variable} ∈ {unknown} <;> simp [h]', 3),
 ('exact preimage_mono {unknown}', 3),
 ('rw [range_comp]', 3),
 ('exact ⟨{variable}, {hypothesis}, {function}⟩', 3),
 ('rw [range_subset_iff]', 3),
 ('apply Subtype.ext', 3),
 ('intro s t h', 3),
 ('by_cases {hypothesis} <;> simp [*]', 3),
 ('funext a b', 3),
 ('rw [ext_iff] at {function}', 3),
 ('rw [pmap_eq_map_attach]', 3),
 ('apply symm', 3),
 ('rw [← not_imp_not]', 3),
 ('simp only [add_zero]', 3),
 ('rcases {set} with ⟨z, hz⟩', 3),
 ('rw [sInter_eq_biInter]', 3),
 ('triv', 3),
 ('inhabit {unknown}', 3),
 ('exact perm_append_comm.append_right _', 3),
 ('funext s', 3),
 ("induction' {unknown} with a {unknown} ih", 3),
 ('by_cases h_cases : {variable} = {variable}', 3),
 ('apply map_congr', 3),
 ('exact image_subset _ {unknown}', 3),
 ('simp [piecewise]', 3),
 ("cases' {hypothesis} with h₁ h₂", 3),
 ('rw [if_neg, if_neg]', 3),
 ('obtain ⟨a, rfl⟩ := {hypothesis}', 3),
 ("refine' Finset.sum_congr rfl _", 3),
 ('simp_rw [{function}]', 3),
 ('intro s _', 3),
 ("induction' {unknown} with head tail ih", 3),
 ('simp only [reverse_reverse]', 3),
 ('obtain ⟨f, hf⟩ := {hypothesis}', 3),
 ('rw [← {hypothesis}, ← {hypothesis}]', 3),
 ('rcases {unknown}.eq_or_lt with ({unknown} | {unknown})', 3),
 ("cases' {unknown} with {variable} {unknown}", 3),
 ('apply List.ext_nthLe', 3),
 ('obtain ⟨n, hn, rfl⟩ := nthLe_of_mem {hypothesis}', 3),
 ("induction' {unknown} using Quot.inductionOn with l", 3),
 ("induction {unknown} using Quotient.inductionOn'", 3),
 ('by_cases hc : {variable} < {variable}', 3),
 ('rcases le_total {variable} {variable} with hab | hab <;> simp [hab] at {unknown}',
  3),
 ('have hxb : {variable} < {variable} := (lt_of_not_ge {hypothesis}).trans_le {unknown}',
  3),
 ('have hxb : {variable} ≤ {variable} := (le_of_not_ge {hypothesis}).trans {unknown}',
  3),
 ('· simp only [{hypothesis}, true_and]', 3),
 ('simp only [{unknown}, true_and]', 3),
 ('simp only [{unknown}, true_or]', 3),
 ('rcases le_total {variable} {variable} with hcd | hcd <;> simp [hcd] at {unknown}',
  3),
 ('rw [← append_assoc]', 3),
 ('exact {unknown}.right', 3),
 ('simpa [{unknown}, {unknown}] using {unknown}', 3),
 ('apply pairwise_iff_forall_sublist.mpr', 3),
 ('intro {variable} {function} hab', 3),
 ('· exact rel_of_sorted_cons {unknown} _ {hypothesis}', 3),
 ('exact rel_of_sorted_cons {unknown} _ {hypothesis}', 3),
 ('rw [tsub_lt_iff_left {unknown}]', 3),
 ('rintro ⟨r, rfl⟩', 3),
 ('rintro ⟨m, hm⟩', 3),
 ('rw [{hypothesis} {variable}]', 3),
 ('exact ⟨{unknown}, {unknown}, {unknown}.cons _⟩', 3),
 ('simp only [find?, mem_cons] at {hypothesis}', 3),
 ('· exact Or.inr (Or.inl {hypothesis})', 3),
 ('exact Or.inr (Or.inl {hypothesis})', 3),
 ('cases {unknown} <;> cases {unknown} <;> cases {unknown} <;> simp [{hypothesis}]',
  3),
 ('letI : NormedSpace ℝ {type} := RestrictScalars.normedSpace ℝ {type} {type}',
  3),
 ('rcases {set}.eq_empty_or_nonempty with (rfl | ⟨y, hy⟩)', 3),
 ("refine' ⟨_, half_pos {unknown}, {unknown}, fun j ij => _⟩", 3),
 ("induction' {unknown} with b {unknown} ih", 3),
 ('exact Nat.sub_lt_left_of_lt_add {unknown} {unknown}', 3),
 ('induction {variable} using Real.ind_mk', 3),
 ('apply MvPolynomial.induction_on {unknown}', 3),
 ('congr with {variable}', 3),
 ('unfold eval', 3),
 ('simp [{function}, {function}]', 3),
 ('solve_by_elim', 3),
 ("repeat' rw [degreeOf]", 3),
 ('funext {variable}', 3),
 ('rw [Multiset.map_map]', 3),
 ('· exact zero_le _', 3),
 ('rw [Finsupp.sum_fintype]', 3),
 ('apply {function}.intro', 3),
 ('by_cases hji : {variable} = {variable}', 3),
 ('rw [← not_forall]', 3),
 ('exact Nat.find_spec {hypothesis}', 3),
 ("refine' le_antisymm (mk_real ▸ mk_set_le _) _", 3),
 ('tfae_have 4 → 3', 3),
 ('tfae_have 3 → 2', 3),
 ('exact ⟨1, by simp⟩', 3),
 ('exact {hypothesis} _ _ _', 3),
 ('rw [← add_halves {variable}]', 3),
 ('let ⟨H₁, H₂⟩ := {hypothesis} _ {unknown}', 3),
 ('exact ⟨{variable}⟩', 3),
 ('rwa [← {unknown}] at {hypothesis}', 3),
 ('exact ⟨{variable}, {hypothesis}.2⟩', 3),
 ('rcases {unknown} with ⟨⟩ <;> rcases {unknown} with ⟨⟩ <;> simp [fromBlocks]',
  3),
 ("ext i' j'", 3),
 ('have := ext_iff.mpr {unknown} {variable} {variable}', 3),
 ('simp [{unknown}, zero_le_one]', 3),
 ('rwa [add_halves] at {unknown}', 3),
 ('ext i hi', 3),
 ('intro f g', 3),
 ('exact ({hypothesis} rfl).elim', 3),
 ('rw [eqvGen_eq]', 3),
 ('exact sInf_le {hypothesis}', 3),
 ('ext ⟨{variable}, {variable}⟩', 3),
 ('rw [insert_eq_of_mem {hypothesis}]', 3),
 ('rintro ⟨a, ha, rfl⟩', 3),
 ('obtain ⟨{variable}, ha⟩ := {unknown}', 3),
 ('simp only [{hypothesis}, update_noteq, Ne.def, not_false_iff]', 3),
 ("refine' Fin.cases _ _", 3),
 ('apply funext', 3),
 ('rw [{function}] at {unknown}', 3),
 ('exact {unknown} ▸ {unknown}', 3),
 ("cases' {hypothesis} with i hi", 3),
 ('by_cases h : {variable} ≤ {variable}', 3),
 ("obtain ⟨{unknown}, rfl⟩ := Nat.exists_eq_succ_of_ne_zero {unknown}.ne'", 3),
 ('apply Nat.gcd_eq_gcd_ab', 3),
 ('· exact Or.inr {unknown}', 3),
 ('· rw [Nat.bitwise_swap, Function.swap]\n  congr\n  funext x y\n  cases x <;> cases y <;> rfl',
  3),
 ('rw [Nat.bitwise_swap, Function.swap]', 3),
 ('by_cases hb : {unknown} = 0', 3),
 ('intro he', 3),
 ('rw [sub_eq_iff_eq_add] at {unknown}', 3),
 ('exact isUnit_one', 3),
 ('simp [dotProduct]', 3),
 ("refine' Finset.cons_induction_on {unknown} _ fun b t _ h => _", 3),
 ('simp [or_imp, forall_and]', 3),
 ('simp [@forall_swap _ {type}]', 3),
 ('obtain ⟨a, ha⟩ := {unknown}', 3),
 ('clear {unknown} {unknown} {unknown}', 3),
 ('simp [{unknown}, lt_irrefl] at {unknown}', 3),
 ('rw [← sum_add_distrib]', 3),
 ('exact (re_add_im {unknown}).symm', 3),
 ("refine' Finset.sum_congr rfl fun _ _ => _", 3),
 ('rw [Finset.mem_singleton]', 3),
 ('intro {variable} {unknown}', 3),
 ("induction' {unknown} with k hk", 3),
 ('rwa [zero_mul]', 3),
 ('rw [← sub_nonneg] at {unknown}', 3),
 ('intro ht', 3),
 ("convert Int.ediv_add_emod' (2 * {unknown} + 1) 2", 3),
 ('simp [Int.add_emod]', 3),
 ('rintro X', 3),
 ('rintro X {unknown} f', 3),
 ('apply {unknown}.symm', 3),
 ('rw [cast_ofNat, Nat.cast_ofNat]', 3),
 ('simp only [{function}]', 3),
 ('· exact {unknown}.1', 3),
 ('apply Set.mem_range_self', 3),
 ('rcases {unknown} with ⟨{unknown}, {unknown}⟩', 3),
 ('intro hj', 3),
 ('subst {unknown} {unknown} {unknown}', 3),
 ('rw [← sub_eq_zero] at {unknown} ⊢', 3),
 ('simp only []', 3),
 ('exact {unknown}.2.1', 3),
 ('rw [Nat.mul_add, ← Nat.mul_assoc, Nat.add_assoc]', 3),
 ('rwa [{unknown}.out, zpow_coe_nat]', 3),
 ('rwa [{unknown}.out, Int.cast_negOfNat, zpow_neg, zpow_coe_nat]', 3),
 ('rwa [mul_comm] at {unknown}', 3),
 ('· rintro rfl\n  rfl', 3),
 ('rw [NormNum.IsInt.neg_to_eq {unknown} rfl]', 3),
 ('ext X', 3),
 ('rintro ⟨H⟩', 3),
 ('· apply {unknown}', 3),
 ('rcases le_one_iff.1 {unknown} with (rfl | rfl)', 3),
 ('exact zero_mul _', 3),
 ('rw [opow_succ]', 3),
 ("cases' {unknown} with f", 3),
 ('exact absurd {unknown} {unknown}', 3),
 ('rw [derivFamily_zero]', 3),
 ('rw [derivFamily_succ]', 3),
 ('rw [← sup_iterate_eq_nfp]', 3),
 ('unfold deriv', 3),
 ('rcases eq_zero_or_pos {unknown} with ha | ha', 3),
 ('rw [or_iff_not_imp_left]', 3),
 ('intro hb', 3),
 ('exact eq_comm', 3),
 ('rw [mul_succ]', 3),
 ('rwa [forall_range_iff]', 3),
 ('refine ⟨?_, ?_⟩', 3),
 ('rintro ⟨f, hf, rfl⟩', 3),
 ('rw [← succ_aleph0]', 3),
 ('rcases le_or_lt ℵ₀ {unknown} with hb | hb', 3),
 ('have := {unknown}', 3),
 ('exact {unknown}.mem_nhds {hypothesis}', 3),
 ('exact isClosed_closure', 3),
 ('apply {hypothesis}.2', 3),
 ("intro V' V'_in V'_closed", 3),
 ('· exact {hypothesis} _ _ {unknown}', 3),
 ('rwa [mk_ordinal_out]', 3),
 ('rwa [{unknown}.card.lift_id]', 3),
 ('exact {unknown}.2.2', 3),
 ('exact lt_of_le_of_ne {unknown}.1 {unknown}.symm', 3),
 ('intro {set} H', 3),
 ('tfae_have 4 → 5', 3),
 ('rw [union_eq_iUnion]', 3),
 ('rw [compl_inter]', 3),
 ('rw [nhds_def]', 3),
 ('exact {unknown}.comp {unknown}', 3),
 ('apply continuous_of_continuous_uncurry', 3),
 ("rw [nhds_coe, tendsto_map'_iff]", 3),
 ('exact tendsto_id', 3),
 ('simp only [true_and]', 3),
 ('exact @Continuous.comp _ _ _ (id _) (id _) _ _ _ {function} {function}', 3),
 ('rw [closure_prod_eq]', 3),
 ('rintro ⟨{hypothesis}, t, ⟨h1, h2⟩⟩', 3),
 ('by_cases hi : {variable} ∈ {unknown}', 3),
 ('rw [toPartENat_apply_of_lt_aleph0 {unknown}]', 3),
 ('rw [toPartENat_apply_of_lt_aleph0 {unknown}, PartENat.coe_le_coe]', 3),
 ('rw [toPartENat_apply_of_aleph0_le {unknown}]', 3),
 ('rw [image_eq_range] at {unknown}', 3),
 ('clear {hypothesis} {function}', 3),
 ('use {set}', 3),
 ('exact ⟨{set}, {hypothesis}, Subset.rfl⟩', 3),
 ('tfae_have 1 → 3', 3),
 ('exact univ_mem', 3),
 ('delta nhdsWithin', 3),
 ('rintro ⟨k, hk⟩', 3),
 ('exact addOrderOf_div_of_gcd_eq_one {unknown} {unknown}', 3),
 ('exact mem_of_superset {hypothesis} {hypothesis}', 3),
 ('letI := nhdsAdjoint {variable} {unknown}', 3),
 ('simpa only [mul_zero] using mul_lt_mul_of_pos_left {unknown} {unknown}', 3),
 ('exact {unknown}.false', 3),
 ('rw [enumOrd_def]', 3),
 ('simp only [{unknown}, mul_zero]', 3),
 ('tfae_have 4 → 2', 3),
 ('tfae_have 1 ↔ 5', 3),
 ('rw [← {hypothesis}] at {hypothesis}', 3),
 ('intro hyp', 3),
 ("rw [{unknown}.restr_source' _ {unknown}.open_source]", 3),
 ('mfld_set_tac', 3),
 ('rw [dist_eq]', 3),
 ('rw [← Nat.cofinite_eq_atTop]', 3),
 ('simp only [SetCoe.forall, Subtype.coe_mk, iUnion_subset_iff]', 3),
 ('obtain ⟨t, ht, htsymm, htr⟩ := comp_symm_mem_uniformity_sets {hypothesis}',
  3),
 ('simp only [Finset.sum_insert {hypothesis}]', 3),
 ('lift {function} to {type} → ℝ≥0 using ENNReal.ne_top_of_tsum_ne_top {unknown}',
  3),
 ('lift {unknown} to ℝ≥0 using {hypothesis}', 3),
 ('apply ENNReal.ofReal_le_of_le_toReal', 3),
 ('intro y h', 3),
 ('delta UniformContinuous', 3),
 ('exact @UniformContinuous.comp _ _ _ (id _) (id _) _ _ _ {function} {unknown}',
  3),
 ('exact {function} rfl', 3),
 ('by_cases hf : UniformContinuous {function}', 3),
 ('exact isPreconnected_connectedComponent', 3),
 ('rw [{unknown}.inter_eq, inter_empty]', 3),
 ('intro u hu {variable} hx', 3),
 ('specialize {unknown} {set} {set} {unknown} {unknown} {unknown}', 3),
 ('exact inter_subset_inter_left _ {unknown}', 3),
 ('obtain rfl | h := {set}.eq_empty_or_nonempty', 3),
 ('rw [connectedComponentIn]', 3),
 ("cases' isEmpty_or_nonempty {type} with hα hα", 3),
 ('apply eq_of_subset_of_subset', 3),
 ('rw [← biUnion_preimage_singleton]', 3),
 ('rw [isOpen_iff_mem_nhds]', 3),
 ('exact ⟨_, {hypothesis}⟩', 3),
 ('rw [mul_comm, ← div_eq_mul_inv]', 3),
 ('rcases le_or_lt {unknown} {unknown} with h | h', 3),
 ('contrapose! {function}', 3),
 ('exact le_succ _', 3),
 ("refine' le_antisymm _ bot_le", 3),
 ('exact ⟨{unknown}, {unknown}.le⟩', 3),
 ('rcases {set} with ⟨{variable}, xs⟩', 3),
 ('rcases {set}.eq_empty_or_nonempty with (rfl | hs)', 3),
 ('rcases mem_uniformity_dist.1 {hypothesis} with ⟨{variable}, εpos, hε⟩', 3),
 ('· exact {unknown} {hypothesis}', 3),
 ('rcases mem_uniformity_isClosed {hypothesis} with ⟨V, hV, hVclosed, hVU⟩',
  3),
 ('exact (continuous_apply ⟨{variable}, {hypothesis}⟩).comp {unknown}', 3),
 ('simp_rw [norm_def, ← dist_zero_right]', 3),
 ('rw [{unknown}.tendsto_right_iff]', 3),
 ('intro x₀', 3),
 ('split_ifs at {unknown} with h', 3),
 ('exact apply_eq_of_lt_firstDiff ({unknown}.trans_le {unknown})', 3),
 ('rcases eq_or_ne {function} {function} with (rfl | hne)', 3),
 ('exact {hypothesis} {unknown} {hypothesis}', 3),
 ("refine' ⟨{function}, {hypothesis}, _⟩", 3),
 ('rw [{unknown}, {unknown}] at {unknown}', 3),
 ('choose x hx using {hypothesis}', 3),
 ('cases {unknown} <;> assumption', 3),
 ('letI : TopologicalSpace ({type} →SL[{function}] {type}) := strongTopology {function} {type} {set}',
  3),
 ('rw [map_smul]', 3),
 ('by_cases ha0 : {variable} = 0', 3),
 ('rw [mem_ball_zero_iff] at {hypothesis}', 3),
 ('split_ifs <;> norm_num', 3),
 ('letI : TopologicalSpace {type} := Preorder.topology {type}', 3),
 ('haveI : OrderTopology {type} := ⟨rfl⟩', 3),
 ('rw [leftLim_eq_sSup {unknown} {unknown}]', 3),
 ('rw [← LinearMap.ker_eq_bot]', 3),
 ('simp only [div_eq_mul_inv]', 3),
 ('rw [closure_Ioo {unknown}.ne]', 3),
 ('apply Or.inr', 3),
 ('exact zero_lt_one', 3),
 ('exact {unknown}.add {unknown}', 3),
 ('rw [codisjoint_iff_le_sup]', 3),
 ('have : UniformAddGroup {type} := comm_topologicalAddGroup_is_uniform', 3),
 ('rw [Nat.zero_add]', 3),
 ('rwa [Nat.sub_add_cancel {unknown}]', 3),
 ('exact {unknown}.tail {unknown}', 3),
 ('· left\n  rfl', 3),
 ('exact {unknown} _', 3),
 ('· ext\n  simp', 3),
 ('unfold swapCore', 3),
 ('· cases {hypothesis} rfl', 3),
 ('rintro h', 3),
 ('refine ⟨fun _ _ ↦ id, fun x ↦ ⟨x, rfl⟩⟩', 3),
 ('simp only [not_le]', 3),
 ('tfae_have 1 ↔ 3', 3),
 ('exact ({unknown} {hypothesis}).2', 3),
 ('· ext; simp', 3),
 ('rintro ⟨i, j⟩', 3),
 ('exact Submodule.zero_mem _', 3),
 ('ext m n', 3),
 ('rw [span_le]', 3),
 ('rw [Set.mem_singleton_iff] at {hypothesis}', 3),
 ('rw [lt_top_iff_ne_top] at {unknown}', 3),
 ('have := {unknown} {hypothesis}', 3),
 ('simp [*] at *', 3),
 ('rcases eq_or_ne {variable} {variable} with h | h', 3),
 ('rw [Finset.sum_comm]', 3),
 ('cases nonempty_fintype {set}', 3),
 ('dsimp only [IsOrtho]', 3),
 ('specialize {hypothesis} ({unknown} {variable})', 3),
 ('specialize {hypothesis} ⟨{variable}, {hypothesis}⟩', 3),
 ('intro ho', 3),
 ('intro hi', 3),
 ('rw [mem_graph_iff] at {hypothesis} ⊢', 3),
 ('simp only at {unknown}', 3),
 ('use ⟨{variable}, {hypothesis}⟩', 3),
 ('· exact zero_mem _', 3),
 ("refine' sum_mem _", 3),
 ('letI := FiniteDimensional.of_fintype_basis {unknown}', 3),
 ('simpa using {function}.linearIndependent', 3),
 ('rw [← finrank_eq_rank] at {unknown}', 3),
 ('apply LinearMap.ker_le_ker_comp', 3),
 ('letI : IsNoetherian {type} {type} := IsNoetherian.iff_fg.2 inferInstance',
  3),
 ('intro y _', 3),
 ('have := Module.nontrivial {type} {type}', 3),
 ('rw [LinearMap.det_def]', 3),
 ('exact add_mem {hypothesis} {hypothesis}', 3),
 ('rw [smul_smul]', 3),
 ('rcases eq_or_ne {variable} 0 with rfl | hy', 3),
 ('exact AddSubmonoid.zero_mem _', 3),
 ('rw [Finset.coe_singleton]', 3),
 ('exact ⟨{variable}, {hypothesis}, {variable}, {hypothesis}, rfl⟩', 3),
 ('· simp [{hypothesis}] at {hypothesis}', 3),
 ('ext w', 3),
 ('intro v', 3),
 ('obtain ⟨m, hm⟩ := {unknown}', 3),
 ('rw [{unknown}, map_zero]', 3),
 ('by_cases hy : {variable} = 0', 3),
 ('· exact False.elim ({hypothesis} {unknown})', 3),
 ('exact False.elim ({hypothesis} {unknown})', 3),
 ('rcases {hypothesis} with ⟨r, hr, rfl⟩', 3),
 ('rw [SameRay.sameRay_comm]', 3),
 ('simp_rw [eq_comm (a := {variable})]', 3),
 ('haveI : Inhabited {type} := ⟨0⟩', 3),
 ('intro s g hg i hi', 3),
 ('subst {hypothesis}', 3),
 ('by_cases ha : IsUnit {variable}', 3),
 ('intro ha', 3),
 ('· simp only [Matrix.vecCons, ne_eq, Nat.cast_ofNat, id_eq, eq_mpr_eq_cast, mul_eq_zero,\n    zero_lt_two, pow_eq_zero_iff]\n  left\n  rw [← Int.cast_zero (R := {type})]\n  exact congr_arg Int.cast rfl',
  3),
 ('simp only [Matrix.vecCons, ne_eq, Nat.cast_ofNat, id_eq, eq_mpr_eq_cast, mul_eq_zero,\n  zero_lt_two, pow_eq_zero_iff]',
  3),
 ('rw [← Int.cast_zero (R := {type})]', 3),
 ('by_cases hb : {variable} = 0', 3),
 ('simp only [direction_eq_vectorSpan, vectorSpan_def]', 3),
 ('rw [Set.mem_singleton_iff]', 3),
 ('rw [← mem_coe] at {hypothesis}', 3),
 ('simp only [mem_singleton_iff, exists_eq_left, mem_setOf_eq] at {hypothesis}',
  3),
 ('exact isCoprime_one_right', 3),
 ('by_contra hxy', 3),
 ('clear! {type}', 3),
 ('exact IsSeparable.separable {type} _', 3),
 ('haveI : Fact {unknown}.Prime := ⟨{unknown}⟩', 3),
 ("refine' sum_congr rfl fun i hi => _", 3),
 ('rw [mul_apply]', 3),
 ('· ring', 3),
 ('refine ⟨{function}, ?_, ?_⟩', 3),
 ('exact topEquiv.toLinearEquiv.finiteDimensional', 3),
 ('rcases eq_or_ne {unknown} 0 with (rfl | hp)', 3),
 ('by_contra hnz', 3),
 ('induction {unknown} using Localization.induction_on', 3),
 ('cases mul_eq_zero.mp {unknown} <;> contradiction', 3),
 ('simp_all [← add_assoc]', 3),
 ('simpa using {unknown}.le', 3),
 ('simpa only [mul_one] using mul_le_mul_of_nonpos_left {unknown} {unknown}',
  3),
 ('rw [if_pos]', 3),
 ('exact {unknown}.mul_right _', 3),
 ('simp_rw [injective_iff_map_eq_zero] at {hypothesis} ⊢', 3),
 ("induction' {unknown} using DirectLimit.induction_on with _ {function}", 3),
 ('rw [lift_of] at {unknown}', 3),
 ('rw [{hypothesis} _ {unknown} {unknown}, _root_.map_zero]', 3),
 ('· rwa [{hypothesis}] at {unknown}', 3),
 ("refine' Subring.InClosure.recOn {unknown} _ _ _ _", 3),
 ('exact sub_self _', 3),
 ('have := Fact.mk {unknown}', 3),
 ('simp only [← SetLike.mem_coe, coe_sInf, Set.mem_iInter₂]', 3),
 ('exact ⟨fun _ => mem_top, fun _ => {hypothesis} {variable}⟩', 3),
 ('exact {unknown}.dvd_of_dvd_pow {unknown}', 3),
 ("induction' {unknown} with {unknown} ih generalizing {variable}", 3),
 ('exact {unknown}.dvd', 3),
 ('exact zero_ne_one {unknown}.symm', 3),
 ('obtain rfl | ha := eq_or_ne {variable} 0', 3),
 ('exact dvd_gcd {unknown} {unknown}', 3),
 ("induction' {unknown} with {unknown} hk", 3),
 ('rw [{unknown}, zero_dvd_iff] at {unknown}', 3),
 ('rcases eq_or_ne {variable} 0 with (rfl | ha)', 3),
 ('rw [gcd_zero_left]', 3),
 ('norm_cast at {hypothesis}', 3),
 ('apply Set.Subset.antisymm', 3),
 ('simp only [indicator]', 3),
 ('rw [{unknown}.eq]', 3),
 ('rw [iterate_succ]', 3),
 ('rw [LE.le] at {unknown}', 3),
 ('exact subset_span', 3),
 ('obtain ⟨c, hc, rfl⟩ := {hypothesis}', 3),
 ('rw [mem_set_smul_def, Submodule.mem_sInf]', 3),
 ('rw [add_smul]', 3),
 ('rw [div_eq_mul_inv, div_eq_mul_inv]', 3),
 ('exact add_right_inj _', 3),
 ("refine Finsupp.ext fun g' => ?_", 3),
 ('obtain ⟨d, rfl⟩ | h := em (∃ d, {variable} = {variable} + d)', 3),
 ('obtain ⟨c, {hypothesis}⟩ := {hypothesis}', 3),
 ("refine' ⟨fun h x => _, fun h x => _⟩", 3),
 ('rw [Algebra.algebraMap_eq_smul_one]', 3),
 ("refine' ⟨1, _⟩", 3),
 ('rw [← Algebra.smul_def]', 3),
 ('apply mul_le_mul', 3),
 ('letI := NoZeroDivisors.to_isDomain {type}', 3),
 ('omega', 3),
 ('simpa [{hypothesis}] using {unknown}', 3),
 ('simp_rw [indicator_apply]', 3),
 ('rw [← abs_mul_abs_self, ← abs_mul_abs_self {variable}]', 3),
 ('obtain ha | ha := le_total {variable} 0', 3),
 ('rw [floor_of_nonpos {unknown}]', 3),
 ('obtain ⟨d, rfl⟩ := exists_add_of_le {unknown}', 3),
 ('rwa [smul_zero]', 3),
 ('rwa [zero_smul]', 3),
 ('exact pow_nonneg {unknown} _', 3),
 ('rw [div_eq_mul_one_div {variable} {variable}, div_eq_mul_one_div {variable} {variable}]',
  3),
 ('rwa [mul_one]', 3),
 ('exact mul_self_nonneg _', 3),
 ('exact MeasurableSet.iUnion {hypothesis}', 3),
 ('· simp [{hypothesis} {variable} {hypothesis}]', 3),
 ('rw [dist_comm]', 3),
 ('rw [{hypothesis}] at {hypothesis}', 3),
 ('obtain ⟨o, xo, o_open, μo⟩ : ∃ o : Set {type}, {variable} ∈ o ∧ IsOpen o ∧ {unknown} o < ∞ :=\n  Measure.exists_isOpen_measure_lt_top {unknown} {variable}',
  3),
 ("refine' (ENNReal.div_le_iff_le_mul _ (Or.inr (bot_le.trans_lt {unknown}).ne')).1 {unknown}.le",
  3),
 ('exact ENNReal.mul_le_of_le_div {unknown}.le', 3),
 ('simp only [{hypothesis}, tsum_zero]', 3),
 ('rw [measure_toMeasurable]', 3),
 ('simp only [not_or, ENNReal.add_eq_top, Pi.add_apply, Ne.def, coe_add] at {unknown}',
  3),
 ('apply inter_subset_left', 3),
 ('rw [ENNReal.tendsto_coe]', 3),
 ('simp only [lintegral_const, MeasurableSet.univ, Measure.restrict_apply, univ_inter]',
  3),
 ("filter_upwards [{hypothesis}, {hypothesis}] with x hx h'x", 3),
 ("apply {function}.congr' _", 3),
 ('exact measurableSet_toMeasurable _ _', 3),
 ('convert {function} using 1', 3),
 ('rw [rnDeriv_eq_zero]', 3),
 ('rw [← withDensity_eq_iff]', 3),
 ('simp_rw [iSup_apply]', 3),
 ('· exact ne_of_lt (measure_lt_top _ _)', 3),
 ('rw [← withDensity_eq_iff_of_sigmaFinite]', 3),
 ("let f' := {unknown}.1.mk {function}", 3),
 ('rw [Finset.mem_filter] at {function}', 3),
 ('rcases lt_or_le {unknown} (last {unknown}) with (H | H)', 3),
 ('intro b _', 3),
 ('rw [disjoint_comm]', 3),
 ('intro {unknown} _', 3),
 ('rcases mem_range.1 {hypothesis} with ⟨a, rfl⟩', 3),
 ('rw [lintegral]', 3),
 ('apply nhdsWithin_le_nhds', 3),
 ('apply Tendsto.mono_left _ nhdsWithin_le_nhds', 3),
 ('exact image_subset {function} (subset_inter Subset.rfl {unknown})', 3),
 ('exact pairwise_disjoint_mono {unknown} fun n => inter_subset_right _ _', 3),
 ('exact Subset.antisymm (subset_inter Subset.rfl {unknown}) (inter_subset_left _ _)',
  3),
 ('exact measurable_spanningSets {unknown} {unknown}', 3),
 ('rw [lintegral_iUnion]', 3),
 ('simp_rw [smul_eq_mul]', 3),
 ('simp only [det_one_smulRight]', 3),
 ('rcases IsCompact.exists_bound_of_continuousOn {unknown} {unknown} with ⟨C, hC⟩',
  3),
 ("induction' {function} using MeasureTheory.SimpleFunc.induction with c s hs f₁ f₂ _ h₁ h₂ generalizing {unknown}",
  3),
 ('simp only [SimpleFunc.coe_const, SimpleFunc.const_zero, SimpleFunc.coe_zero,\n  Set.piecewise_eq_indicator, SimpleFunc.coe_piecewise]',
  3),
 ('simp_rw [mul_add]', 3),
 ('rw [integral_eq_lintegral_of_nonneg_ae, integral_eq_lintegral_of_nonneg_ae]',
  3),
 ('refine {hypothesis} {variable} ?_', 3),
 ('rw [← lt_top_iff_ne_top]', 3),
 ('exact two_ne_zero', 3),
 ('simp [lt_of_lt_of_le zero_lt_one {unknown}]', 3),
 ('exact ENNReal.toReal_pos {unknown} {unknown}', 3),
 ("simp [snorm', {unknown}]", 3),
 ('by_cases hμ : {unknown} = 0', 3),
 ('simp_rw [← ofReal_norm_eq_coe_nnnorm]', 3),
 ('rw [snorm_smul_measure_of_ne_zero {hypothesis}, smul_eq_mul]', 3),
 ('· simp only [{hypothesis}, snorm_exponent_zero]', 3),
 ('simp_rw [snorm_eq_lintegral_rpow_nnnorm {hypothesis} {hypothesis}]', 3),
 ('simp only [{unknown}, snorm_exponent_zero]', 3),
 ("refine' lintegral_mono_ae _", 3),
 ('by_cases hq_top : {unknown} = ∞', 3),
 ('simp [measure_univ]', 3),
 ('· simp only [{unknown}, one_div, Ne.def, ENNReal.inv_eq_top, not_false_iff]',
  3),
 ('simp only [{unknown}, one_div, Ne.def, ENNReal.inv_eq_top, not_false_iff]',
  3),
 ('exact norm_nonneg _', 3),
 ('rw [← memℒp_one_iff_integrable] at {unknown} {unknown} ⊢', 3),
 ('apply {hypothesis}.mono', 3),
 ('rw [snorm_congr_ae (Lp.coeFn_sub _ _).symm]', 3),
 ('exact Lp.snorm_ne_top _', 3),
 ('specialize {hypothesis} {unknown} {unknown} {unknown} {unknown} {unknown}',
  3),
 ('simp_rw [sub_eq_add_neg]', 3),
 ('obtain rfl | hμ := eq_or_ne {unknown} 0', 3),
 ('rw [← measure_diff_null {unknown}] at {unknown}', 3),
 ("obtain ⟨{unknown}, hx, hxN⟩ := nonempty_of_measure_ne_zero {unknown}.ne'",
  3),
 ('have ah :\n    Pairwise fun {unknown} {unknown} => {unknown}.r {unknown} ≤ ‖{unknown}.c {unknown} - {unknown}.c {unknown}‖ ∧ {unknown}.r {unknown} ≤ {variable} * {unknown}.r {unknown} ∨\n      {unknown}.r {unknown} ≤ ‖{unknown}.c {unknown} - {unknown}.c {unknown}‖ ∧ {unknown}.r {unknown} ≤ {variable} * {unknown}.r {unknown} := by\n  simpa only [dist_eq_norm] using {unknown}.h',
  3),
 ('have δnonneg : 0 ≤ {variable} := by linarith only [{unknown}, {unknown}]',
  3),
 ('simpa only [dist_eq_norm] using {unknown}.h', 3),
 ('apply add_le_add_left', 3),
 ('intro f hf', 3),
 ('intro z _', 3),
 ('rw [lintegral_const]', 3),
 ('simp_rw [← iSup_apply]', 3),
 ('· measurability', 3),
 ('rw [← iSup_lintegral_measurable_le_eq_lintegral, ← iSup_lintegral_measurable_le_eq_lintegral]',
  3),
 ('congr 1 with a : 1', 3),
 ("let f' := {unknown}.mk {function}", 3),
 ('rw [intervalIntegrable_iff]', 3),
 ('simp only [integral]', 3),
 ('rw [integral_undef {hypothesis}]', 3),
 ('rw [L1.norm_def]', 3),
 ('by_cases hfi : Integrable {function} {unknown}', 3),
 ('rw [integral_undef {hypothesis}, integral_undef]', 3),
 ("refine' Finset.sum_congr rfl fun x hx => _", 3),
 ('exact {unknown} {variable}', 3),
 ('rw [setToL1_eq_setToL1SCLM]', 3),
 ('rw [setToFun_eq {unknown} {unknown}]', 3),
 ('by_cases hr : {variable} = 0', 3),
 ('push_neg at {hypothesis} {hypothesis}', 3),
 ('rw [IntegrableOn, Measure.restrict_restrict {unknown}]', 3),
 ('have hfg_int : ∀ {set}, MeasurableSet {set} → {unknown} {set} < ∞ → IntegrableOn ({function} - {function}) {set} {unknown} := fun {set} hs hμs =>\n  ({hypothesis} {set} hs hμs).sub ({hypothesis} {set} hs hμs)',
  3),
 ('exact ae_of_ae_restrict_of_ae_restrict_compl _ {unknown} {unknown}', 3),
 ('exact eventually_of_forall {hypothesis}', 3),
 ('· exact {hypothesis} {unknown} {unknown}', 3),
 ('exact if_pos {unknown}', 3),
 ('· exact {unknown}.inter (MeasurableSet.disjointed {hypothesis} {unknown})',
  3),
 ('· exact Or.inl ⟨{hypothesis}, {hypothesis}⟩', 3),
 ('exact Or.inl ⟨{hypothesis}, {hypothesis}⟩', 3),
 ('intro xt', 3),
 ("rwa [EventuallyLE, ae_restrict_iff' {unknown}]", 3),
 ("refine' (Memℒ{unknown}.coeFn_toLp ((Lp.memℒ{unknown} {unknown}).restrict {set})).mp _",
  3),
 ('simp [{hypothesis} {variable} {variable} {hypothesis} {hypothesis}]', 3),
 ('specialize {hypothesis} {variable} {hypothesis} {variable} {hypothesis}',
  3),
 ('rw [extend_eq]', 3),
 ('by_cases hab : {variable} < {variable}', 3),
 ('rw [borel_eq_generateFrom_Ioi]', 3),
 ('rw [tendsto_atTop_atTop]', 3),
 ('refine eventually_of_forall fun n => ?_', 3),
 ('apply mem_iUnion_of_mem {unknown}', 3),
 ('rcases {unknown}.exists_seq_tendsto with ⟨v, hv⟩', 3),
 ('borelize ({type} × {type})', 3),
 ('rw [{hypothesis} {variable} {hypothesis} {unknown}]', 3),
 ('simpa using {hypothesis} {unknown}', 3),
 ("rw [ae_restrict_iff' {unknown}]", 3),
 ('simp only [Classical.not_not, mem_setOf_eq, mem_compl_iff] at {hypothesis}',
  3),
 ('rw [← measure_iUnion_eq_iSup, {unknown}]', 3),
 ('rw [measurable_pi_iff]', 3),
 ("induction' {unknown} with i {unknown} ih", 3),
 ('rw [setOf_forall]', 3),
 ("refine' Measure.ext fun s hs => _", 3),
 ('simp only [Pi.add_apply]', 3),
 ('simp only [{unknown}, Pi.mul_apply]', 3),
 ('· abel', 3),
 ('simp only [inter_comm]', 3),
 ('rw [← {unknown}.image_uIcc]', 3),
 ('apply MeasurableSet.iUnion', 3),
 ('apply measurableSet_generateFrom', 3),
 ('exact measurableSet_generateFrom {hypothesis}', 3),
 ('rw [← pi_univ_Icc]', 3),
 ('intro h2f', 3),
 ("refine' prod_eq fun s t hs ht => _", 3),
 ('simp [map_of_not_aemeasurable, {hypothesis}, {hypothesis}]', 3),
 ('filter_upwards [{unknown}.prod_right_ae, {unknown}.prod_right_ae] with _ h2f h2g',
  3),
 ('apply LinearMap.continuous_on_pi', 3),
 ("apply tendsto_const_nhds.congr' _", 3),
 ('apply tendsto_nhdsWithin_of_tendsto_nhds_of_eventually_within', 3),
 ('exact analyticSet_range_of_polishSpace {unknown}', 3),
 ('refine mem_iInter.2 fun n => ?_', 3),
 ('letI := upgradeStandardBorel {type}', 3),
 ('apply Metric.mem_nhds_iff.1', 3),
 ('exact {unknown}.continuousAt.preimage_mem_nhds ({unknown}.mem_nhds {hypothesis})',
  3),
 ('obtain ⟨m, rfl⟩ := exists_add_of_le {unknown}', 3),
 ('simp [Nat.succ_le_succ_iff] at {unknown}', 3),
 ('intro {hypothesis} hx', 3),
 ('rintro _ ⟨a, rfl⟩', 3),
 ('erw [subtypePerm_apply]', 3),
 ('obtain ⟨m, hm⟩ := exists_pow_eq_self_of_coprime {unknown}', 3),
 ("cases' {unknown} with y {unknown}", 3),
 ('rw [cycleFactorsFinset_eq_finset]', 3),
 ('obtain rfl | hb := eq_or_ne {variable} 0', 3),
 ('simp_rw [Equiv.Perm.coe_pow]', 3),
 ('exact {unknown}.iterate', 3),
 ('rw [mem_support] at {hypothesis}', 3),
 ('refine ltByCases {variable} {variable} (fun h => ?_) (fun h => ?_) (fun h => ?_)',
  3),
 ('rintro ⟨{variable}, ha, rfl⟩', 3),
 ('rwa [{unknown}.inj ({hypothesis} {unknown})]', 3),
 ('rw [sup_comm] at {unknown}', 3),
 ('intro _ _ h₁ h₂', 3),
 ('rintro ⟨i, rfl⟩', 3),
 ("refine' Fin.lastCases _ (fun {unknown} => _) {unknown}", 3),
 ('rintro ⟨b, hb⟩', 3),
 ('intro {variable} ha', 3),
 ('exact {hypothesis} {hypothesis} {hypothesis}', 3),
 ('rw [← compl_inj_iff]', 3),
 ('simp [Nat.lt_succ_iff]', 3),
 ('rw [sup_comm]', 3),
 ('exact sSup_le_sSup {unknown}', 3),
 ('{ exact ⟨{variable}, {variable}, {variable}, {unknown}, {unknown}, Or.inr ⟨{unknown}, {unknown}⟩⟩ }',
  3),
 ('exact ⟨{variable}, {variable}, {variable}, {unknown}, {unknown}, Or.inr ⟨{unknown}, {unknown}⟩⟩',
  3),
 ('rw [← {unknown}.symm_apply_apply {variable}] at {unknown}', 3),
 ('rintro - ⟨x, rfl⟩', 3),
 ('convert le_ciSup {function} {unknown}', 3),
 ('· intro _ _\n  exact le_top', 3),
 ('exact some_le_some.1 ({hypothesis} {function})', 3),
 ('obtain ⟨{set}, ht, {set}, hs, hst⟩ := eventually_prod_iff.1 {hypothesis}',
  3),
 ('rw [← prod_atBot_atBot_eq]', 3),
 ('rw [← prod_atTop_atTop_eq]', 3),
 ('beta_reduce at *', 3),
 ('have x₀_in : {variable} ∈ ball {variable} {variable} := mem_ball_self {unknown}',
  3),
 ('filter_upwards [{hypothesis}] with n hn', 3),
 ('exact tendsto_pow_atTop_nhds_zero_of_lt_one one_half_pos.le one_half_lt_one',
  3),
 ('simpa only [div_eq_mul_inv] using {unknown}.mul_const {variable}⁻¹', 3),
 ('rwa [add_apply, comp_apply, flip_apply, smulRight_apply, smulRight_apply, one_apply, one_smul,\n  one_smul, add_comm] at {unknown}',
  3),
 ('rcases eq_or_ne {variable} 0 with (rfl | hr)', 3),
 ('simp_rw [{unknown}, {function}.map_zero₂]', 3),
 ('intro p {variable} hp hx', 3),
 ('simpa only [← smul_eq_mul, ← NNReal.smul_def, smul_one_smul ℝ≥0 {variable} (_ : ℝ)] using\n  mul_max_of_nonneg {variable} {variable} ({variable} • (1 : ℝ≥0) : ℝ≥0).coe_nonneg',
  3),
 ('rintro _ ⟨{variable}, rfl⟩', 3),
 ('rw [← ball_normSeminorm {type}]', 3),
 ('let A : NormedSpace ℝ {type} := RestrictScalars.normedSpace ℝ {type} {type}',
  3),
 ('exact exists_deriv_eq_slope {function} {unknown} ({unknown}.mono {unknown}) ({unknown}.mono {unknown})',
  3),
 ('obtain ⟨a, ⟨hxa, hay⟩, ha⟩ : ∃ a ∈ Ioo {variable} {variable}, deriv {function} a = ({function} {variable} - {function} {variable}) / ({variable} - {variable})',
  3),
 ('simpa only [mem_preimage, apply_symm_apply] using {hypothesis}', 3),
 ('simp [(· ∘ ·)]', 3),
 ('simp only [*]', 3),
 ('rw [iteratedFDeriv_zero_eq_comp]', 3),
 ("convert {unknown}.mul' {unknown}", 3),
 ("convert {unknown}.mul_const' {variable}", 3),
 ('rw [norm_smul, norm_inv]', 3),
 ('rw [map_zero] at {unknown}', 3),
 ('intro V hV', 3),
 ('abel_nf', 3),
 ('rw [norm_smul]', 3),
 ('refine MeasurableSet.iInter (fun _ ↦ ?_)', 3),
 ('obtain ⟨i, {hypothesis}⟩ := {hypothesis}', 3),
 ('obtain rfl | hxy := eq_or_ne {variable} {variable}', 3),
 ('rw [mem_preimage, {function}.map_add, {function}.map_smul, {function}.map_smul]',
  3),
 ('calc\n  {variable} = {variable} • {variable} + {variable} • {variable} := (Convex.combo_self {unknown} _).symm\n  _ ≤ {variable} • {variable} + {variable} • {variable} := by gcongr',
  3),
 ('calc\n  {variable} • {variable} + {variable} • {variable} ≤ {variable} • {variable} + {variable} • {variable} := by gcongr\n  _ = {variable} := Convex.combo_self {unknown} _',
  3),
 ('rintro _ ⟨a, b, ha, hb, hab, rfl⟩', 3),
 ('rw [contDiff_top_iff_fderiv]', 3),
 ('rcases {unknown} {unknown} {unknown} with ⟨{set}, hu, {function}, hp⟩', 3),
 ("induction' {unknown} using Finset.induction_on with i {unknown} is IH", 3),
 ('rw [← inner_conj_symm, conj_re]', 3),
 ('rw [← inner_conj_symm]', 3),
 ("refine' Finset.sum_congr rfl fun i hi => _", 3),
 ('tfae_have 3 ↔ 4', 3),
 ('simp only [norm_nonneg, pow_nonneg]', 3),
 ('simp only [mul_zero]', 3),
 ('simp only [sq]', 3),
 ('letI : Module ℝ {type} := RestrictScalars.module ℝ {type} {type}', 3),
 ('have h₁ := {hypothesis} {function} {hypothesis}', 3),
 ('exact le_of_eq {unknown}', 3),
 ('· exact ⟨{variable}, {variable}, {unknown}, {unknown}, {unknown}, rfl⟩', 3),
 ('exact ⟨{variable}, {variable}, {unknown}, {unknown}, {unknown}, rfl⟩', 3),
 ('rw [segment_eq_image₂, segment_eq_image₂, image_image]', 3),
 ('simp [Convex.combo_self {hypothesis}.2.2]', 3),
 ('haveI : FiniteDimensional ℝ {type} := .of_fact_finrank_eq_succ {unknown}',
  3),
 ('rw [vsub_eq_zero_iff_eq] at {unknown}', 3),
 ('rw [wbtw_comm] at *', 3),
 ('simp only [IsBigO_def]', 3),
 ('simpa only [sub_eq_add_neg] using {unknown}.add {unknown}.neg_left', 3),
 ('have : TopologicalAddGroup {type} := {unknown}.topologicalAddGroup', 3),
 ('rw [div_lt_iff {unknown}]', 3),
 ('rw [lt_div_iff {unknown}]', 3),
 ('field_simp at {unknown} ⊢', 3),
 ('rcases eq_or_ne {variable} 0 with (rfl | hp)', 3),
 ('simp [dist_eq_norm]', 3),
 ('contrapose {hypothesis}', 3),
 ('ext1 m', 3),
 ('rcases {unknown}.exists_eq_smul with ⟨u, a, b, ha, hb, -, rfl, rfl⟩', 3),
 ('intro {unknown} hJ', 3),
 ('exact min_le_right _ _', 3),
 ('rw [{unknown}.1, {unknown}.2, div_mul_comm, div_mul_comm {variable}]', 3),
 ('exact {unknown}.le_bot ⟨{unknown}, {unknown}⟩', 3),
 ('rw [← {hypothesis}] at {unknown}', 3),
 ("apply mul_le_mul_right'", 3),
 ('exact if_neg ENNReal.top_ne_zero', 3),
 ("refine' ⟨{unknown}, _, {unknown}⟩", 3),
 ('rw [lt_iff_not_ge] at {unknown}', 3),
 ('by_cases hx₁ : -1 ≤ {variable}', 3),
 ('by_cases hx₂ : {variable} ≤ 1', 3),
 ('simp_rw [Real.rpow_def_of_pos (by positivity : 0 < {variable})]', 3),
 ('· lift {unknown} to ℝ using {unknown}\n  simpa using {unknown}.ne', 3),
 ('exact log_nonneg {unknown}', 3),
 ('rintro {variable} (hx : {variable} < 0) {variable} (hy : {variable} < 0) hxy',
  3),
 ('rwa [abs_of_neg {unknown}, abs_of_neg {unknown}, neg_lt_neg_iff]', 3),
 ("refine' mem_emetric_ball_zero_iff.2 (lt_of_le_of_lt _ {unknown})", 3),
 ('exact mod_cast nnnorm_add_le {variable} {variable}', 3),
 ("refine' HasStrictFDerivAt.congr_of_eventuallyEq _ {function}.symm", 3),
 ('lift {unknown} to ℝ≥0 using ne_of_lt (lt_of_lt_of_le {unknown} le_top)', 3),
 ('lift {unknown} to ℝ≥0 using ne_of_lt (lt_of_le_of_lt {unknown} coe_lt_top)',
  3),
 ('convert Iff.rfl', 3),
 ('intro θ', 3),
 ('rw [two_nsmul_eq_iff] at {unknown}', 3),
 ('exact {unknown}.zero_mem', 3),
 ('rw [exp_le_exp]', 3),
 ('rwa [{unknown}.out, rpow_nat_cast]', 3),
 ('rwa [{unknown}.out, Real.rpow_int_cast]', 3),
 ('rw [cpow_def_of_ne_zero {unknown}]', 3),
 ('exact Submodule.fg_bot', 3),
 ('apply Ideal.subset_span', 3),
 ('apply AddSubmonoid.mem_sup_left', 3),
 ('exact {unknown}.neg_mem {hypothesis}', 3),
 ('exact subset_span {hypothesis}', 3),
 ('obtain ⟨S, hS⟩ := {unknown}', 3),
 ('letI : Module.Finite {type} {type} := {unknown}', 3),
 ('by_cases h₀ : {unknown} = 0', 3),
 ('· intro h\n  rw [← abs_mul_cos_add_sin_mul_I {unknown}, h]\n  simp [{hypothesis}]',
  3),
 ('rcases lt_trichotomy {unknown}.im 0 with (hi | hi | hi)', 3),
 ('rw [(ext rfl {unknown} : {unknown} = {unknown}.re)]', 3),
 ('rcases lt_trichotomy {unknown}.re 0 with (hr | hr | hr)', 3),
 ('ext (i j)', 3),
 ('haveI : FiniteDimensional {type} {type} := FiniteDimensional.right {type} {type} {type}',
  3),
 ('haveI : IsSeparable {type} {type} := isSeparable_tower_top_of_isSeparable {type} {type} {type}',
  3),
 ('letI : Algebra {type} {type} := {function}.toRingHom.toAlgebra', 3),
 ('apply {unknown}.not_unit', 3),
 ('rw [sdiff_sdiff_comm, sdiff_singleton_eq_erase {variable}, erase_insert {hypothesis}]',
  3),
 ('simp only [mem_iInf] at {hypothesis} {unknown}', 3),
 ('intro j hj ij', 3),
 ('rw [associated_iff_eq] at {unknown}', 3),
 ('letI := Classical.decEq (Associates {type})', 3),
 ('rw [_root_.pow_zero]', 3),
 ('simp [{unknown}, pow_add, mul_comm, mul_assoc, mul_left_comm]', 3),
 ('simp only [coeff_eq_zero_of_natDegree_lt {unknown}, zero_mul, mul_zero]',
  3),
 ('exact not_isCoprime_zero_zero {unknown}', 3),
 ('exact {unknown}.of_add_mul_left_left', 3),
 ('rw [map_map]', 3),
 ('rw [as_sum {unknown}]', 3),
 ('obtain ⟨{unknown}, rfl⟩ := Ideal.Quotient.mk_surjective {unknown}', 3),
 ('obtain ⟨k, hk⟩ := exists_add_of_le {unknown}', 3),
 ('exact iSup_le fun i {variable} hx ↦ Set.mem_iUnion.2 ⟨i, hx⟩', 3),
 ("simp only [sSup_eq_iSup', mem_iSup_of_directed {function}.directed_val, SetCoe.exists, Subtype.coe_mk,\n  exists_prop]",
  3),
 ('apply {unknown} {unknown}', 3),
 ('exact {unknown}.pow_of_coprime {unknown} {unknown}', 3),
 ('rw [mem_primitiveRoots {unknown}] at {hypothesis}', 3),
 ('exact Monic.ne_zero {unknown} (degree_eq_bot.1 {unknown})', 3),
 ('rw [← map_cyclotomic_int]', 3),
 ('by_cases hg : {variable} ∈ Set.range {unknown}', 3),
 ('rw [Multiset.prod_add]', 3),
 ('apply Or.intro_right', 3),
 ('let _ : Algebra {type} {type} := {function}.toAlgebra', 3),
 ('exact subset_adjoin', 3),
 ('· rcases {hypothesis} with ⟨P, rfl⟩\n  rcases {hypothesis} with ⟨Q, rfl⟩\n  exact ⟨P + Q, AlgHom.map_add _ _ _⟩',
  3),
 ('apply Ideal.ext', 3),
 ("rcases {unknown} with ⟨p', q', hpq'⟩", 3),
 ('rw [aeval_mul, LinearMap.mul_apply, LinearMap.mem_ker.1 {hypothesis}, LinearMap.map_zero]',
  3),
 ('use {variable} * {variable}', 3),
 ('by_cases hI : {unknown} = ⊥', 3),
 ('rw [lt_iff_le_not_le]', 3),
 ("apply Nat.add_lt_of_lt_sub'", 3),
 ('split <;> rfl', 3),
 ('split <;> rename_i h', 3),
 ('rwa [Nat.mul_comm]', 3),
 ('by_cases h : {unknown} {variable} <;> simp [h]', 3),
 ('simp only [countP_eq_length_filter]', 3),
 ('specialize {unknown} {hypothesis}.2 {unknown} {hypothesis}', 3),
 ('rcases {hypothesis} with ({hypothesis} | {hypothesis} | ⟨k, hkt, {hypothesis}⟩)',
  3),
 ("have hp' : ∀ j ∈ {unknown}, IsPrime ({function} j) := by\n  intro j hj\n  refine' {hypothesis} j (Finset.mem_insert_of_mem hj) _ _ <;> rintro rfl <;>\n    solve_by_elim only [Finset.mem_insert_of_mem, *]",
  3),
 ("rw [Finset.coe_insert, Set.biUnion_insert, ← Set.union_self ({function} {variable} : Set {type}),\n  subset_union_prime' {hypothesis}, ← or_assoc, or_self_iff] at {hypothesis}",
  3),
 ('rwa [Finset.exists_mem_insert]', 3),
 ("refine' {hypothesis} {variable} (Finset.mem_insert_of_mem {hypothesis}) _ _ <;> rintro rfl <;>\n  solve_by_elim only [Finset.mem_insert_of_mem, *]",
  3),
 ('exact fun _ => zero_smul _ _', 3),
 ('simp [{unknown}, {unknown}, Buckets.size_eq]', 3),
 ('funext l', 3),
 ('exact ⟨{unknown}, {unknown}.trans {unknown}, {unknown}⟩', 3),
 ('apply Exists.intro ({unknown}+1)', 3),
 ('unfold bitwise', 3),
 ('unfold zoom', 3),
 ('simp [SatisfiesM_Id_eq] at {unknown}', 3),
 ('simp [Nat.add_right_cancel_iff, {hypothesis}]', 3),
 ('case hnc => simp [← {unknown}, Pos.ext_iff]; exact ne_self_add_add_csize',
  3),
 ('simpa [Nat.add_assoc, Nat.add_comm] using {unknown}', 3),
 ('simp [← {unknown}, Pos.ext_iff]', 3),
 ('exact ne_self_add_add_csize', 3),
 ('simp [extract]', 3),
 ('rw [size_extract_loop]', 3),
 ('apply aux', 3),
 ('dsimp [bmod]', 3),
 ('rw [{unknown}, cons_append]', 3),
 ('induction {unknown} generalizing {unknown} <;> cases {unknown} <;> simp_all',
  3),
 ('· cases {unknown}\n  · simp\n  · simp [{hypothesis}]', 3),
 ('simp [← mem_toList, {unknown}]', 3),
 ('apply Nat.gcd_dvd_left', 3),
 ('rw [add, {unknown}]', 3),
 ('intro sqr_eq', 3),
 ('apply pow_two_nonneg', 3),
 ('use max {unknown} {unknown}', 3),
 ('intro n ngt', 3),
 ('apply abs_nonneg', 3),
 ('apply neg_eq_of_add_eq_zero', 3),
 ('rw [mul_comm {variable} {variable}]', 3),
 ("convert Nat.dvd_sub' {unknown} {unknown}", 3),
 ('exact ⟨⟨{variable}, {hypothesis}, rfl⟩, {hypothesis}⟩', 3),
 ('dsimp at *', 3),
 ('unfold Name.quickCmp', 2),
 ('exact False.elim {unknown}', 2),
 ('cases {unknown} <;> cases {unknown} <;> cases {unknown} <;> decide', 2),
 ('apply heq_of_eq', 2),
 ('apply div_rec_lemma', 2),
 ('rw [mod_eq_of_lt {unknown}]', 2),
 ('simp! [append_denote]', 2),
 ('have {unknown} := congrArg (Poly.denote {unknown}) (eq_of_beq {unknown})',
  2),
 ('simp_wf', 2),
 ('apply Nat.lt_trans {unknown}', 2),
 ('apply Nat.zero_le', 2),
 ('simp [mergeIdem, mergeIdem.loop]', 2),
 ('. simp [evalList, {unknown}.1, EvalInformation.evalOp]', 2),
 ('simp [evalList, {unknown}.1, EvalInformation.evalOp]', 2),
 ('simp [sort.loop, evalList_insert {unknown} {unknown}]', 2),
 ('apply evalList_sort_congr {unknown} {unknown}', 2),
 ('all_goals simp [insert_nonEmpty]', 2),
 ('simp [Expr.toList]', 2),
 ('split at {unknown} <;> simp_all', 2),
 ('simp [get]', 2),
 ('simp_arith [{unknown}]', 2),
 ('have aux := congrArg length {unknown}', 2),
 ('injection {unknown} with h₁ h₂', 2),
 ('· exact False.elim <| {unknown} (List.lt.head _ _ {hypothesis})', 2),
 ('exact False.elim <| {unknown} (List.lt.head _ _ {unknown})', 2),
 ('have {unknown} : {unknown} ≤ {unknown} := fun h => {unknown} (List.lt.tail {hypothesis} {hypothesis} h)',
  2),
 ('apply Nat.le_of_lt', 2),
 ('apply Nat.lt_succ_self', 2),
 ('have {unknown} := Nat.eq_add_of_sub_eq (Nat.le_trans {unknown} (Nat.le_add_left ..)) {unknown}',
  2),
 ('exact utf8PrevAux_lt_of_pos _ _ _ {unknown}', 2),
 ('exact Nat.lt_of_lt_of_le {unknown} (Nat.le_add_right ..)', 2),
 ('rw [Nat.succ_add] at {unknown}', 2),
 ('simp [mapTR.loop, map]', 2),
 ('simp [filterTR.loop, filter]', 2),
 ('apply Mem.head', 2),
 ('apply Mem.tail', 2),
 ('simp [← {unknown}, {unknown}]', 2),
 ('simp [eq_of_beq {hypothesis}]', 2),
 ('by_cases hltv : Nat.{unknown} {unknown} {unknown} <;> simp [hltv]', 2),
 ('try assumption', 2),
 ('by_cases hgtv : Nat.{unknown} {unknown} {unknown} <;> simp [hgtv]', 2),
 ('by_cases hltk : Nat.{unknown} {unknown} {unknown} <;> simp [hltk]', 2),
 ('apply Nat.sub_eq_of_eq_add', 2),
 ('by_cases hgtk : Nat.{unknown} {unknown} {unknown} <;> simp [hgtk]', 2),
 ('rw [← Nat.add_assoc, ← Nat.add_assoc] at {unknown}', 2),
 ('by_cases hltv : Nat.{unknown} {unknown} {unknown} <;> simp [hltv] at {unknown}',
  2),
 ('by_cases hgtv : Nat.{unknown} {unknown} {unknown} <;> simp [hgtv] at {unknown}',
  2),
 ('by_cases hltk : Nat.{unknown} {unknown} {unknown} <;> simp [hltk] at {unknown}',
  2),
 ('by_cases hgtk : Nat.blt {unknown} {unknown} <;> simp [hgtk] at {unknown}',
  2),
 ('apply Nat.le_sub_of_add_le', 2),
 ('simp [toPoly, {unknown}, -Poly.mul]', 2),
 ('by_cases h : {unknown} = true <;> simp [h]', 2),
 ('rw [Poly.denote_eq_cancel_eq]', 2),
 ('simp [eq_of_beq {unknown}, Var.denote]', 2),
 ('exact Nat.le_trans {unknown} (Nat.le_add_right ..)', 2),
 ('by_cases he : {function} = true <;> simp [he, {function}, Poly.denote_eq, Poly.denote_le]',
  2),
 ('simp [toExpr, Expr.denote, Poly.denote]', 2),
 ('rw [List.length_cons] at {unknown}', 2),
 ("apply ext'", 2),
 ('simp [funext {hypothesis}]', 2),
 ('exact {unknown}.left_of_comp {unknown}', 2),
 ('change _ = _', 2),
 ('rw [minimalPeriod_prod_map]', 2),
 ("split_ifs with h h'", 2),
 ('· exact False.elim (lt_asymm {hypothesis} {hypothesis})', 2),
 ('exact False.elim (lt_asymm {unknown} {unknown})', 2),
 ('dsimp [ofSet]', 2),
 ('rw [Part.toOption_eq_none_iff.2 {hypothesis}]', 2),
 ('· simp at {function}', 2),
 ('have {unknown} := {unknown} _ {hypothesis}', 2),
 ('apply Function.hfunext', 2),
 ('· simp only [{hypothesis}, not_false_iff, exists_prop_of_false]', 2),
 ('simp only [{hypothesis}, not_false_iff, exists_prop_of_false]', 2),
 ('apply {hypothesis} _ {hypothesis}', 2),
 ('simp [mod_def]', 2),
 ('simp [append_def]', 2),
 ('simp [inter_def]', 2),
 ('simp [union_def]', 2),
 ('simp [sdiff_def]', 2),
 ('exact {unknown}.iterate {unknown}', 2),
 ('simpa only [{unknown}.comp_eq] using bijOn_fixedPoints_comp {function} {function}',
  2),
 ('exact Rat.num_nonneg_iff_zero_le.2 {unknown}.2', 2),
 ("cases' e : {unknown} /. {unknown} with n d h c", 2),
 ('rw [Int.ediv_mul_cancel {unknown}]', 2),
 ('apply Nat.div_dvd_of_dvd', 2),
 ('exact ({unknown}.reduced.mul_right {unknown}.reduced).mul ({unknown}.reduced.mul_right {unknown}.reduced)',
  2),
 ('have hs : ({unknown}.den * {unknown}.den : ℤ) ≠ 0 := Int.coe_nat_ne_zero_iff_pos.mpr (mul_pos {unknown}.pos {unknown}.pos)',
  2),
 ('rw [{unknown}, mul_assoc, mul_comm]', 2),
 ("repeat' rw [Int.mul_assoc]", 2),
 ('apply mul_eq_mul_left_iff.2', 2),
 ('rw [num_den, num_den] at {unknown}', 2),
 ('apply Rat.eq_iff_mul_eq_mul.mp', 2),
 ('rw [← divInt_eq_div]', 2),
 ('rw [Int.mul_assoc]', 2),
 ('rintro ⟨d, rfl⟩', 2),
 ("rw [← Int.natAbs_of_nonneg {unknown}.le, ← Rat.divInt_eq_div,\n  ← mk_eq_divInt _ _ (Int.natAbs_ne_zero.mpr {unknown}.ne') {unknown}]",
  2),
 ("rw [← ofInt_eq_cast, ofInt, mk_eq_divInt, Rat.inv_def', divInt_eq_div, Nat.cast_one]",
  2),
 ('rw [Int.natAbs_one]', 2),
 ('exact Nat.coprime_one_left _', 2),
 ('simp only [lt_iff_le_not_le]', 2),
 ('simp [div_num_den, Rat.le_def {unknown} {unknown}]', 2),
 ('apply not_congr', 2),
 ("rw [num_den'] at {unknown}", 2),
 ('rw [divInt]', 2),
 ('have := den_nz {unknown}', 2),
 ('rw [coe_int_eq_divInt]', 2),
 ('unfold Rat.inv', 2),
 ('by_cases hi : {unknown} = {unknown}', 2),
 ('rw [{unknown}, RingHom.comp_assoc]', 2),
 ('simp only [RingEquiv.toRingHom_eq_coe, RingEquiv.symm_apply_apply, Function.comp_apply,\n  RingHom.coe_comp, RingEquiv.coe_toRingHom]',
  2),
 ('induction {unknown} using recTopCoe', 2),
 ('exact toReal_nonneg', 2),
 ('obtain h | h := le_total {variable} {variable}', 2),
 ('lift {function} to {unknown} → ℝ≥0 using {hypothesis}', 2),
 ('have hf : ∀ i, ((↑) : {set} → ℝ≥0∞) i ≠ ∞ := fun ⟨{unknown}, rs⟩ => {hypothesis} {unknown} rs',
  2),
 ('rcases lt_iff_exists_coe.1 {unknown} with ⟨{unknown}, rfl, _⟩', 2),
 ('rw [coe_lt_coe] at {unknown}', 2),
 ('by_contra! hlt', 2),
 ("refine' ⟨ball_image_iff.2 fun x hx => ball_image_iff.2 fun y hy z hz => _⟩",
  2),
 ('rintro {unknown} rfl', 2),
 ('intro {unknown} {unknown} h', 2),
 ("have A : {unknown} ≠ 0 := by simpa only [Ne.def, coe_eq_zero] using (zero_lt_one.trans {unknown}).ne'",
  2),
 ('· rwa [← ENNReal.coe_zpow {unknown}, ENNReal.coe_le_coe]', 2),
 ('· rwa [← ENNReal.coe_zpow {unknown}, ENNReal.coe_lt_coe]', 2),
 ("simpa only [Ne.def, coe_eq_zero] using (zero_lt_one.trans {unknown}).ne'",
  2),
 ('rwa [← ENNReal.coe_zpow {unknown}, ENNReal.coe_le_coe]', 2),
 ('rwa [← ENNReal.coe_zpow {unknown}, ENNReal.coe_lt_coe]', 2),
 ("induction' {unknown} using LaurentPolynomial.induction_on_mul_T with {unknown} n",
  2),
 ('rw [← single_eq_C_mul_T]', 2),
 ('by_cases a0 : {variable} = 0', 2),
 ('exact ⟨1, rfl⟩', 2),
 ("refine' Iff.trans _ Quotient.eq", 2),
 ('rw [Quotient.out_eq {unknown}]', 2),
 ('· exact le_add_left card_support_C_mul_X_pow_le_one', 2),
 ('· exact le_trans (natDegree_C_mul_X_pow_le {unknown}.leadingCoeff {unknown}.natDegree) {unknown}',
  2),
 ('· exact Nat.lt_succ_iff.mp (gt_of_ge_of_gt {unknown} (eraseLead_support_card_lt {hypothesis}))',
  2),
 ('· exact le_trans eraseLead_natDegree_le_aux {unknown}', 2),
 ('exact le_add_left card_support_C_mul_X_pow_le_one', 2),
 ('exact le_trans (natDegree_C_mul_X_pow_le {unknown}.leadingCoeff {unknown}.natDegree) {unknown}',
  2),
 ('exact Nat.lt_succ_iff.mp (gt_of_ge_of_gt {unknown} (eraseLead_support_card_lt {hypothesis}))',
  2),
 ('exact le_trans eraseLead_natDegree_le_aux {unknown}', 2),
 ('apply natTrailingDegree_le_of_ne_zero', 2),
 ('rw [← le_zero_iff]', 2),
 ('apply subset_span', 2),
 ('by_cases nd : {unknown} = {unknown}.natDegree', 2),
 ('simpa using (coeff_eq_zero_of_natDegree_lt {unknown}).symm', 2),
 ('rw [eraseLead_coeff, coeff_add, coeff_add, eraseLead_coeff, if_neg, if_neg {hypothesis}]',
  2),
 ('by_contra h₂', 2),
 ('have := natDegree_pos_of_nextCoeff_ne_zero {unknown}', 2),
 ('rw [Fin.strictMono_castSucc.injective.extend_apply]', 2),
 ('exact {unknown}.succ_ne_zero', 2),
 ('all_goals exact {hypothesis}', 2),
 ('simp only [coe_mapRingHom, lifts, Set.mem_range, RingHom.mem_rangeS]', 2),
 ('simp only [coe_mapRingHom, map_C, Set.mem_univ, Subsemiring.coe_top, eq_self_iff_true,\n  and_self_iff]',
  2),
 ('obtain ⟨r, rfl⟩ := Set.mem_range.1 {hypothesis}', 2),
 ('intro habs', 2),
 ('simp only [{unknown}, RingHom.map_zero] at {unknown}', 2),
 ('exact {hypothesis} {unknown}.symm', 2),
 ('exact degree_erase_lt {unknown}', 2),
 ('rw [← leadingCoeff, Ne.def, leadingCoeff_eq_zero]', 2),
 ('have := congr_arg degree {unknown}', 2),
 ('by_cases hp : {unknown}.map {function} = 0', 2),
 ('rw [← {function}] at {hypothesis}', 2),
 ('rw [Finset.forall_mem_insert] at {hypothesis} ⊢', 2),
 ('set m := {unknown}.rootMultiplicity {variable}', 2),
 ('exact Submonoid.one_mem _', 2),
 ('linarith [{unknown}]', 2),
 ('rw [degree_mul, degree_C {unknown}, add_zero]', 2),
 ('simp [*, le_refl]', 2),
 ('unfold divModByMonicAux', 2),
 ('rwa [degree_mul_leadingCoeff_inv {unknown} {unknown}]', 2),
 ('rw [{unknown}, Polynomial.eval₂_mul, {unknown}, zero_mul]', 2),
 ('by_cases H : {unknown} = 0', 2),
 ('by_cases H : {variable} = 0', 2),
 ('rw [{unknown}, C_0]', 2),
 ('contrapose! {unknown} with h', 2),
 ('· rw [← tsub_add_cancel_of_le {hypothesis}, coeff_mul_X_pow, add_tsub_cancel_right]',
  2),
 ("· refine' (coeff_mul _ _ _).trans (Finset.sum_eq_zero fun x hx => _)\n  rw [coeff_X_pow, if_neg, mul_zero]\n  exact ((le_of_add_le_right (mem_antidiagonal.mp hx).le).trans_lt <| not_le.mp {hypothesis}).ne",
  2),
 ('rw [← tsub_add_cancel_of_le {unknown}, coeff_mul_X_pow, add_tsub_cancel_right]',
  2),
 ('rw [coeff_X_pow, if_neg, mul_zero]', 2),
 ('exact ((le_of_add_le_right (mem_antidiagonal.mp {hypothesis}).le).trans_lt <| not_le.mp {hypothesis}).ne',
  2),
 ('· rintro ⟨{unknown}, rfl⟩ c\n  rw [coeff_C_mul]\n  apply dvd_mul_right', 2),
 ('rw [coeff_C_mul]', 2),
 ('choose c hc using {hypothesis}', 2),
 ('rwa [mul_zero]', 2),
 ('simp [AlgHomClass.commutes]', 2),
 ('simp_rw [Algebra.smul_def]', 2),
 ('rw [not_mem_support_iff] at {hypothesis}', 2),
 ('rw [not_mem_support_iff.1 {hypothesis}]', 2),
 ('intro k _', 2),
 ('exact Nat.mul_le_mul_right {unknown} {unknown}', 2),
 ("refine' le_degree_of_ne_zero _", 2),
 ('rwa [{unknown}, isUnit_C]', 2),
 ('rintro rfl rfl rfl', 2),
 ('rw [eval₂_mul {function} {variable}]', 2),
 ('refine (degree_le_iff_coeff_zero _ _).2 fun {unknown} hm => ?_', 2),
 ('by_cases hq : {unknown}.Monic', 2),
 ('· simp_rw [modByMonic_eq_of_not_monic _ {hypothesis}]', 2),
 ('simp_rw [modByMonic_eq_of_not_monic _ {hypothesis}]', 2),
 ('rw [{unknown}, natDegree_one, eq_comm, add_eq_zero_iff] at {unknown}', 2),
 ('apply one_dvd', 2),
 ('simp [C_eq_algebraMap]', 2),
 ("by_contra hx'", 2),
 ('rw [le_rootMultiplicity_iff {unknown}, pow_add]', 2),
 ('rw [le_rootMultiplicity_iff {unknown}]', 2),
 ('exact degree_C_le', 2),
 ('simp only [Nat.zero_eq, nthRootsFinset_zero, not_mem_empty] at {hypothesis}',
  2),
 ('exact NoZeroSMulDivisors.algebraMap_injective {type} {type}', 2),
 ('let ⟨u, hu⟩ := {unknown}', 2),
 ('rwa [Units.mul_right_dvd]', 2),
 ('obtain ⟨r, hr⟩ := {unknown}', 2),
 ('obtain rfl | hq := eq_or_ne {unknown} 0', 2),
 ('simp [{unknown}, {unknown}] at {unknown}', 2),
 ('rw [{unknown}.leadingCoeff, C_1, one_mul]', 2),
 ('rw [go_eq]', 2),
 ('by_cases h : (1 < {unknown} ∧ 0 < {unknown} ∧ {unknown} % {unknown} = 0)',
  2),
 ('have : {unknown} / {unknown} < {unknown} := by apply Nat.div_lt_self <;> aesop',
  2),
 ('apply go_succ', 2),
 ('apply Nat.div_lt_self <;> aesop', 2),
 ('dsimp [maxPowDiv]', 2),
 ('rw [{unknown}, natDegree_zero]', 2),
 ('rw [natDegree_C {variable}]', 2),
 ('rw [← C_1, ← {unknown}, RingHom.map_mul, ← mul_assoc]', 2),
 ("refine' mem_support_iff.mpr _", 2),
 ('rw [coeff_eq_zero_of_natDegree_lt {unknown}, mul_zero]', 2),
 ("induction' {unknown} using Finset.induction_on with x {unknown} hx IH", 2),
 ('rcases {unknown}.eq_empty_or_nonempty with (rfl | hs)', 2),
 ('rw [{function}, {function}] at {unknown}', 2),
 ('· simp [{hypothesis}, {function}]', 2),
 ("by_cases hb' : {function} {variable} = 0", 2),
 ('rw [← natDegree_neg] at {unknown}', 2),
 ('rw [← {unknown}, coeff_natDegree, leadingCoeff_eq_zero] at {unknown}', 2),
 ('rintro ⟨t, rfl⟩', 2),
 ('simpa only [pow_two] using {unknown}', 2),
 ('rw [log]', 2),
 ("induction' {unknown} using Nat.strong_induction_on with {unknown} ih generalizing {unknown}",
  2),
 ('rcases eq_or_ne {unknown} 0 with (rfl | hm)', 2),
 ('· rw [log_of_left_le_one {unknown}]\n  exact zero_le _', 2),
 ('rw [log_of_left_le_one {unknown}]', 2),
 ('rw [lt_iff_le_not_le, not_le_iff_exists]', 2),
 ('· exact (not_lt_bot {unknown}).elim', 2),
 ('intro _ H', 2),
 ('by_cases hp : {unknown}.natDegree = 0', 2),
 ('simp only [Nat.succ_ne_zero, false_or_iff]', 2),
 ('rw [add_comm, add_assoc]', 2),
 ('obtain ⟨{unknown}, rfl⟩ := Nat.exists_eq_succ_of_ne_zero {unknown}', 2),
 ('rwa [one_mul]', 2),
 ('haveI := Nontrivial.of_polynomial_ne {hypothesis}', 2),
 ('have : ¬degree {unknown} ≤ degree {unknown} := not_le_of_gt {unknown}', 2),
 ('rw [dif_pos {unknown}, if_neg (mt And.left {hypothesis})]', 2),
 ('unfold modByMonic divByMonic divModByMonicAux', 2),
 ('unfold divByMonic divModByMonicAux', 2),
 ('exact ⟨{unknown}.1.symm, {unknown}.2.symm⟩', 2),
 ('rw [{unknown}, Polynomial.map_zero]', 2),
 ('rw [degree_X_sub_C]', 2),
 ('cases Subsingleton.elim ‹DecidableEq {type}› (Classical.decEq {type})', 2),
 ("rcases eq_or_ne {unknown} 0 with (rfl | hn')", 2),
 ('by_cases hp : {unknown}.Prime', 2),
 ('· subst {unknown}\n  contradiction', 2),
 ('apply (Nat.eq_or_lt_of_le {unknown}).resolve_left', 2),
 ('intro me', 2),
 ('simp [{unknown}, left_distrib]', 2),
 ('rcases Nat.eq_zero_or_pos {unknown} with n0 | n0', 2),
 ('· subst {unknown}\n  cases {hypothesis} (by decide)', 2),
 ("refine' fun {unknown} pp dp => succ_le_of_lt (lt_of_le_of_ne pp.two_le _)",
  2),
 ('cases {hypothesis} (by decide)', 2),
 ('rw [Ne.def, Multiset.prod_eq_zero_iff]', 2),
 ('· intro z hz\n  apply irreducible_of_normalized_factor z\n  rw [← Multiset.mem_toFinset]\n  apply {unknown} hz',
  2),
 ('apply irreducible_of_normalized_factor {unknown}', 2),
 ('rw [← Multiset.mem_toFinset]', 2),
 ('exact ⟨{unknown}, {unknown}, {unknown}, {unknown}⟩', 2),
 ("rw [leadingCoeff_mul' _] <;> simp [leadingCoeff_C {variable}, {unknown}]",
  2),
 ('rw [Monic.def] at {hypothesis} ⊢', 2),
 ('rwa [degree_neg]', 2),
 ('· simpa [{hypothesis}]', 2),
 ('rw [← {unknown}.degree_le_zero_iff_eq_one, not_le] at {hypothesis}', 2),
 ("refine' (lt_trans _ {unknown}).ne'", 2),
 ('intro q r h', 2),
 ('rw [degree_le_iff_coeff_zero]', 2),
 ("refine' sum_congr rfl fun x _ => _", 2),
 ("refine' monic_mul_C_of_leadingCoeff_mul_eq_one _", 2),
 ('rw [Fintype.card_ofFinset, card_Iic]', 2),
 ('rw [Fintype.card_ofFinset, card_Iio]', 2),
 ('rw [mem_Ico] at {hypothesis} ⊢', 2),
 ('rintro k hk l hl (hkl : k % {unknown} = l % {unknown})', 2),
 ('by_contra ha', 2),
 ('rw [add_mod_right] at {unknown}', 2),
 ('simp only [lt_add_iff_pos_right, Set.left_mem_Ico, Finset.coe_Ico, {unknown}]',
  2),
 ('intro hia', 2),
 ('obtain hi | hi := lt_or_le {unknown} ({unknown} % {unknown})', 2),
 ('rintro ⟨{unknown}, _, rfl⟩', 2),
 ("refine' ⟨{unknown} + {unknown} * ({unknown} / {unknown} + 1), ⟨_, _⟩, _⟩",
  2),
 ("· refine' lt_of_lt_of_le (add_lt_add_right {unknown} ({unknown} * ({unknown} / {unknown} + 1))) _\n  rw [mul_add, mul_one, ← add_assoc, {unknown}]",
  2),
 ('· rw [Nat.add_mul_mod_self_left, Nat.mod_eq_of_lt {unknown}]', 2),
 ('rw [add_comm ({unknown} / {unknown}), mul_add, mul_one, ← add_assoc]', 2),
 ("refine' {unknown}.symm.le.trans (add_le_add_right _ _)", 2),
 ("refine' lt_of_lt_of_le (add_lt_add_right {unknown} ({unknown} * ({unknown} / {unknown} + 1))) _",
  2),
 ('rw [mul_add, mul_one, ← add_assoc, {unknown}]', 2),
 ('rw [Nat.add_mul_mod_self_left, Nat.mod_eq_of_lt {unknown}]', 2),
 ("refine' ⟨{unknown} + {unknown} * ({unknown} / {unknown}), ⟨_, _⟩, _⟩", 2),
 ('· exact {unknown}.symm.le.trans (add_le_add_right {unknown} _)', 2),
 ('exact {unknown}.symm.le.trans (add_le_add_right {unknown} _)', 2),
 ('simp only [zero_le, le_add_iff_nonneg_left]', 2),
 ('rw [Finset.disjoint_left]', 2),
 ('exact (self_le_add_right _ _).not_lt {unknown}', 2),
 ('by_cases h : {function} {unknown} <;> simp [h, count_succ]', 2),
 ('wlog hmn : {unknown} < {unknown}', 2),
 ('rw [modEq_iff_dvd] at *', 2),
 ('rw [Int.ofNat_sub {unknown}]', 2),
 ('rw [modEq_iff_dvd]', 2),
 ('rcases {unknown}.eq_zero_or_pos with (rfl | hm)', 2),
 ('rw [succ_le_iff]', 2),
 ('rw [leRecOn_self]', 2),
 ('intro x y H', 2),
 ('rw [leRecOn_succ]', 2),
 ('exact support_add', 2),
 ('rw [Finsupp.single_apply]', 2),
 ('rw [coeff_C, if_neg {unknown}]', 2),
 ('rw [← C_mul_X_pow_eq_monomial, pow_one]', 2),
 ('apply top_unique', 2),
 ('exact {hypothesis} {unknown} {variable}', 2),
 ('rw [← ofFinsupp_single, support]', 2),
 ('rw [Nat.mod_two_of_bodd]', 2),
 ('· rintro ⟨rfl, rfl⟩\n  rfl', 2),
 ('rw [binaryRec]', 2),
 ('rw [binaryRec_zero]', 2),
 ('rw [bodd_bit, div2_bit]', 2),
 ("induction' {unknown} using Nat.binaryRec' with b {unknown} h _", 2),
 ('simp [pow_succ]', 2),
 ('exact List.mem_cons_of_mem _ {hypothesis}', 2),
 ('exact {hypothesis} {unknown} (List.mem_cons_self _ _)', 2),
 ('rcases {unknown} with rfl', 2),
 ('intro {unknown} h', 2),
 ('rw [Nat.mod_add_div]', 2),
 ('rw [ofDigits_digits]', 2),
 ('rw [List.getLast_cons]', 2),
 ("induction' {unknown} using Nat.strong_induction_on with {unknown} IH", 2),
 ('rw [← pos_iff_ne_zero]', 2),
 ('rcases Decidable.eq_or_ne {unknown} 0 with (rfl | hn)', 2),
 ("induction' {unknown} with {unknown} hi", 2),
 ('exact (ofDigits_digits {unknown} {unknown}).symm', 2),
 ('obtain h | rfl | h : 1 < {unknown} ∨ 1 = {unknown} ∨ {unknown} < 1 := trichotomous 1 {unknown}',
  2),
 ('simp [lt_one_iff.mp {unknown}]', 2),
 ('rcases eq_or_ne {unknown} 0 with rfl | hn', 2),
 ('all_goals simp', 2),
 ('rw [← ofDigits_one]', 2),
 ('conv =>\n  congr\n  · skip\n  · rw [← ofDigits_digits {unknown} {unknown}]',
  2),
 ("refine' ⟨_, {unknown}, {unknown}⟩", 2),
 ('exact {unknown}.symm ▸ {unknown}', 2),
 ('obtain {unknown} | {unknown} := {unknown}.symm.lt_or_lt', 2),
 ('simp only [Subtype.coe_mk, Subtype.coe_eta]', 2),
 ("· simp [{unknown}.ne']", 2),
 ('apply gcd_dvd_right', 2),
 ('rcases {unknown} with ⟨_, rfl⟩', 2),
 ('rwa [succ_le_iff]', 2),
 ('rw [{hypothesis}, {hypothesis}]', 2),
 ('apply mul_mul_mul_comm', 2),
 ('exact Or.inr ⟨{unknown}, {unknown}.symm⟩', 2),
 ('· simp [Function.update_noteq {unknown}.symm, {unknown}, erase_ne, {unknown}.symm]',
  2),
 ('simp [Function.update_noteq {unknown}.symm, {unknown}, erase_ne, {unknown}.symm]',
  2),
 ('· subst {hypothesis}\n  simp [{unknown}]', 2),
 ('simp [Set.subset_def]', 2),
 ('split_ifs with hb', 2),
 ('intro i y hy', 2),
 ('exact ⟨DFinsupp.single {variable} ⟨{variable}, {hypothesis}⟩, DFinsupp.sumAddHom_single _ _ _⟩',
  2),
 ("refine' iSup₂_le fun i hi y hy => ⟨DFinsupp.single i ⟨y, hy⟩, _⟩", 2),
 ('by_cases hp : {function} {unknown}', 2),
 ('exact Finset.sum_comm', 2),
 ('rw [liftAddHom_symm_apply, AddMonoidHom.comp_assoc, liftAddHom_comp_single]',
  2),
 ('· intros; dsimp; simp only [map_zero]', 2),
 ('rw [← one_pow {unknown}]', 2),
 ("simp only [pow_succ']", 2),
 ('rw [← pow_one {unknown}]', 2),
 ('rw [← one_shiftLeft]', 2),
 ('apply binaryRec _ _ {unknown}', 2),
 ('· apply {hypothesis} {hypothesis}', 2),
 ('by_cases h₁ : {unknown} < {unknown} <;> simp [pair, h₁, add_assoc]', 2),
 ('by_cases h₂ : {unknown} < {unknown} <;> simp [pair, h₂, {unknown}]', 2),
 ('simp? at {hypothesis} says simp only [not_lt] at {hypothesis}', 2),
 ('exact mul_self_lt_mul_self {unknown}', 2),
 ('exact le_add_self', 2),
 ('exact congr_arg _ (succ_iterate {unknown} {unknown})', 2),
 ('exact congr_arg _ (pred_iterate {unknown} {unknown})', 2),
 ('rw [not_forall] at {hypothesis}', 2),
 ("cases' {hypothesis} with hx' {hypothesis}", 2),
 ('· apply le_top', 2),
 ('rcases ne_top_iff.mp (ne_top_of_lt {unknown}) with ⟨m, rfl⟩', 2),
 ('rcases ne_top_iff.mp {unknown} with ⟨m, rfl⟩', 2),
 ('apply natCast_lt_top', 2),
 ('apply Nat.lt_succ_of_le', 2),
 ('exact lt_of_le_of_lt {unknown} {unknown}', 2),
 ('· norm_cast', 2),
 ('rw [one_div]', 2),
 ('exact Nat.cast_add_one_pos _', 2),
 ('exact le_trailingDegree_of_ne_zero {unknown}', 2),
 ('rw [trailingDegree_eq_natTrailingDegree {hypothesis}]', 2),
 ('rw [C_mul_X_pow_eq_monomial]', 2),
 ('rw [mem_support_iff, coeff_mul] at {hypothesis}', 2),
 ('rw [coeff_mul]', 2),
 ('rw [not_lt] at {hypothesis} {hypothesis}', 2),
 ('have hp : {unknown} ≠ 0 := fun hp => {unknown} (by rw [hp, trailingCoeff_zero, zero_mul])',
  2),
 ('have hq : {unknown} ≠ 0 := fun hq => {unknown} (by rw [hq, trailingCoeff_zero, mul_zero])',
  2),
 ('rw [{unknown}, trailingCoeff_zero, zero_mul]', 2),
 ('rw [{unknown}, trailingCoeff_zero, mul_zero]', 2),
 ('rw [zero_dvd_iff] at {unknown}', 2),
 ('revert {unknown} {unknown} {unknown}', 2),
 ('by_cases n0 : {unknown} = 0', 2),
 ('simp [Nat.not_prime_one]', 2),
 ('apply not_mem_empty', 2),
 ('rw [not_odd_iff, even_iff]', 2),
 ('rcases even_or_odd {unknown} with (⟨k, rfl⟩ | ⟨k, rfl⟩) <;> use k', 2),
 ('rw [even_add, even_iff_not_odd, even_iff_not_odd, not_iff_not]', 2),
 ('simp [← Nat.even_iff, ← Nat.not_even_iff, parity_simps]', 2),
 ('rw [add_comm, odd_add]', 2),
 ('rcases {unknown} with ⟨m, rfl⟩', 2),
 ('dsimp at {unknown} ⊢', 2),
 ('unfold sqrt.iter', 2),
 ('· exact zero_lt_two', 2),
 ('exact zero_lt_two', 2),
 ('unfold natDegree', 2),
 ('exact le_degree_of_ne_zero {unknown}', 2),
 ('rw [degree_eq_natDegree {hypothesis}]', 2),
 ('exact WithBot.bot_lt_coe {unknown}', 2),
 ('simp only [C_mul_X_pow_eq_monomial]', 2),
 ('simp [coeff_C]', 2),
 ('simp [mul_sub]', 2),
 ('intro hq', 2),
 ('obtain _ | {unknown} := {unknown}', 2),
 ('rw [mem_support_iff]', 2),
 ('exact {unknown}.trans {unknown}.symm', 2),
 ('rwa [{unknown}, Nat.pos_iff_ne_zero]', 2),
 ('rw [succ_nsmul]', 2),
 ('have h₁ : leadingCoeff {unknown} ^ {unknown} ≠ 0 := fun h₁ => {unknown} <| by rw [pow_succ, h₁, mul_zero]',
  2),
 ('rw [pow_succ, {unknown}, mul_zero]', 2),
 ('rw [Nat.cast_add]', 2),
 ('rw [mem_antidiagonal]', 2),
 ('rw [← {unknown}.degree_neg] at {unknown}', 2),
 ('rw [← degree_neg {unknown}] at {unknown}', 2),
 ('simp [map_zero {variable}]', 2),
 ('rintro (_ | n)', 2),
 ('exact absurd {unknown}.1 (by decide)', 2),
 ('simp [Coprime]', 2),
 ('convert one_mul {unknown}', 2),
 ('exact ⟨{unknown}, {unknown}, {unknown}, {unknown}, rfl⟩', 2),
 ('exact mul_dvd_mul {unknown} {unknown}', 2),
 ("refine' ⟨fun h => _, fun h => pow_dvd_pow_of_dvd h _⟩", 2),
 ('rw [← Ne.def, ← nonempty_iff_ne_empty]', 2),
 ('simp [Nat.eq_zero_of_le_zero {unknown}]', 2),
 ('rw [← factorial_lt <| pos_of_gt {unknown}, {unknown}] at {unknown}', 2),
 ('cases lt_irrefl _ {unknown}', 2),
 ('rw [← add_assoc]', 2),
 ("rw [pow_succ']", 2),
 ('rw [ascFactorial_zero, pow_zero]', 2),
 ('rw [ascFactorial_zero, descFactorial_zero]', 2),
 ('rw [descFactorial_zero, pow_zero]', 2),
 ('rw [succ_sub_succ]', 2),
 ('let m := minFac ({unknown} + 2)', 2),
 ('have : ({unknown} + 2) / {function} < ({unknown} + 2) := factors_lemma', 2),
 ('rw [zero_mul] at {unknown}', 2),
 ('simp_rw [← prime_iff]', 2),
 ('rcases {unknown}.eq_zero_or_pos with (rfl | ha)', 2),
 ('· simp [(coprime_zero_left _).mp {unknown}]', 2),
 ('· simp [(coprime_zero_right _).mp {unknown}]', 2),
 ('simp [(coprime_zero_left _).mp {unknown}]', 2),
 ('simp [(coprime_zero_right _).mp {unknown}]', 2),
 ('· exact dvd_zero _', 2),
 ('simp [factorial_succ, mul_comm, mul_left_comm, mul_assoc]', 2),
 ('rw [← choose_mul_factorial_mul_factorial {unknown}, mul_assoc]', 2),
 ('exact dvd_mul_left _ _', 2),
 ("exact (Nat.mul_div_cancel' <| factorial_dvd_ascFactorial _ _).symm", 2),
 ('conv_lhs => unfold bitwise', 2),
 ('cases bodd {unknown} <;> rfl', 2),
 ('rw [← bit_ne_zero_iff] at {function}', 2),
 ('simpa using {hypothesis} 0', 2),
 ('· exact False.elim ({unknown} rfl)', 2),
 ('exact False.elim ({unknown} rfl)', 2),
 ('by_cases h : {unknown} = {unknown}', 2),
 ('· cases {hypothesis}\n  simp', 2),
 ('· simp [{unknown}, testBit_two_pow_of_ne {unknown}]', 2),
 ('simp [{unknown}, testBit_two_pow_of_ne {unknown}]', 2),
 ('split <;> assumption', 2),
 ('simpa [mul_assoc] using {unknown}', 2),
 ('rw [← image_prod]', 2),
 ('rw [image2_swap {function}, image2_swap {function}]', 2),
 ('exact image2_assoc fun _ _ _ => {hypothesis} _ _ _', 2),
 ('exact mem_image2_of_mem (mem_image2_of_mem {hypothesis} {hypothesis}) (mem_image2_of_mem {hypothesis} {hypothesis})',
  2),
 ('exact image2_inter_union_subset {hypothesis}', 2),
 ('unfold boolIndicator', 2),
 ("cases' Set.mem_iUnion.1 {unknown}.prop with {function} hi", 2),
 ('apply countable_range', 2),
 ('lift {set} to Set {set} using {unknown}', 2),
 ('rcases le_total {unknown} {unknown} with H | H <;> simp [H]', 2),
 ('rw [le_iff_lt_or_eq, lt_add_one_iff]', 2),
 ('obtain ⟨_, rfl⟩ := {unknown}', 2),
 ('rcases Nat.eq_zero_or_pos {unknown} with (rfl | hk)', 2),
 ('exact lt_irrefl 0 {unknown}', 2),
 ('rintro ⟨hle, hP, hm⟩', 2),
 ('rw [{unknown}.factors_pow, List.subperm_ext_iff]', 2),
 ('by_cases pp : {unknown}.Prime', 2),
 ('rw [← factorization_le_iff_dvd {unknown} {unknown}]', 2),
 ('rcases eq_or_ne {unknown} {unknown} with (rfl | hqp)', 2),
 ('exact (factorization_le_iff_dvd {unknown} {unknown}).2 {unknown} {unknown}',
  2),
 ('simpa [{hypothesis}] using {hypothesis} {unknown}', 2),
 ('rw [← factorization_le_iff_dvd {unknown} {unknown}, {unknown}]', 2),
 ('· rw [smul_zero, map_zero, smul_zero]', 2),
 ("induction' {unknown} using Polynomial.induction_on' with _ _ e₁ e₂", 2),
 ('rw [smul_zero, map_zero, smul_zero]', 2),
 ('rw [smul_add, map_add, {unknown}, {unknown}, map_add, smul_add]', 2),
 ('rw [comp_apply]', 2),
 ("simp [{unknown}, {unknown}.ne']", 2),
 ('rintro ⟨{variable}, {unknown}, hp, hk, rfl⟩', 2),
 ('· simp only [Nat.coprime_zero_left] at {unknown}\n  simp [{unknown}, Finset.filter_singleton, not_isPrimePow_one]',
  2),
 ('· simp only [Nat.coprime_zero_right] at {unknown}\n  simp [{unknown}, Finset.filter_singleton, not_isPrimePow_one]',
  2),
 ('simp only [Nat.coprime_zero_left] at {unknown}', 2),
 ('simp only [Nat.coprime_zero_right] at {unknown}', 2),
 ('rcases exists_add_of_le {unknown} with ⟨{unknown}, rfl⟩', 2),
 ('exact succ_pos {unknown}', 2),
 ('rw [fastFibAux, binaryRec_eq]', 2),
 ('rw [{unknown}, add_assoc]', 2),
 ('rcases {hypothesis} with ⟨w, hw⟩', 2),
 ('· rw [succ_eq_iff_isMax.2 {hypothesis}] at {unknown} ⊢\n  exact {function} (le_trans (le_succ _) {unknown}) {unknown}',
  2),
 ('rw [succ_eq_iff_isMax.2 {unknown}] at {unknown} ⊢', 2),
 ('exact {function} (le_trans (le_succ _) {unknown}) {unknown}', 2),
 ('simp [{function}, Subtype.ext_iff]', 2),
 ('simpa [{function}]', 2),
 ('exact ⟨{variable} + {variable}, {unknown}, by rw [add_right_comm]⟩', 2),
 ('rcases {unknown}.exists_between {hypothesis} with ⟨y, hys, _, hyx⟩', 2),
 ('apply mem_iUnion₂.mpr', 2),
 ('simp [prod_eq]', 2),
 ('apply range_comp_subset_range', 2),
 ('simp [subset_def]', 2),
 ('· simp [{set}, prod_eq_empty_iff.1 {set}]', 2),
 ('simp [{unknown}, prod_eq_empty_iff.1 {unknown}]', 2),
 ('· rintro ⟨{unknown}, hx, rfl⟩\n  exact ⟨rfl, hx, hx⟩', 2),
 ('exact ⟨rfl, {hypothesis}, {hypothesis}⟩', 2),
 ('rintro ⟨rfl : {variable} = {variable}, h2x⟩', 2),
 ('rintro h3', 2),
 ('simp [Classical.skolem, Set.Nonempty]', 2),
 ('simp [funext_iff]', 2),
 ('· exact ⟨{hypothesis}.1 {variable} {hypothesis}, {hypothesis}.2 {variable} {hypothesis}⟩',
  2),
 ('exact ⟨{hypothesis}.1 {variable} {hypothesis}, {hypothesis}.2 {variable} {hypothesis}⟩',
  2),
 ('exact fun h => {hypothesis} (h ▸ {hypothesis})', 2),
 ('obtain rfl | h := eq_or_ne {variable} {variable}', 2),
 ('simp_rw [mem_univ_pi]', 2),
 ("refine' forall_congr' fun {variable} => _", 2),
 ('cases ({unknown} : ℕ) % ({unknown} : ℕ) with\n| zero =>\n  rw [if_pos rfl]\n  rfl\n| succ n =>\n  rw [if_neg n.succ_ne_zero]\n  rfl',
  2)
]

In [11]:
from collections import defaultdict

unknown_commands = defaultdict(int)

for tac, freq in tac_templates:
    if tac.startswith('·') or tac.startswith('.'):
        tac = tac[1:].strip()
    command = tac.split(' ')[0]
    while len(command) > 0 and not command[-1].isalnum():
        command = command[:-1]
    if command not in lean_tactics:
        unknown_commands[command] += 1
        print(tac)
        
sorted_commands = sorted(unknown_commands.items(), key=lambda item: item[1], reverse=True)
[x for x,y in sorted_commands]

congr
decide
gcongr
simpa using {unknown}
simpa
congr 1
dsimp
simpa using {hypothesis}
positivity
right
symm
aesop
infer_instance
abel
nontriviality {type}
dsimp only
split_ifs with h
push_cast
field_simp
exfalso
ext1 x
convert {unknown}
aesop_cat
by_contra h
split_ifs
rwa [{unknown}]
subst_vars
ext1
ac_rfl
borelize {type}
lift {unknown} to ℝ≥0 using {unknown}
simpa using {unknown}
inhabit {type}
simpa [{unknown}] using {unknown}
simpa using {function}
by_contra! h
convert {unknown} using 1
haveI := Classical.decEq {type}
haveI := {unknown}
simpa only [sub_eq_add_neg] using {unknown}.add {unknown}.neg
letI := Classical.decEq {type}
continuity
ring_nf
congr 2
haveI := Fact.mk {unknown}
rwa [{unknown}] at {unknown}
rwa [← {unknown}]
nlinarith
convert {hypothesis}
tfae_finish
dsimp at {unknown}
ext1 {unknown}
simpa
simp_rw [{unknown}]
isBoundedDefault
filter_upwards [self_mem_nhdsWithin]
rotate_left
funext x y
simp_rw [{hypothesis}]
haveI := Fintype.ofFinite {type}
funext x
lift {function

['obtain',
 'simpa',
 'rwa',
 'simp_rw',
 'haveI',
 'by_contra',
 'lift',
 'letI',
 'dsimp',
 'split_ifs',
 'ext1',
 'convert',
 'tfae_have',
 'funext',
 'congr',
 'filter_upwards',
 'choose',
 'field_simp',
 'aesop',
 'nth_rw',
 'conv_lhs',
 'simp_arith',
 'erw',
 'delta',
 'gcongr',
 'positivity',
 'right',
 'infer_instance',
 'abel',
 'nontriviality',
 'push_cast',
 'borelize',
 'inhabit',
 'ring_nf',
 'nlinarith',
 'fin_cases',
 'trans',
 'measurability',
 'exact_mod_cast',
 'rename_i',
 'calc',
 'decide',
 'symm',
 'exfalso',
 'aesop_cat',
 'subst_vars',
 'ac_rfl',
 'continuity',
 'tfae_finish',
 'isBoundedDefault',
 'rotate_left',
 'classical',
 'fconstructor',
 'sorry',
 'clear_value',
 'conv_rhs',
 'next',
 'assumption_mod_cast',
 '(intro',
 'substs',
 'exists',
 'mono',
 'interval_cases',
 'show',
 'bitwise_assoc_tac',
 'triv',
 'mfld_set_tac',
 '',
 'beta_reduce',
 'abel_nf',
 'case',
 'simp_wf',
 'set',
 'wlog',
 'conv']